In [ ]:
%%writefile rps_meta_fix.py
code = compile(
    """
import random

RNA={'RR':'1','RP':'2','RS':'3','PR':'4','PP':'5','PS':'6','SR':'7','SP':'8','SS':'9'}
mix={'RR':'R','RP':'R','RS':'S','PR':'R','PP':'P','PS':'P','SR':'S','SP':'P','SS':'S'}
rot={'R':'P','P':'S','S':'R'}

if not input:
   DNA=[""]*3
   prin=[random.choice("RPS")]*18
   meta=[random.choice("RPS")]*6
   skor1=[[0]*18,[0]*18,[0]*18,[0]*18,[0]*18,[0]*18]
   skor2=[0]*6
else:
   for j in range(18):
       for i in range(4):
           skor1[i][j]*=0.8
       for i in range(4,6):
           skor1[i][j]*=0.5
       for i in range(0,6,2):
           skor1[i][j]-=(input==rot[rot[prin[j]]])
           skor1[i+1][j]-=(output==rot[rot[prin[j]]])
       for i in range(2,6,2):
           skor1[i][j]+=(input==prin[j])
           skor1[i+1][j]+=(output==prin[j])
       skor1[0][j]+=1.3*(input==prin[j])-0.3*(input==rot[prin[j]])
       skor1[1][j]+=1.3*(output==prin[j])-0.3*(output==rot[prin[j]])
   for i in range(6):
       skor2[i]=0.9*skor2[i]+(input==meta[i])-(input==rot[rot[meta[i]]])
   DNA[0]+=input
   DNA[1]+=output
   DNA[2]+=RNA[input+output]
   for i in range(3):
       j=min(21,len(DNA[2]))
       k=-1
       while j>1 and k<0:
             j-=1
             k=DNA[i].rfind(DNA[i][-j:],0,-1)
       prin[2*i]=DNA[0][j+k]
       prin[2*i+1]=rot[DNA[1][j+k]]
       k=DNA[i].rfind(DNA[i][-j:],0,j+k-1)
       prin[2*i]=mix[prin[2*i]+DNA[0][j+k]]
       prin[2*i+1]=mix[prin[2*i+1]+rot[DNA[1][j+k]]]
   for i in range(6,18):
       prin[i]=rot[prin[i-6]]
   for i in range(0,6,2):
       meta[i]=prin[skor1[i].index(max(skor1[i]))]
       meta[i+1]=rot[prin[skor1[i+1].index(max(skor1[i+1]))]]
output=rot[meta[skor2.index(max(skor2))]]
""", '<string>', 'exec')
gg = {}


def run(observation, configuration):
    global gg
    global code
    inp = ''
    try:
        inp = 'RPS'[observation.lastOpponentAction]
    except:
        pass
    gg['input'] = inp
    exec(code, gg)
    return {'R': 0, 'P': 1, 'S': 2}[gg['output']]


In [ ]:
%%writefile ensemble_v3.py
code_deepbot = compile(
    """
import random
from random import randint

class RAM:

    def __init__(self, addressSize, entrySize, addressing, decay=None, up=None):
        self.addressing = addressing
        self.decay = decay
        self.up = up
        self.ram = {}
        self.address = [ randint(0, entrySize-1) for x in range(addressSize) ]

    def _addressToIndex(self, entry):
        binCode = []
        for i in self.address:
            binCode.append(entry[i])
        return self.addressing(binCode)

    def _acumulateRam(self, index):
        if index not in self.ram:
            self.ram[index] = 0
        self.ram[index] += 1

    def _getValue(self, index):
        if index not in self.ram:
            return 0
        else:
            return self.ram[index]

    def train(self, entry, negative=False):
        index = self._addressToIndex(entry)
        if not negative:
            if self.up is None:
                self._acumulateRam(index)
            else:
                self.up(entry=entry, ram=self.ram, address=self.address, index=index)
        else:
            self.decay(entry=entry, ram=self.ram, address=self.address, index=index)

    def classify(self, entry):
        index = self._addressToIndex(entry)
        return self._getValue(index)


class Discriminator:

    def __init__(self, name, entrySize, addressSize, addressing, numberOfRAMS=None, decay=None, up=None):
        if numberOfRAMS is None:
            numberOfRAMS = int(entrySize/addressSize)
        self.rams = [ RAM(addressSize, entrySize, addressing, decay=decay, up=up) for x in range(numberOfRAMS) ]

    def train(self, entry, negative=False):
        for ram in self.rams:
            ram.train(entry, negative)

    def classify(self, entry):
        return [ ram.classify(entry) for ram in self.rams ]

class Addressing:
    def __call__(self, binCode): # binCode is a list of values of selected points of entry
        index = 0
        for i,e in enumerate(binCode):
            if e > 0:
                index += e*pow(3,i)
        return index


class WiSARD:
    def __init__(self,
            addressSize = 3,
            numberOfRAMS = None,
            bleachingActivated = True,
            seed = random.randint(0, 1000000),
            sizeOfEntry = None,
            classes = [],
            verbose = True,
            addressing = Addressing(),
            makeBleaching = None,
            decay = None,
            up = None):

        self.seed = seed
        self.decay = decay
        self.up = up
        random.seed(seed)

        if addressSize < 3:
            self.addressSize = 3
        else:
            self.addressSize = addressSize
        self.numberOfRAMS = numberOfRAMS
        self.discriminators = {}
        self.bleachingActivated = bleachingActivated
        self.addressing = addressing
        if makeBleaching is None:
            self.makeBleaching = self._makeBleaching
        else:
            self.makeBleaching = makeBleaching

        if sizeOfEntry is not None:
            for aclass in classes:
                self.discriminators[aclass] = Discriminator(
                    aclass, sizeOfEntry, self.addressSize,
                    self.addressing, self.numberOfRAMS, decay=self.decay, up=self.up)

    def _makeBleaching(self, discriminatorsoutput):
        bleaching = 0
        ambiguity = True
        biggestVote = 2
        while ambiguity and biggestVote > 1:
            bleaching += 1
            biggestVote = None
            ambiguity = False
            for key in discriminatorsoutput:
                discriminator = discriminatorsoutput[key]
                limit = lambda x: 1 if x >= bleaching else 0
                discriminator[1] = sum(map(limit, discriminator[0]))
                if biggestVote is None or discriminator[1] > biggestVote:
                    biggestVote = discriminator[1]
                    ambiguity = False
                elif discriminator[1] == biggestVote:
                    ambiguity = True
            if self.bleachingActivated:
                break

        return discriminatorsoutput

    def train(self, entries, classes):
        sizeOfEntry = len(entries[0])
        for i,entry in enumerate(entries):
            aclass = str(classes[i])
            if aclass not in self.discriminators:
                self.discriminators[aclass] = Discriminator(
                    aclass, sizeOfEntry, self.addressSize,
                    self.addressing, self.numberOfRAMS, decay=self.decay, up= self.up)
            self.discriminators[aclass].train(entry)
            if self.decay is not None:
                for key in self.discriminators:
                    if key != aclass:
                        self.discriminators[key].train(entry, negative=True)

    def classifyEntry(self, entry):
        discriminatorsoutput = {}
        for keyClass in self.discriminators:
            discriminatorsoutput[keyClass] = [self.discriminators[keyClass].classify(entry),0]

        discriminatorsoutput = self.makeBleaching(discriminatorsoutput)

        calc = lambda key: (key, float(discriminatorsoutput[key][1])/len(discriminatorsoutput[key][0]))
        classes = list(map(calc,discriminatorsoutput))
        classes.sort(key=lambda x: x[1], reverse=True)

        return classes


    def classify(self, entries):
        output=[]
        for i,entry in enumerate(entries):
            aclass = self.classifyEntry(entry)[0][0]
            output.append((entry, aclass))
        return output

class Decay:

    def __call__(self, **kwargs):
        if kwargs['index'] in kwargs['ram']:
            value = kwargs['ram'][kwargs['index']]
            kwargs['ram'][kwargs['index']] = 0.5*value - 0.1

class Up:

    def __call__(self, **kwargs):
        if kwargs['index'] not in kwargs['ram']:
            kwargs['ram'][kwargs['index']] = 1
        else:
            value = kwargs['ram'][kwargs['index']]
            kwargs['ram'][kwargs['index']] = 0.5*value + 1

class MakeBleaching:

    def __call__(self, discriminatorsoutput):
        for key in discriminatorsoutput:
            ramsoutput = discriminatorsoutput[key][0]
            discriminatorsoutput[key][1] = sum(ramsoutput)
        return discriminatorsoutput


if input == "":
    history = []
    correctly_predictions = []
    ai = WiSARD(4, classes=['R','P','S'], sizeOfEntry=16, decay=Decay(), up=Up(), makeBleaching=MakeBleaching())
    char2num = {"R":0, "P":1, "S":2}
    num2char = {0:"R", 1:"P", 2:"S"}
    defeat = {"R":"P", "P":"S", "S":"R"}
    output = random.choice(["R","P","S"])
    prediction = output
    counter = 1
    nonrandom_output = 'Q'

elif len(history) < 16:
    output = random.choice(["R","P","S"])
    prediction = output
    history.append(char2num[last])
    history.append(char2num[input])
    correctly_predictions.append(1 if last_prediction == input else 0)
    counter += 1
    nonrandom_output = 'Q'

elif counter < 100:
    counter += 1
    ai.train([history], [input])
    
    history.append(char2num[last])
    history.append(char2num[input])
    history.pop(0)
    history.pop(0)

    output = random.choice(["R","P","S"])
    prediction = output
    nonrandom_output = 'Q'

else:
    correctly_predictions.append(1 if last_prediction == input else 0)
    if len(correctly_predictions) > 50: correctly_predictions.pop(0)

    ai.train([history], [input])

    history.append(char2num[last])
    history.append(char2num[input])
    history.pop(0)
    history.pop(0)

    prediction = ai.classify([history])[0][1]

    if sum(correctly_predictions) > 35: blefe_rate = 0
    elif sum(correctly_predictions) > 25: blefe_rate = 0.3
    else: blefe_rate = 0.45

    if random.random() < blefe_rate: output = prediction
    else: output = defeat[prediction]
    
    nonrandom_output = defeat[prediction]

last = output
last_prediction = prediction
""", '<string>', 'exec')

code_mutation = compile(
    """
import random
numPre = 30
numMeta = 6
if not input:
    limit = 8
    beat={'R':'P','P':'S','S':'R'}
    moves=['','','','']
    pScore=[[5]*numPre,[5]*numPre,[5]*numPre,[5]*numPre,[5]*numPre,[5]*numPre]
    centrifuge={'RP':0,'PS':1,'SR':2,'PR':3,'SP':4,'RS':5,'RR':6,'PP':7,'SS':8}
    centripete={'R':0,'P':1,'S':2}
    soma = [0,0,0,0,0,0,0,0,0];
    rps = [1,1,1];
    a="RPS"
    best = [0,0,0];
    length=0
    p=[random.choice("RPS")]*numPre
    m=[random.choice("RPS")]*numMeta
    mScore=[5,2,5,2,4,2]
else:
    for i in range(numPre):
        pp = p[i]
        bpp = beat[pp]
        bbpp = beat[bpp]
        pScore[0][i]=0.9*pScore[0][i]+((input==pp)-(input==bbpp))*3
        pScore[1][i]=0.87*pScore[1][i]+((output==pp)-(output==bbpp))*3
        pScore[2][i]=0.84*pScore[2][i]+(input==pp)*3.3-(input==bpp)*1.2-(input==bbpp)*2.3
        pScore[3][i]=0.81*pScore[3][i]+(output==pp)*3.3-(output==bpp)*1.2-(output==bbpp)*2.3
        pScore[4][i]=(pScore[4][i]+(input==pp)*3)*(1-(input==bbpp))
        pScore[5][i]=(pScore[5][i]+(output==pp)*3)*(1-(output==bbpp))
    for i in range(numMeta):
        mScore[i]=0.95*(mScore[i]+(input==m[i])-(input==beat[beat[m[i]]]))
    soma[centrifuge[input+output]] +=1;
    rps[centripete[input]] +=1;
    moves[0]+=str(centrifuge[input+output])
    moves[1]+=input
    moves[2]+=output
    length+=1
    for y in range(3):
        j=min([length,limit])
        while j>=1 and not moves[y][length-j:length] in moves[y][0:length-1]:
            j-=1
        i = moves[y].rfind(moves[y][length-j:length],0,length-1)
        p[0+2*y] = moves[1][j+i] 
        p[1+2*y] = beat[moves[2][j+i]]
    j=min([length,limit])
    while j>=2 and not moves[0][length-j:length-1] in moves[0][0:length-2]:
        j-=1
    i = moves[0].rfind(moves[0][length-j:length-1],0,length-2)
    if j+i>=length:
        p[6] = p[7] = random.choice("RPS")
    else:
        p[6] = moves[1][j+i] 
        p[7] = beat[moves[2][j+i]]
        
    best[0] = soma[centrifuge[output+'R']]*rps[0]/rps[centripete[output]]
    best[1] = soma[centrifuge[output+'P']]*rps[1]/rps[centripete[output]]
    best[2] = soma[centrifuge[output+'S']]*rps[2]/rps[centripete[output]]
    p[8] = p[9] = a[best.index(max(best))]
    
    for i in range(10,numPre):
        p[i]=beat[beat[p[i-10]]]
        
    for i in range(0,numMeta,2):
        m[i]=       p[pScore[i  ].index(max(pScore[i  ]))]
        m[i+1]=beat[p[pScore[i+1].index(max(pScore[i+1]))]]
output = beat[m[mScore.index(max(mScore))]]
COREYSPROBS = {}
for coreysidx in range(len(m)):
    coreyskey = beat[m[coreysidx]]
    if coreyskey not in COREYSPROBS.keys():
        COREYSPROBS[coreyskey] = mScore[coreysidx]
    elif mScore[coreysidx] > COREYSPROBS[coreyskey]:
        COREYSPROBS[coreyskey] = mScore[coreysidx]
nonrandom_output = output
if max(mScore)<0.07 or random.randint(5,37)>length:
    output=beat[random.choice("RPS")]

""", '<string>', 'exec')

code_diludefensive = compile(
    """
# see also www.dllu.net/rps
# This is another mostly rfind-based bot.
# Compared with dllu1, dllu1_defensive implements some defensive measures. The defense mechanism here isn't quite as agressive as, say, the derp_v6 program by evolvingstuff (which gets more and more random the more it loses, until it finally switches to nihilism whereupon it becomes totally random) that scores 48% against zai_all_switch_meta_corrector. But too much defending also reduces overall effectiveness against weaker bots.
# Due to the increased amount of randomness, dllu1_defensive has a slightly lower overall win rate (by about 0.2%) against an arbitrary pool of 39 bots. But it is slightly less bad against zai_all_switch_meta_corrector and fares well against the original dllu1.

# The key differences are:
# 1) dithering. Some random noise is added to the metapredictor scores.
# 2) more randomness when maximum metapredictor score is below a threshold (which indicates that none of the metapredictors are very effective).

'''
    dllu1, dllu1_defensive versus bayes14:
    
    bots/dllu1.py: won 79.2% of matches (396 of 500)
    won 35.3% of rounds (176595 of 500000)
    avg score 23.2, net score 11624.0, time 116037.42636
    
    bots/dllu1_defensive.py: won 79.0% of matches (395 of 500)
    won 34.9% of rounds (174737 of 500000)
    avg score 19.6, net score 9797.0, time 117635.02678
    
    bots/bayes14.py: won 20.1% of matches (201 of 1000)
    won 33.0% of rounds (329911 of 1000000)
    avg score -21.4, net score -21421.0, time 266629.78977
    
    --------------------------------------------------
    dllu1, dllu1_defensive versus zai_all_switch_meta_corrector:
    
    bots/zai_all_meta_switch_corrector.py: won 80.3% of matches (803 of 1000)
    won 34.9% of rounds (348731 of 1000000)
    avg score 26.8, net score 26836.0, time 95782.26727
    
    bots/dllu1_defensive.py: won 33.2% of matches (166 of 500)
    won 32.9% of rounds (164574 of 500000)
    avg score -9.9, net score -4943.0, time 116385.98595
    
    bots/dllu1.py: won 4.6% of matches (23 of 500)
    won 31.5% of rounds (157321 of 500000)
    avg score -43.8, net score -21893.0, time 114404.34772
    
    --------------------------------------------------
    dllu1 versus dllu1_defensive:
    
    bots/dllu1_defensive.py: won 76.0% of matches (380 of 500)
    won 34.6% of rounds (172993 of 500000)
    avg score 17.9, net score 8960.0, time 115415.65399
    
    bots/dllu1.py: won 23.2% of matches (116 of 500)
    won 32.8% of rounds (164033 of 500000)
    avg score -17.9, net score -8960.0, time 113650.73649
'''

# for more results for dllu1 and dllu1_defensive against an arbitrary pool of strong bots, see: http://pastebin.com/59kCHq3r

import random
numPre = 30
numMeta = 6
if not input:
    limit = 8
    beat={'R':'P','P':'S','S':'R'}
    moves=['','','','']
    pScore=[[5]*numPre,[5]*numPre,[5]*numPre,[5]*numPre,[5]*numPre,[5]*numPre]
    centrifuge={'RP':0,'PS':1,'SR':2,'PR':3,'SP':4,'RS':5,'RR':6,'PP':7,'SS':8}
    centripete={'R':0,'P':1,'S':2}
    soma = [0,0,0,0,0,0,0,0,0];
    rps = [1,1,1];
    a="RPS"
    best = [0,0,0];
    length=0
    p=[random.choice("RPS")]*numPre
    m=[random.choice("RPS")]*numMeta
    mScore=[5,2,5,2,4,2]
    dithering = 0.7
else:
    for i in range(numPre):
        pp = p[i]
        bpp = beat[pp]
        bbpp = beat[bpp]
        pScore[0][i]=0.9*pScore[0][i]+((input==pp)-(input==bbpp))*3
        pScore[1][i]=0.9*pScore[1][i]+((output==pp)-(output==bbpp))*3
        pScore[2][i]=0.87*pScore[2][i]+(input==pp)*3.3-(input==bpp)*1.2-(input==bbpp)*2.3
        pScore[3][i]=0.87*pScore[3][i]+(output==pp)*3.3-(output==bpp)*1.2-(output==bbpp)*2.3
        pScore[4][i]=(pScore[4][i]+(input==pp)*3)*(1-(input==bbpp))
        pScore[5][i]=(pScore[5][i]+(output==pp)*3)*(1-(output==bbpp))
    for i in range(numMeta):
        mScore[i]=0.96*(mScore[i]+(input==m[i])-(input==beat[beat[m[i]]])) + (random.random()-0.5)*dithering
    soma[centrifuge[input+output]] +=1;
    rps[centripete[input]] +=1;
    moves[0]+=str(centrifuge[input+output])
    moves[1]+=input
    moves[2]+=output
    length+=1
    for y in range(3):
        j=min([length,limit])
        while j>=1 and not moves[y][length-j:length] in moves[y][0:length-1]:
            j-=1
        i = moves[y].rfind(moves[y][length-j:length],0,length-1)
        p[0+2*y] = moves[1][j+i] 
        p[1+2*y] = beat[moves[2][j+i]]
    j=min([length,limit])
    while j>=2 and not moves[0][length-j:length-1] in moves[0][0:length-2]:
        j-=1
    i = moves[0].rfind(moves[0][length-j:length-1],0,length-2)
    if j+i>=length:
        p[6] = p[7] = random.choice("RPS")
    else:
        p[6] = moves[1][j+i] 
        p[7] = beat[moves[2][j+i]]
        
    best[0] = soma[centrifuge[output+'R']]*rps[0]/rps[centripete[output]]
    best[1] = soma[centrifuge[output+'P']]*rps[1]/rps[centripete[output]]
    best[2] = soma[centrifuge[output+'S']]*rps[2]/rps[centripete[output]]
    p[8] = p[9] = a[best.index(max(best))]
    
    for i in range(10,numPre):
        p[i]=beat[beat[p[i-10]]]
        
    for i in range(0,numMeta,2):
        m[i]=       p[pScore[i  ].index(max(pScore[i  ]))]
        m[i+1]=beat[p[pScore[i+1].index(max(pScore[i+1]))]]
output = beat[m[mScore.index(max(mScore))]]
COREYSPROBS = {}
for coreysidx in range(len(m)):
    coreyskey = beat[m[coreysidx]]
    if coreyskey not in COREYSPROBS.keys():
        COREYSPROBS[coreyskey] = mScore[coreysidx]
    elif mScore[coreysidx] > COREYSPROBS[coreyskey]:
        COREYSPROBS[coreyskey] = mScore[coreysidx]
nonrandom_output = output
if max(mScore)<3+random.random() or random.randint(3,40)>length:
    output=beat[random.choice("RPS")]

""", '<string>', 'exec')

code_dumbneural4 = compile(
    """
#added random fallback to dumbneurals-v0.1

if input == "":
    import random
    import math

    class NN(object):
        learnrate = {"win": 0.01, "tie": 0.2, "loss": 0.5}

        def __init__(self, ins):
            #number of nodes
            self.ni = ins
            self.no = 3
            #init activations
            self.ia = [0] * self.ni
            self.oa = [0] * self.no
            #init weights
            #initial weights need(?) to be below 3/self.ni
            self.weights = [[random.uniform(0, 3./self.ni) for i in range(self.ni)]
                            for j in range(self.no)]
         
        def feed(self, inps):        
            self.ia = inps
            self.oa = [sum(self.weights[j][i] * self.ia[i]
                           for i in range(self.ni))
                       for j in range(self.no)]
            return self.oa

        def train(self, target, last_round):
            #hebb learning rule
            #factor of 1/self.ni*3 is there to ensure output
            #in range 0,1 if lernrates are < 1
            #and inputs are 1 for every third inputnode
            #maybe important for convergence?
            result_last_round = result[last_round]
            learnrate = self.learnrate[result_last_round]
            error = list(map(lambda x, y: x - y, target, self.oa))
            for i in range(self.ni):
                for j in range(self.no):
                    self.weights[j][i] += (learnrate * error[j] * self.ia[i] /
                                             self.ni * 3)

    result = {"RS": "win", "SP": "win", "PR": "win",
              "RR": "tie", "SS": "tie", "PP": "tie",
              "SR": "loss", "PS": "loss", "RP": "loss"}
    num_moves = 5 #number of recent rounds the neurons get to see
    nn = NN(num_moves * 6)
    ourmoves = ""
    oppmoves = ""
    nnpred = "S"
    round_cnt = 0
    score = 0
    scores = {"win": 1, "tie": 0, "loss": -1}

    def moves_to_repr(moves):
        repr_lookup = {"R": [1, 0, 0], "S": [0, 1, 0], "P" : [0, 0, 1]}
        nninput = []
        for c in moves:
            nninput.extend(repr_lookup[c])
        return nninput

    def repr_to_move(nnoutput):
        ciphers = "RSP"
        index = max(enumerate(nnoutput), key=lambda x:x[1])[0]
        COREYSPROBS = {}
        for a,b in enumerate(nnoutput):
            COREYSPROBS[ciphers[a]] = b
        return ciphers[index], COREYSPROBS
    
    def feedneurons():
        inps = moves_to_repr(ourmoves[-num_moves:] + oppmoves[-num_moves:])
        ourmove = nn.feed(inps)
        return repr_to_move(ourmove)

    def trainneurons(nn_last_round):
        win = dict(zip("RSP", "PRS"))
        target = moves_to_repr(win[oppmoves[-1]])
        nn.train(target, nn_last_round)
        
    
    output = random.choice("RSP")
    ourmoves += output
    nonrandom_output = 'Q'
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
else:
    oppmoves += input
    round_cnt += 1
    if round_cnt < 5:
        output = random.choice("RSP")
        COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
    else:
        nn_last_round = nnpred + oppmoves[-1]
        trainneurons(nn_last_round)
        score = 0.9 * score + scores[result[nn_last_round]]
        nnpred, COREYSPROBS = feedneurons()
        nonrandom_output = nnpred
        if score > 0.5: #do we have confidence?
            output = nnpred
        else: #fall back if we cannot beat you
            output = random.choice("RSP")
    ourmoves += output

""", '<string>', 'exec')

code_puppy1003 = compile(
    """
#see also www.dllu.net/rps.html
import random
numPre = 18
numMeta = 6
if not input:
    limit = 8
    beat={'R':'P','P':'S','S':'R'}
    moves=['','','']
    pScore=[[3]*numPre,[3]*numPre,[3]*numPre,[3]*numPre,[3]*numPre,[3]*numPre]
    centrifuge={'RP':'a','PS':'b','SR':'c','PR':'d','SP':'e','RS':'f','RR':'g','PP':'h','SS':'i'}
    length=0
    p=[random.choice("RPS")]*numPre
    m=[random.choice("RPS")]*numMeta
    mScore=[3]*numMeta
else:
    for i in range(numPre):
        pScore[0][i]=0.8*pScore[0][i]+((input==p[i])-(input==beat[beat[p[i]]]))*3
        pScore[1][i]=0.8*pScore[1][i]+((output==p[i])-(output==beat[beat[p[i]]]))*3
        pScore[2][i]=0.87*pScore[2][i]+(input==p[i])*3.3-(input==beat[p[i]])*0.9-(input==beat[beat[p[i]]])*3
        pScore[3][i]=0.87*pScore[3][i]+(output==p[i])*3.3-(output==beat[p[i]])*0.9-(output==beat[beat[p[i]]])*3
        pScore[4][i]=(pScore[4][i]+(input==p[i])*3)*(1-(input==beat[beat[p[i]]]))
        pScore[5][i]=(pScore[5][i]+(output==p[i])*3)*(1-(output==beat[beat[p[i]]]))
    for i in range(numMeta):
        mScore[i]=(mScore[i]+(input==m[i]))*(1-(input==beat[beat[m[i]]]))
    moves[0]+=centrifuge[input+output]
    moves[1]+=input        
    moves[2]+=output
    length+=1
    for y in range(3):
        j=min([length,limit])
        while j>=1 and not moves[y][length-j:length] in moves[y][0:length-1]:
            j-=1
        i = moves[y].rfind(moves[y][length-j:length],0,length-1)
        p[0+2*y] = moves[1][j+i] 
        p[1+2*y] = beat[moves[2][j+i]] 
    
    for i in range(6,6*3):
        p[i]=beat[beat[p[i-6]]]
        
    for i in range(0,6,2):
        m[i]=       p[pScore[i  ].index(max(pScore[i  ]))]
        m[i+1]=beat[p[pScore[i+1].index(max(pScore[i+1]))]]
output = beat[m[mScore.index(max(mScore))]]
COREYSPROBS = {}
for coreysidx in range(len(m)):
    coreyskey = beat[m[coreysidx]]
    if coreyskey not in COREYSPROBS.keys():
        COREYSPROBS[coreyskey] = mScore[coreysidx]
    elif mScore[coreysidx] > COREYSPROBS[coreyskey]:
        COREYSPROBS[coreyskey] = mScore[coreysidx]
nonrandom_output = output
if max(mScore)<0.07 or random.randint(3,40)>length:
    output=beat[random.choice("RPS")]

""", '<string>', 'exec')

code_neural12 = compile(
    """
# Neural net without numpy.  A wee bit on the slow side.  Learning rate = 1.0.

if input:
    output, COREYSPROBS = rps.play(input)
    nonrandom_output = output
else:

    import random, math
    #from itertools import imap
    from operator import mul
    
    imap = lambda *args, **kwargs: list(map(*args, **kwargs))

    class RPS:

        letter2move = {
            'R': [1,0,0],
            'P': [0,1,0],
            'S': [0,0,1],
        }

        training = {
            'R': [0.02, 0.97, 0.01],
            'P': [0.01, 0.02, 0.97],
            'S': [0.97, 0.01, 0.02],
        }

        def __init__(self, memory_length=12):

            nn_input_size = memory_length*2*3
            self.moves = [0] * nn_input_size
            layer_sizes = [nn_input_size, int(nn_input_size / 2), 3]
            self.nn = NN(layer_sizes)
            self.turn = 0


        def play(self, their_last_letter=None):
            if their_last_letter:
                their_last_move = self.letter2move[their_last_letter]
                self.nn.train(self.training[their_last_letter])
                self.moves = self.my_last_move + their_last_move + self.moves[:-6]

            my_inclinations = self.nn.query(self.moves)
            my_letter = self.choose(my_inclinations)
            self.my_last_move = self.letter2move[my_letter]

            self.turn += 1

            COREYSPROBS = {'R':my_inclinations[0],'P':my_inclinations[1],'S':my_inclinations[2]}
            return my_letter, COREYSPROBS


        def choose(self, inclinations):
            total = sum(inclinations)
            cut = random.uniform(0, total)
            if cut < inclinations[0]:
                return 'R'
            elif cut < (inclinations[0] + inclinations[1]):
                return 'P'
            else:
                return 'S'


    class NN:
        def __init__(self, layer_sizes):
            self.weights = []
            self.weight_dimensions = []
            for n_before, n_after in zip(layer_sizes[:-1], layer_sizes[1:]):
                std_dev = n_after**-.5
                weights = [[random.gauss(0.0, std_dev) for x in range(n_before)] for y in range(n_after)]
                self.weights.append(weights)
                self.weight_dimensions.append((n_after, n_before))

        def train(self, targets):
            errors = [t-l for t,l in zip(targets, self.layers[-1])]
            #errors = list(imap(sub, targets, self.layers[-1]))

            for i in range(len(self.weights)-1, -1, -1):
                weights = self.weights[i]
                dimensions = self.weight_dimensions[i]
                after = self.layers[i+1]
                before = self.layers[i]

                new_errors = []
                for j in range(dimensions[1]):
                    column = [row[j] for row in weights]
                    new_errors.append(sum(imap(mul, errors, column)))

                for n_after in range(dimensions[0]):
                    after_total = after[n_after]
                    after_slope = after_total * (1.0 - after_total)
                    after_contrib = after_slope * errors[n_after]
                    weights[n_after] = [weights[n_after][n_before] + before[n_before] * after_contrib for n_before in range(dimensions[1])]

                errors = new_errors


        def query(self, inputs):
            current = inputs
            self.layers = [current]
            for weights in self.weights:
                current = [1.0 / (1.0 + math.exp(-sum(imap(mul, current, row)))) for row in weights]
                self.layers.append(current)
            return current


    rps = RPS(6)
    output, COREYSPROBS = rps.play()
    nonrandom_output = 'Q'

""", '<string>', 'exec')

code_puppy16H = compile(
    """
#see also www.dllu.net/rps.html
#Centrifugal Bumblepuppy + Helicase
import random
numPre = 60
numMeta = 18
if not input:
    limits = [50,20,10]
    beat={'R':'P','P':'S','S':'R'}
    moves=['','','']
    pScore=[[3]*numPre,[3]*numPre,[3]*numPre,[3]*numPre,[3]*numPre,[3]*numPre]
    centrifuge={'RP':'a','PS':'b','SR':'c','PR':'d','SP':'e','RS':'f','RR':'g','PP':'h','SS':'i'}
    length=0
    p=[random.choice("RPS")]*numPre
    m=[random.choice("RPS")]*numMeta
    mScore=[3]*numMeta
    helicase=[0,0,0,0,0,0]
else:
    for i in range(numPre):
        pScore[0][i]=0.8*pScore[0][i]+((input==p[i])-(input==beat[beat[p[i]]]))*3
        pScore[1][i]=0.8*pScore[1][i]+((output==p[i])-(output==beat[beat[p[i]]]))*3
        pScore[2][i]=0.87*pScore[2][i]+(input==p[i])*3.3-(input==beat[p[i]])*0.9-(input==beat[beat[p[i]]])*3
        pScore[3][i]=0.87*pScore[3][i]+(output==p[i])*3.3-(output==beat[p[i]])*0.9-(output==beat[beat[p[i]]])*3
        pScore[4][i]=(pScore[4][i]+(input==p[i])*3)*(1-(input==beat[beat[p[i]]]))
        pScore[5][i]=(pScore[5][i]+(output==p[i])*3)*(1-(output==beat[beat[p[i]]]))
    for i in range(numMeta):
        mScore[i]=(mScore[i]+(input==m[i]))*(1-(input==beat[beat[m[i]]]))
    moves[0]+=centrifuge[input+output]
    moves[1]+=input
    moves[2]+=output
    length+=1
    for z in range(3):
        limit = min([length,limits[z]])
        for y in range(3):
            j=limit
            while j>=1 and not moves[y][length-j:length] in moves[y][0:length-1]:
                j-=1
            i = moves[y].rfind(moves[y][length-j:length],0,length-1)
            p[0+2*y+6*z] = moves[1][j+i] 
            p[1+2*y+6*z] = beat[moves[2][j+i]] 
            
    helicase[0] = helicase[0]*0.95+{'R':0,'P':-0.1,'S':0.1}[output]
    helicase[1] = helicase[1]*0.95+{'R':0.1,'P':0,'S':-0.1}[output]
    helicase[2] = helicase[2]*0.95+{'R':-0.1,'P':0.1,'S':0}[output]
    p[18] = {0:'R',1:'P',2:'S',3:'R',4:'P',5:'S'}[helicase.index(max(helicase[0:3]))]
    
    helicase[3] = helicase[3]*0.95+{'R':0.1,'P':0,'S':-0.1}[input]
    helicase[4] = helicase[4]*0.95+{'R':-0.1,'P':0.1,'S':0}[input]
    helicase[5] = helicase[5]*0.95+{'R':0,'P':-0.1,'S':0.1}[input]
    p[19] = {0:'R',1:'P',2:'S',3:'R',4:'P',5:'S'}[helicase.index(max(helicase[3:6]))]
    
    for i in range(20,20*3):
        p[i]=beat[beat[p[i-20]]]
        
    for i in range(0,6,2):
        m[i]=       p[pScore[i  ].index(max(pScore[i  ]))]
        m[i+1]=beat[p[pScore[i+1].index(max(pScore[i+1]))]]
    for i in range(6,18):
        m[i]=beat[beat[m[i-6]]]
output = beat[m[mScore.index(max(mScore))]]
COREYSPROBS = {}
for coreysidx in range(len(m)):
    coreyskey = beat[m[coreysidx]]
    if coreyskey not in COREYSPROBS.keys():
        COREYSPROBS[coreyskey] = mScore[coreysidx]
    elif mScore[coreysidx] > COREYSPROBS[coreyskey]:
        COREYSPROBS[coreyskey] = mScore[coreysidx]
nonrandom_output = output
if random.random()<0.13 or random.randint(3,40)>length:
    output=beat[random.choice("RPS")]

""", '<string>', 'exec')

code_puppy17 = compile(
    """
#see also www.dllu.net/rps.html
import random
numPre = 60
numMeta = 18
if not input:
    limits = [42,19,10]
    beat={'R':'P','P':'S','S':'R'}
    moves=['','','']
    pScore=[[3]*numPre,[3]*numPre,[3]*numPre,[3]*numPre,[3]*numPre,[3]*numPre]
    centrifuge={'RP':'a','PS':'b','SR':'c','PR':'d','SP':'e','RS':'f','RR':'g','PP':'h','SS':'i'}
    length=0
    p=[random.choice("RPS")]*numPre
    m=[random.choice("RPS")]*numMeta
    mScore=[3]*numMeta
    helicase=[0,0,0,0,0,0]
else:
    for i in range(numPre):
        pScore[0][i]=0.8*pScore[0][i]+((input==p[i])-(input==beat[beat[p[i]]]))*3
        pScore[1][i]=0.8*pScore[1][i]+((output==p[i])-(output==beat[beat[p[i]]]))*3
        pScore[2][i]=0.87*pScore[2][i]+(input==p[i])*3.3-(input==beat[p[i]])*0.9-(input==beat[beat[p[i]]])*3
        pScore[3][i]=0.87*pScore[3][i]+(output==p[i])*3.3-(output==beat[p[i]])*0.9-(output==beat[beat[p[i]]])*3
        pScore[4][i]=(pScore[4][i]+(input==p[i])*3)*(1-(input==beat[beat[p[i]]]))
        pScore[5][i]=(pScore[5][i]+(output==p[i])*3)*(1-(output==beat[beat[p[i]]]))
    for i in range(numMeta):
        mScore[i]=(mScore[i]+(input==m[i]))*(1-(input==beat[beat[m[i]]]))
    moves[0]+=centrifuge[input+output]
    moves[1]+=input        
    moves[2]+=output
    length+=1
    for z in range(3):
        limit = min([length,limits[z]])
        j=length+1
        for y in range(3):
            if j>limit:
                j=limit
                while j>=1 and not moves[y][length-j:length] in moves[y][0:length-1]:
                    j-=1
                i = moves[y].rfind(moves[y][length-j:length],0,length-1)
            p[0+2*y+6*z] = moves[1][j+i]
            p[1+2*y+6*z] = beat[moves[2][j+i]]
    helicase[0] = helicase[0]*0.95+{'R':0,'P':-0.1,'S':0.1}[output]
    helicase[1] = helicase[1]*0.95+{'R':0.1,'P':0,'S':-0.1}[output]
    helicase[2] = helicase[2]*0.95+{'R':-0.1,'P':0.1,'S':0}[output]
    p[18] = {0:'R',1:'P',2:'S',3:'R',4:'P',5:'S'}[helicase.index(max(helicase[0:3]))]
    
    helicase[3] = helicase[3]*0.95+{'R':0.1,'P':0,'S':-0.1}[input]
    helicase[4] = helicase[4]*0.95+{'R':-0.1,'P':0.1,'S':0}[input]
    helicase[5] = helicase[5]*0.95+{'R':0,'P':-0.1,'S':0.1}[input]
    p[19] = {0:'R',1:'P',2:'S',3:'R',4:'P',5:'S'}[helicase.index(max(helicase[3:6]))]
    
    for i in range(20,20*3):
        p[i]=beat[beat[p[i-20]]]
        
    for i in range(0,6,2):
        m[i]=       p[pScore[i  ].index(max(pScore[i  ]))]
        m[i+1]=beat[p[pScore[i+1].index(max(pScore[i+1]))]]
    for i in range(6,18):
        m[i]=beat[beat[m[i-6]]]
output = beat[m[mScore.index(max(mScore))]]
COREYSPROBS = {}
for coreysidx in range(len(m)):
    coreyskey = beat[m[coreysidx]]
    if coreyskey not in COREYSPROBS.keys():
        COREYSPROBS[coreyskey] = mScore[coreysidx]
    elif mScore[coreysidx] > COREYSPROBS[coreyskey]:
        COREYSPROBS[coreyskey] = mScore[coreysidx]
nonrandom_output = output
if random.random()<0.13 or random.randint(3,40)>length:
    output=beat[random.choice("RPS")]

""", '<string>', 'exec')

code_roseguadeloupe = compile(
    """
# learn about Martian cuisine at http://www.codeforces.com/problemset/problem/77/E

import random

if not input:
    beat = {'R':'P','P':'S','S':'R'}
    fusion = {'RP':'a','PS':'b','SR':'c','PR':'d','SP':'e','RS':'f','RR':'g','PP':'h','SS':'i'}
    limits = [5,11,24,52,6]
    warps = [0,0,0,0,5]
    moves = ["","",""]
    numPredictors = 6*len(limits)*len(moves)
    numMetapredictors = 6
    predictors = [random.choice("RPS") for i in range(numPredictors)]
    predictorscore = [2*random.random() for i in range(numPredictors)]
    opponentscore = [2*random.random() for i in range(numPredictors)]
    metapredictors = [random.choice("RPS") for i in range(numMetapredictors)]
    metascore = [12,12,0,0,0,0]
    threat = [0,0,0]
    outcome = random.choice([-1, 0, 1])
    length = 0
    output = random.choice("RPS")
    nonrandom_output = 'Q'
    COREYSPROBS = {'R':.33,'P':.33,'S':.33}
else:
    oldoutcome = outcome
    if (beat[input] == output):
        outcome = 1
    elif (input == beat[output]):
        outcome = -1
    else:
        outcome = 0
    threat[oldoutcome + 1] *= 0.96
    threat[oldoutcome + 1] -= 0.04*outcome
    for i in range(numPredictors):
        predictorscore[i] *= 0.8
        predictorscore[i] += (beat[input] == predictors[i])
        predictorscore[i] -= (input == beat[predictors[i]])
        opponentscore[i] *= 0.8
        opponentscore[i] += (beat[output] == predictors[i])
        opponentscore[i] -= (output == beat[predictors[i]])
        if input == beat[predictors[i]] and random.random() < 0.3:
            predictorscore[i] = 0
        if output == beat[predictors[i]] and random.random() < 0.3:
            opponentscore[i] = 0
    for i in range(numMetapredictors):
        metascore[i] *= 0.951
        metascore[i] += (beat[input] == metapredictors[i])
        metascore[i] -= (input == beat[metapredictors[i]])
        if input == beat[metapredictors[i]] and random.random() < 0.2:
            metascore[i] = 0
    moves[0] += input
    moves[1] += output
    moves[2] += fusion[input+output]
    length += 1
    
    for z in range(3*len(limits)):
        wormhole = length - warps[z//3]
        if wormhole <= 0:
            predictors[2 * z] = random.choice("RPS")
            predictors[2*z+1] = random.choice("RPS")
        else:
            j = min(wormhole-1, limits[z//3])
            while not moves[z%3][wormhole-j:wormhole] in moves[z%3][0:wormhole-1]:
                j-=1
            i = moves[z%3].rfind(moves[z%3][wormhole-j:wormhole], 0, wormhole-1)
            predictors[2 * z] = moves[0][i+j+warps[z//3]]
            predictors[2*z+1] = moves[1][i+j+warps[z//3]]
    for i in range(int(numPredictors/3), numPredictors):
        predictors[i] = beat[predictors[i - int(numPredictors/3)]]
    metapredictors[0] = predictors[predictorscore.index(max(predictorscore))]
    metapredictors[1] = beat[predictors[opponentscore.index(max(opponentscore))]]
    for i in range(int(numMetapredictors/3), numMetapredictors):
        metapredictors[i] = beat[metapredictors[i - int(numMetapredictors/3)]]
    
    if length < 6 or random.random() < 2*threat[outcome+1]+0.1:
        output = random.choice("RPS")
    else:
        output = metapredictors[metascore.index(max(metascore))]
    nonrandom_output = metapredictors[metascore.index(max(metascore))]

    COREYSPROBS = {}
    for coreysidx in range(len(metapredictors)):
        coreyskey = metapredictors[coreysidx]
        if coreyskey not in COREYSPROBS.keys():
            COREYSPROBS[coreyskey] = metascore[coreysidx]
        elif metascore[coreysidx] > COREYSPROBS[coreyskey]:
            COREYSPROBS[coreyskey] = metascore[coreysidx]

""", '<string>', 'exec')

code_randomdropswitch = compile(
    """
# after a wrong prediction, drops the strategy with 50% probability

import random

if not input:
    beat = {'R':'P','P':'S','S':'R'}
    fusion = {'RP':'a','PS':'b','SR':'c','PR':'d','SP':'e','RS':'f','RR':'g','PP':'h','SS':'i'}
    limits = [5,12,28,64]
    moves = ["","",""]
    numPredictors = 6*len(limits)*len(moves)
    numMetapredictors = 6
    predictors = [random.choice("RPS") for i in range(numPredictors)]
    predictorscore = [2*random.random() for i in range(numPredictors)]
    opponentscore = [2*random.random() for i in range(numPredictors)]
    metapredictors = [random.choice("RPS") for i in range(numMetapredictors)]
    metascore = [20,10,0,0,0,0]
    threat = [0,0,0]
    outcome = 0
    length = 0
    output = random.choice(['R','P','S'])
    wait = random.randint(18, 30)
    nonrandom_output = 'Q'
    COREYSPROBS = {'R':.33,'P':.33,'S':.33}
else:
    oldoutcome = outcome
    if (beat[input] == output):
        outcome = 1
    elif (input == beat[output]):
        outcome = -1
    else:
        outcome = 0
    threat[oldoutcome + 1] *= 0.958
    threat[oldoutcome + 1] -= 0.042*outcome
    for i in range(numPredictors):
        predictorscore[i] *= 0.8
        predictorscore[i] += (beat[input] == predictors[i])
        predictorscore[i] -= (input == beat[predictors[i]])
        opponentscore[i] *= 0.8
        opponentscore[i] += (beat[output] == predictors[i])
        opponentscore[i] -= (output == beat[predictors[i]])
        if input == beat[predictors[i]] and random.random() < 0.5:
            predictorscore[i] = 0
        if output == beat[predictors[i]] and random.random() < 0.5:
            opponentscore[i] = 0
    for i in range(numMetapredictors):
        metascore[i] *= 0.958
        metascore[i] += (beat[input] == metapredictors[i])
        metascore[i] -= (input == beat[metapredictors[i]])
        if input == beat[metapredictors[i]] and random.random() < 0.5:
            metascore[i] = 0
    moves[0] += input
    moves[1] += output
    moves[2] += fusion[input+output]
    length += 1
    
    for z in range(3*len(limits)):
        j = min([length-1, limits[z//3]])
        while not moves[z%3][length-j:length] in moves[z%3][0:length-1]:
            j-=1
        i = moves[z%3].rfind(moves[z%3][length-j:length], 0, length-1)
        predictors[2 * z] = moves[0][i+j]
        predictors[2*z+1] = moves[1][i+j]
    for i in range(int(numPredictors/3), numPredictors):
        predictors[i] = beat[predictors[i - int(numPredictors/3)]]
    metapredictors[0] = predictors[predictorscore.index(max(predictorscore))]
    metapredictors[1] = beat[predictors[opponentscore.index(max(opponentscore))]]
    for i in range(int(numMetapredictors/3), numMetapredictors):
        metapredictors[i] = beat[metapredictors[i - int(numMetapredictors/3)]]
    
    if length < wait or random.random() < 0.4*threat[outcome+1]+0.1:
        output = random.choice("RPS")
    else:
        output = metapredictors[metascore.index(max(metascore))]
        
    nonrandom_output = metapredictors[metascore.index(max(metascore))]
    COREYSPROBS = {}
    for coreysidx in range(len(metapredictors)):
        coreyskey = metapredictors[coreysidx]
        if coreyskey not in COREYSPROBS.keys():
            COREYSPROBS[coreyskey] = metascore[coreysidx]
        elif metascore[coreysidx] > COREYSPROBS[coreyskey]:
            COREYSPROBS[coreyskey] = metascore[coreysidx]

""", '<string>', 'exec')

code_omelettedeimos = compile(
    """
# throws rocks!

import random

if not input:
    beat = {'R':'P','P':'S','S':'R'}
    fusion = {'RP':'a','PS':'b','SR':'c','PR':'d','SP':'e','RS':'f','RR':'g','PP':'h','SS':'i'}
    limits = [5,12,28,64]
    moves = ["","",""]
    numPredictors = 6*len(limits)*len(moves)
    predictors = list(range(numPredictors))
    predictorscore = list(range(numPredictors))
    opponentscore = list(range(numPredictors))
    for i in range(numPredictors):
        predictors[i] = random.choice(['R','P','S'])
        predictorscore[i] = 2*random.random()
        opponentscore[i] = 2*random.random()
    metapredictors = [random.choice(['R','P','S']),random.choice(['R','P','S'])]
    metascore = 20
    threat = [0,0,0]
    outcome = 0
    length = 0
    output = random.choice(['R','P','S'])
    nonrandom_output = 'Q'
else:
    oldoutcome = outcome
    if (beat[input] == output):
        outcome = 1
    elif (input == beat[output]):
        outcome = -1
    else:
        outcome = 0
    threat[oldoutcome + 1] *= 0.96
    threat[oldoutcome + 1] -= outcome
    for i in range(numPredictors):
        predictorscore[i] *= 0.8
        predictorscore[i] += (beat[input] == predictors[i])
        predictorscore[i] -= (input == beat[predictors[i]])
        opponentscore[i] *= 0.8
        opponentscore[i] += (beat[output] == predictors[i])
        opponentscore[i] -= (output == beat[predictors[i]])
    metascore *= 0.96
    metascore += (beat[input] == metapredictors[0])
    metascore -= (beat[input] == metapredictors[1])
    metascore -= (input == beat[metapredictors[0]])
    metascore += (input == beat[metapredictors[1]])
    moves[0] += input
    moves[1] += output
    moves[2] += fusion[input+output]
    length += 1
    for z in range(3*len(limits)):
        j = min([length-1, limits[z//3]])
        while not moves[z%3][length-j:length] in moves[z%3][0:length-1]:
            j-=1
        i = moves[z%3].rfind(moves[z%3][length-j:length], 0, length-1)
        predictors[2 * z] = moves[0][j+i]
        predictors[2*z+1] = moves[1][j+i]
    for i in range(int(numPredictors/3), numPredictors):
        predictors[i] = beat[predictors[i - int(numPredictors/3)]]
    metapredictors[0] = predictors[predictorscore.index(max(predictorscore))]
    metapredictors[1] = beat[predictors[opponentscore.index(max(opponentscore))]]
    
    if random.random() < 0.45*threat[outcome+1]+0.15:
        output = random.choice(['R','P','S'])
    elif metascore >= 0:
        output = metapredictors[0]
    else:
        output = metapredictors[1]
        
    if metascore >= 0:
        nonrandom_output = metapredictors[0]
    else:
        nonrandom_output = metapredictors[1]

""", '<string>', 'exec')

code_rps_meta_fixed = compile(
    """
import random
import secrets

RNA={'RR':'1','RP':'2','RS':'3','PR':'4','PP':'5','PS':'6','SR':'7','SP':'8','SS':'9'}
mix={'RR':'R','RP':'R','RS':'S','PR':'R','PP':'P','PS':'P','SR':'S','SP':'P','SS':'S'}
rot={'R':'P','P':'S','S':'R'}

if not input:
   DNA=[""]*3
   prin=['R']*36
   meta=['R']*6
   skor1=[[0]*36,[0]*36,[0]*36,[0]*36,[0]*36,[0]*36]
   skor2=[0]*6
else:
   for j in range(36):
       for i in range(4):
           skor1[i][j]*=0.8
       for i in range(4,6):
           skor1[i][j]*=0.5
       for i in range(0,6,2):
           skor1[i][j]-=(input==rot[rot[prin[j]]])
           skor1[i+1][j]-=(output==rot[rot[prin[j]]])
       for i in range(2,6,2):
           skor1[i][j]+=(input==prin[j])
           skor1[i+1][j]+=(output==prin[j])
       skor1[0][j]+=1.3*(input==prin[j])-0.3*(input==rot[prin[j]])
       skor1[1][j]+=1.3*(output==prin[j])-0.3*(output==rot[prin[j]])
   for i in range(6):
       skor2[i]=0.9*skor2[i]+(input==meta[i])-(input==rot[rot[meta[i]]])
   DNA[0]+=input
   DNA[1]+=output
   DNA[2]+=RNA[input+output]
   for i in range(3):
       j=min(11,len(DNA[2]))
       k=-1
       while j>1 and k<0:
             j-=1
             k=DNA[i].rfind(DNA[i][-j:],0,-1)
       prin[4*i]=DNA[0][j+k]
       prin[4*i+1]=rot[DNA[1][j+k]]
       k=DNA[i].rfind(DNA[i][-j:],0,j+k-1)
       k=k*(k>-1)-j*(k<0)
       prin[4*i+2]=mix[prin[4*i]+DNA[0][j+k]]
       prin[4*i+3]=mix[prin[4*i+1]+rot[DNA[1][j+k]]]
   for i in range(12,36):
       prin[i]=rot[prin[i-12]]
   for i in range(0,6,2):
       meta[i]=prin[skor1[i].index(max(skor1[i]))]
       meta[i+1]=rot[prin[skor1[i+1].index(max(skor1[i+1]))]]
if max(skor2)<1:
   output=random.choice("RPS")
else:
   output=rot[meta[skor2.index(max(skor2))]]
   
nonrandom_output = rot[meta[skor2.index(max(skor2))]]

COREYSPROBS = {}
for coreysidx in range(len(meta)):
    coreyskey = rot[meta[coreysidx]]
    if coreyskey not in COREYSPROBS.keys():
        COREYSPROBS[coreyskey] = skor2[coreysidx]
    elif skor2[coreysidx] > COREYSPROBS[coreyskey]:
        COREYSPROBS[coreyskey] = skor2[coreysidx]
    
""", '<string>', 'exec')

code_rps_meta_fixed2 = compile(
    """
import random
import secrets

RNA={'RR':'1','RP':'2','RS':'3','PR':'4','PP':'5','PS':'6','SR':'7','SP':'8','SS':'9'}
mix={'RR':'R','RP':'R','RS':'S','PR':'R','PP':'P','PS':'P','SR':'S','SP':'P','SS':'S'}
rot={'R':'P','P':'S','S':'R'}

if not input:
   DNA=[""]*3
   prin=['P']*36
   meta=['R']*6
   skor1=[[0]*36,[0]*36,[0]*36,[0]*36,[0]*36,[0]*36]
   skor2=[0]*6
else:
   for j in range(36):
       for i in range(4):
           skor1[i][j]*=0.8
       for i in range(4,6):
           skor1[i][j]*=0.5
       for i in range(0,6,2):
           skor1[i][j]-=(input==rot[rot[prin[j]]])
           skor1[i+1][j]-=(output==rot[rot[prin[j]]])
       for i in range(2,6,2):
           skor1[i][j]+=(input==prin[j])
           skor1[i+1][j]+=(output==prin[j])
       skor1[0][j]+=1.3*(input==prin[j])-0.3*(input==rot[prin[j]])
       skor1[1][j]+=1.3*(output==prin[j])-0.3*(output==rot[prin[j]])
   for i in range(6):
       skor2[i]=0.9*skor2[i]+(input==meta[i])-(input==rot[rot[meta[i]]])
   DNA[0]+=input
   DNA[1]+=output
   DNA[2]+=RNA[input+output]
   for i in range(3):
       j=min(11,len(DNA[2]))
       k=-1
       while j>1 and k<0:
             j-=1
             k=DNA[i].rfind(DNA[i][-j:],0,-1)
       prin[4*i]=DNA[0][j+k]
       prin[4*i+1]=rot[DNA[1][j+k]]
       k=DNA[i].rfind(DNA[i][-j:],0,j+k-1)
       k=k*(k>-1)-j*(k<0)
       prin[4*i+2]=mix[prin[4*i]+DNA[0][j+k]]
       prin[4*i+3]=mix[prin[4*i+1]+rot[DNA[1][j+k]]]
   for i in range(12,36):
       prin[i]=rot[prin[i-12]]
   for i in range(0,6,2):
       meta[i]=prin[skor1[i].index(max(skor1[i]))]
       meta[i+1]=rot[prin[skor1[i+1].index(max(skor1[i+1]))]]
if max(skor2)<1:
   output=random.choice("RPS")
else:
   output=rot[meta[skor2.index(max(skor2))]]
   
nonrandom_output = rot[meta[skor2.index(max(skor2))]]
COREYSPROBS = {}
for coreysidx in range(len(meta)):
    coreyskey = rot[meta[coreysidx]]
    if coreyskey not in COREYSPROBS.keys():
        COREYSPROBS[coreyskey] = skor2[coreysidx]
    elif skor2[coreysidx] > COREYSPROBS[coreyskey]:
        COREYSPROBS[coreyskey] = skor2[coreysidx]
    
""", '<string>', 'exec')

code_rps_meta_fixed3 = compile(
    """
import random
import secrets

RNA={'RR':'1','RP':'2','RS':'3','PR':'4','PP':'5','PS':'6','SR':'7','SP':'8','SS':'9'}
mix={'RR':'R','RP':'R','RS':'S','PR':'R','PP':'P','PS':'P','SR':'S','SP':'P','SS':'S'}
rot={'R':'P','P':'S','S':'R'}

if not input:
   DNA=[""]*3
   prin=['S']*36
   meta=['R']*6
   skor1=[[0]*36,[0]*36,[0]*36,[0]*36,[0]*36,[0]*36]
   skor2=[0]*6
else:
   for j in range(36):
       for i in range(4):
           skor1[i][j]*=0.8
       for i in range(4,6):
           skor1[i][j]*=0.5
       for i in range(0,6,2):
           skor1[i][j]-=(input==rot[rot[prin[j]]])
           skor1[i+1][j]-=(output==rot[rot[prin[j]]])
       for i in range(2,6,2):
           skor1[i][j]+=(input==prin[j])
           skor1[i+1][j]+=(output==prin[j])
       skor1[0][j]+=1.3*(input==prin[j])-0.3*(input==rot[prin[j]])
       skor1[1][j]+=1.3*(output==prin[j])-0.3*(output==rot[prin[j]])
   for i in range(6):
       skor2[i]=0.9*skor2[i]+(input==meta[i])-(input==rot[rot[meta[i]]])
   DNA[0]+=input
   DNA[1]+=output
   DNA[2]+=RNA[input+output]
   for i in range(3):
       j=min(11,len(DNA[2]))
       k=-1
       while j>1 and k<0:
             j-=1
             k=DNA[i].rfind(DNA[i][-j:],0,-1)
       prin[4*i]=DNA[0][j+k]
       prin[4*i+1]=rot[DNA[1][j+k]]
       k=DNA[i].rfind(DNA[i][-j:],0,j+k-1)
       k=k*(k>-1)-j*(k<0)
       prin[4*i+2]=mix[prin[4*i]+DNA[0][j+k]]
       prin[4*i+3]=mix[prin[4*i+1]+rot[DNA[1][j+k]]]
   for i in range(12,36):
       prin[i]=rot[prin[i-12]]
   for i in range(0,6,2):
       meta[i]=prin[skor1[i].index(max(skor1[i]))]
       meta[i+1]=rot[prin[skor1[i+1].index(max(skor1[i+1]))]]
if max(skor2)<1:
   output=random.choice("RPS")
else:
   output=rot[meta[skor2.index(max(skor2))]]
   
nonrandom_output = rot[meta[skor2.index(max(skor2))]]
COREYSPROBS = {}
for coreysidx in range(len(meta)):
    coreyskey = rot[meta[coreysidx]]
    if coreyskey not in COREYSPROBS.keys():
        COREYSPROBS[coreyskey] = skor2[coreysidx]
    elif skor2[coreysidx] > COREYSPROBS[coreyskey]:
        COREYSPROBS[coreyskey] = skor2[coreysidx]
    
""", '<string>', 'exec')

code_rps_meta_fixed4 = compile(
    """
import random
import secrets

RNA={'RR':'1','RP':'2','RS':'3','PR':'4','PP':'5','PS':'6','SR':'7','SP':'8','SS':'9'}
mix={'RR':'R','RP':'R','RS':'S','PR':'R','PP':'P','PS':'P','SR':'S','SP':'P','SS':'S'}
rot={'R':'P','P':'S','S':'R'}

if not input:
   DNA=[""]*3
   prin=['R']*36
   meta=['P']*6
   skor1=[[0]*36,[0]*36,[0]*36,[0]*36,[0]*36,[0]*36]
   skor2=[0]*6
else:
   for j in range(36):
       for i in range(4):
           skor1[i][j]*=0.8
       for i in range(4,6):
           skor1[i][j]*=0.5
       for i in range(0,6,2):
           skor1[i][j]-=(input==rot[rot[prin[j]]])
           skor1[i+1][j]-=(output==rot[rot[prin[j]]])
       for i in range(2,6,2):
           skor1[i][j]+=(input==prin[j])
           skor1[i+1][j]+=(output==prin[j])
       skor1[0][j]+=1.3*(input==prin[j])-0.3*(input==rot[prin[j]])
       skor1[1][j]+=1.3*(output==prin[j])-0.3*(output==rot[prin[j]])
   for i in range(6):
       skor2[i]=0.9*skor2[i]+(input==meta[i])-(input==rot[rot[meta[i]]])
   DNA[0]+=input
   DNA[1]+=output
   DNA[2]+=RNA[input+output]
   for i in range(3):
       j=min(11,len(DNA[2]))
       k=-1
       while j>1 and k<0:
             j-=1
             k=DNA[i].rfind(DNA[i][-j:],0,-1)
       prin[4*i]=DNA[0][j+k]
       prin[4*i+1]=rot[DNA[1][j+k]]
       k=DNA[i].rfind(DNA[i][-j:],0,j+k-1)
       k=k*(k>-1)-j*(k<0)
       prin[4*i+2]=mix[prin[4*i]+DNA[0][j+k]]
       prin[4*i+3]=mix[prin[4*i+1]+rot[DNA[1][j+k]]]
   for i in range(12,36):
       prin[i]=rot[prin[i-12]]
   for i in range(0,6,2):
       meta[i]=prin[skor1[i].index(max(skor1[i]))]
       meta[i+1]=rot[prin[skor1[i+1].index(max(skor1[i+1]))]]
if max(skor2)<1:
   output=random.choice("RPS")
else:
   output=rot[meta[skor2.index(max(skor2))]]
   
nonrandom_output = rot[meta[skor2.index(max(skor2))]]
COREYSPROBS = {}
for coreysidx in range(len(meta)):
    coreyskey = rot[meta[coreysidx]]
    if coreyskey not in COREYSPROBS.keys():
        COREYSPROBS[coreyskey] = skor2[coreysidx]
    elif skor2[coreysidx] > COREYSPROBS[coreyskey]:
        COREYSPROBS[coreyskey] = skor2[coreysidx]
    
""", '<string>', 'exec')

code_rps_meta_fixed5 = compile(
    """
import random
import secrets

RNA={'RR':'1','RP':'2','RS':'3','PR':'4','PP':'5','PS':'6','SR':'7','SP':'8','SS':'9'}
mix={'RR':'R','RP':'R','RS':'S','PR':'R','PP':'P','PS':'P','SR':'S','SP':'P','SS':'S'}
rot={'R':'P','P':'S','S':'R'}

if not input:
   DNA=[""]*3
   prin=['P']*36
   meta=['P']*6
   skor1=[[0]*36,[0]*36,[0]*36,[0]*36,[0]*36,[0]*36]
   skor2=[0]*6
else:
   for j in range(36):
       for i in range(4):
           skor1[i][j]*=0.8
       for i in range(4,6):
           skor1[i][j]*=0.5
       for i in range(0,6,2):
           skor1[i][j]-=(input==rot[rot[prin[j]]])
           skor1[i+1][j]-=(output==rot[rot[prin[j]]])
       for i in range(2,6,2):
           skor1[i][j]+=(input==prin[j])
           skor1[i+1][j]+=(output==prin[j])
       skor1[0][j]+=1.3*(input==prin[j])-0.3*(input==rot[prin[j]])
       skor1[1][j]+=1.3*(output==prin[j])-0.3*(output==rot[prin[j]])
   for i in range(6):
       skor2[i]=0.9*skor2[i]+(input==meta[i])-(input==rot[rot[meta[i]]])
   DNA[0]+=input
   DNA[1]+=output
   DNA[2]+=RNA[input+output]
   for i in range(3):
       j=min(11,len(DNA[2]))
       k=-1
       while j>1 and k<0:
             j-=1
             k=DNA[i].rfind(DNA[i][-j:],0,-1)
       prin[4*i]=DNA[0][j+k]
       prin[4*i+1]=rot[DNA[1][j+k]]
       k=DNA[i].rfind(DNA[i][-j:],0,j+k-1)
       k=k*(k>-1)-j*(k<0)
       prin[4*i+2]=mix[prin[4*i]+DNA[0][j+k]]
       prin[4*i+3]=mix[prin[4*i+1]+rot[DNA[1][j+k]]]
   for i in range(12,36):
       prin[i]=rot[prin[i-12]]
   for i in range(0,6,2):
       meta[i]=prin[skor1[i].index(max(skor1[i]))]
       meta[i+1]=rot[prin[skor1[i+1].index(max(skor1[i+1]))]]
if max(skor2)<1:
   output=random.choice("RPS")
else:
   output=rot[meta[skor2.index(max(skor2))]]
   
nonrandom_output = rot[meta[skor2.index(max(skor2))]]
COREYSPROBS = {}
for coreysidx in range(len(meta)):
    coreyskey = rot[meta[coreysidx]]
    if coreyskey not in COREYSPROBS.keys():
        COREYSPROBS[coreyskey] = skor2[coreysidx]
    elif skor2[coreysidx] > COREYSPROBS[coreyskey]:
        COREYSPROBS[coreyskey] = skor2[coreysidx]
    
""", '<string>', 'exec')

code_rps_meta_fixed6 = compile(
    """
import random
import secrets

RNA={'RR':'1','RP':'2','RS':'3','PR':'4','PP':'5','PS':'6','SR':'7','SP':'8','SS':'9'}
mix={'RR':'R','RP':'R','RS':'S','PR':'R','PP':'P','PS':'P','SR':'S','SP':'P','SS':'S'}
rot={'R':'P','P':'S','S':'R'}

if not input:
   DNA=[""]*3
   prin=['S']*36
   meta=['P']*6
   skor1=[[0]*36,[0]*36,[0]*36,[0]*36,[0]*36,[0]*36]
   skor2=[0]*6
else:
   for j in range(36):
       for i in range(4):
           skor1[i][j]*=0.8
       for i in range(4,6):
           skor1[i][j]*=0.5
       for i in range(0,6,2):
           skor1[i][j]-=(input==rot[rot[prin[j]]])
           skor1[i+1][j]-=(output==rot[rot[prin[j]]])
       for i in range(2,6,2):
           skor1[i][j]+=(input==prin[j])
           skor1[i+1][j]+=(output==prin[j])
       skor1[0][j]+=1.3*(input==prin[j])-0.3*(input==rot[prin[j]])
       skor1[1][j]+=1.3*(output==prin[j])-0.3*(output==rot[prin[j]])
   for i in range(6):
       skor2[i]=0.9*skor2[i]+(input==meta[i])-(input==rot[rot[meta[i]]])
   DNA[0]+=input
   DNA[1]+=output
   DNA[2]+=RNA[input+output]
   for i in range(3):
       j=min(11,len(DNA[2]))
       k=-1
       while j>1 and k<0:
             j-=1
             k=DNA[i].rfind(DNA[i][-j:],0,-1)
       prin[4*i]=DNA[0][j+k]
       prin[4*i+1]=rot[DNA[1][j+k]]
       k=DNA[i].rfind(DNA[i][-j:],0,j+k-1)
       k=k*(k>-1)-j*(k<0)
       prin[4*i+2]=mix[prin[4*i]+DNA[0][j+k]]
       prin[4*i+3]=mix[prin[4*i+1]+rot[DNA[1][j+k]]]
   for i in range(12,36):
       prin[i]=rot[prin[i-12]]
   for i in range(0,6,2):
       meta[i]=prin[skor1[i].index(max(skor1[i]))]
       meta[i+1]=rot[prin[skor1[i+1].index(max(skor1[i+1]))]]
if max(skor2)<1:
   output=random.choice("RPS")
else:
   output=rot[meta[skor2.index(max(skor2))]]
   
nonrandom_output = rot[meta[skor2.index(max(skor2))]]
COREYSPROBS = {}
for coreysidx in range(len(meta)):
    coreyskey = rot[meta[coreysidx]]
    if coreyskey not in COREYSPROBS.keys():
        COREYSPROBS[coreyskey] = skor2[coreysidx]
    elif skor2[coreysidx] > COREYSPROBS[coreyskey]:
        COREYSPROBS[coreyskey] = skor2[coreysidx]
    
""", '<string>', 'exec')

code_rps_meta_fixed7 = compile(
    """
import random
import secrets

RNA={'RR':'1','RP':'2','RS':'3','PR':'4','PP':'5','PS':'6','SR':'7','SP':'8','SS':'9'}
mix={'RR':'R','RP':'R','RS':'S','PR':'R','PP':'P','PS':'P','SR':'S','SP':'P','SS':'S'}
rot={'R':'P','P':'S','S':'R'}

if not input:
   DNA=[""]*3
   prin=['R']*36
   meta=['S']*6
   skor1=[[0]*36,[0]*36,[0]*36,[0]*36,[0]*36,[0]*36]
   skor2=[0]*6
else:
   for j in range(36):
       for i in range(4):
           skor1[i][j]*=0.8
       for i in range(4,6):
           skor1[i][j]*=0.5
       for i in range(0,6,2):
           skor1[i][j]-=(input==rot[rot[prin[j]]])
           skor1[i+1][j]-=(output==rot[rot[prin[j]]])
       for i in range(2,6,2):
           skor1[i][j]+=(input==prin[j])
           skor1[i+1][j]+=(output==prin[j])
       skor1[0][j]+=1.3*(input==prin[j])-0.3*(input==rot[prin[j]])
       skor1[1][j]+=1.3*(output==prin[j])-0.3*(output==rot[prin[j]])
   for i in range(6):
       skor2[i]=0.9*skor2[i]+(input==meta[i])-(input==rot[rot[meta[i]]])
   DNA[0]+=input
   DNA[1]+=output
   DNA[2]+=RNA[input+output]
   for i in range(3):
       j=min(11,len(DNA[2]))
       k=-1
       while j>1 and k<0:
             j-=1
             k=DNA[i].rfind(DNA[i][-j:],0,-1)
       prin[4*i]=DNA[0][j+k]
       prin[4*i+1]=rot[DNA[1][j+k]]
       k=DNA[i].rfind(DNA[i][-j:],0,j+k-1)
       k=k*(k>-1)-j*(k<0)
       prin[4*i+2]=mix[prin[4*i]+DNA[0][j+k]]
       prin[4*i+3]=mix[prin[4*i+1]+rot[DNA[1][j+k]]]
   for i in range(12,36):
       prin[i]=rot[prin[i-12]]
   for i in range(0,6,2):
       meta[i]=prin[skor1[i].index(max(skor1[i]))]
       meta[i+1]=rot[prin[skor1[i+1].index(max(skor1[i+1]))]]
if max(skor2)<1:
   output=random.choice("RPS")
else:
   output=rot[meta[skor2.index(max(skor2))]]
   
nonrandom_output = rot[meta[skor2.index(max(skor2))]]
COREYSPROBS = {}
for coreysidx in range(len(meta)):
    coreyskey = rot[meta[coreysidx]]
    if coreyskey not in COREYSPROBS.keys():
        COREYSPROBS[coreyskey] = skor2[coreysidx]
    elif skor2[coreysidx] > COREYSPROBS[coreyskey]:
        COREYSPROBS[coreyskey] = skor2[coreysidx]
    
""", '<string>', 'exec')

code_rps_meta_fixed8 = compile(
    """
import random
import secrets

RNA={'RR':'1','RP':'2','RS':'3','PR':'4','PP':'5','PS':'6','SR':'7','SP':'8','SS':'9'}
mix={'RR':'R','RP':'R','RS':'S','PR':'R','PP':'P','PS':'P','SR':'S','SP':'P','SS':'S'}
rot={'R':'P','P':'S','S':'R'}

if not input:
   DNA=[""]*3
   prin=['P']*36
   meta=['S']*6
   skor1=[[0]*36,[0]*36,[0]*36,[0]*36,[0]*36,[0]*36]
   skor2=[0]*6
else:
   for j in range(36):
       for i in range(4):
           skor1[i][j]*=0.8
       for i in range(4,6):
           skor1[i][j]*=0.5
       for i in range(0,6,2):
           skor1[i][j]-=(input==rot[rot[prin[j]]])
           skor1[i+1][j]-=(output==rot[rot[prin[j]]])
       for i in range(2,6,2):
           skor1[i][j]+=(input==prin[j])
           skor1[i+1][j]+=(output==prin[j])
       skor1[0][j]+=1.3*(input==prin[j])-0.3*(input==rot[prin[j]])
       skor1[1][j]+=1.3*(output==prin[j])-0.3*(output==rot[prin[j]])
   for i in range(6):
       skor2[i]=0.9*skor2[i]+(input==meta[i])-(input==rot[rot[meta[i]]])
   DNA[0]+=input
   DNA[1]+=output
   DNA[2]+=RNA[input+output]
   for i in range(3):
       j=min(11,len(DNA[2]))
       k=-1
       while j>1 and k<0:
             j-=1
             k=DNA[i].rfind(DNA[i][-j:],0,-1)
       prin[4*i]=DNA[0][j+k]
       prin[4*i+1]=rot[DNA[1][j+k]]
       k=DNA[i].rfind(DNA[i][-j:],0,j+k-1)
       k=k*(k>-1)-j*(k<0)
       prin[4*i+2]=mix[prin[4*i]+DNA[0][j+k]]
       prin[4*i+3]=mix[prin[4*i+1]+rot[DNA[1][j+k]]]
   for i in range(12,36):
       prin[i]=rot[prin[i-12]]
   for i in range(0,6,2):
       meta[i]=prin[skor1[i].index(max(skor1[i]))]
       meta[i+1]=rot[prin[skor1[i+1].index(max(skor1[i+1]))]]
if max(skor2)<1:
   output=random.choice("RPS")
else:
   output=rot[meta[skor2.index(max(skor2))]]
   
nonrandom_output = rot[meta[skor2.index(max(skor2))]]
COREYSPROBS = {}
for coreysidx in range(len(meta)):
    coreyskey = rot[meta[coreysidx]]
    if coreyskey not in COREYSPROBS.keys():
        COREYSPROBS[coreyskey] = skor2[coreysidx]
    elif skor2[coreysidx] > COREYSPROBS[coreyskey]:
        COREYSPROBS[coreyskey] = skor2[coreysidx]
    
""", '<string>', 'exec')

code_rps_meta_fixed9 = compile(
    """
import random
import secrets

RNA={'RR':'1','RP':'2','RS':'3','PR':'4','PP':'5','PS':'6','SR':'7','SP':'8','SS':'9'}
mix={'RR':'R','RP':'R','RS':'S','PR':'R','PP':'P','PS':'P','SR':'S','SP':'P','SS':'S'}
rot={'R':'P','P':'S','S':'R'}

if not input:
   DNA=[""]*3
   prin=['S']*36
   meta=['S']*6
   skor1=[[0]*36,[0]*36,[0]*36,[0]*36,[0]*36,[0]*36]
   skor2=[0]*6
else:
   for j in range(36):
       for i in range(4):
           skor1[i][j]*=0.8
       for i in range(4,6):
           skor1[i][j]*=0.5
       for i in range(0,6,2):
           skor1[i][j]-=(input==rot[rot[prin[j]]])
           skor1[i+1][j]-=(output==rot[rot[prin[j]]])
       for i in range(2,6,2):
           skor1[i][j]+=(input==prin[j])
           skor1[i+1][j]+=(output==prin[j])
       skor1[0][j]+=1.3*(input==prin[j])-0.3*(input==rot[prin[j]])
       skor1[1][j]+=1.3*(output==prin[j])-0.3*(output==rot[prin[j]])
   for i in range(6):
       skor2[i]=0.9*skor2[i]+(input==meta[i])-(input==rot[rot[meta[i]]])
   DNA[0]+=input
   DNA[1]+=output
   DNA[2]+=RNA[input+output]
   for i in range(3):
       j=min(11,len(DNA[2]))
       k=-1
       while j>1 and k<0:
             j-=1
             k=DNA[i].rfind(DNA[i][-j:],0,-1)
       prin[4*i]=DNA[0][j+k]
       prin[4*i+1]=rot[DNA[1][j+k]]
       k=DNA[i].rfind(DNA[i][-j:],0,j+k-1)
       k=k*(k>-1)-j*(k<0)
       prin[4*i+2]=mix[prin[4*i]+DNA[0][j+k]]
       prin[4*i+3]=mix[prin[4*i+1]+rot[DNA[1][j+k]]]
   for i in range(12,36):
       prin[i]=rot[prin[i-12]]
   for i in range(0,6,2):
       meta[i]=prin[skor1[i].index(max(skor1[i]))]
       meta[i+1]=rot[prin[skor1[i+1].index(max(skor1[i+1]))]]
if max(skor2)<1:
   output=random.choice("RPS")
else:
   output=rot[meta[skor2.index(max(skor2))]]
   
nonrandom_output = rot[meta[skor2.index(max(skor2))]]
COREYSPROBS = {}
for coreysidx in range(len(meta)):
    coreyskey = rot[meta[coreysidx]]
    if coreyskey not in COREYSPROBS.keys():
        COREYSPROBS[coreyskey] = skor2[coreysidx]
    elif skor2[coreysidx] > COREYSPROBS[coreyskey]:
        COREYSPROBS[coreyskey] = skor2[coreysidx]
    
""", '<string>', 'exec')

code_rps_meta_fix = compile(
    """
import random

RNA={'RR':'1','RP':'2','RS':'3','PR':'4','PP':'5','PS':'6','SR':'7','SP':'8','SS':'9'}
mix={'RR':'R','RP':'R','RS':'S','PR':'R','PP':'P','PS':'P','SR':'S','SP':'P','SS':'S'}
rot={'R':'P','P':'S','S':'R'}

if not input:
   DNA=[""]*3
   prin=['R']*18
   meta=['R']*6
   skor1=[[0]*18,[0]*18,[0]*18,[0]*18,[0]*18,[0]*18]
   skor2=[0]*6
else:
   for j in range(18):
       for i in range(4):
           skor1[i][j]*=0.8
       for i in range(4,6):
           skor1[i][j]*=0.5
       for i in range(0,6,2):
           skor1[i][j]-=(input==rot[rot[prin[j]]])
           skor1[i+1][j]-=(output==rot[rot[prin[j]]])
       for i in range(2,6,2):
           skor1[i][j]+=(input==prin[j])
           skor1[i+1][j]+=(output==prin[j])
       skor1[0][j]+=1.3*(input==prin[j])-0.3*(input==rot[prin[j]])
       skor1[1][j]+=1.3*(output==prin[j])-0.3*(output==rot[prin[j]])
   for i in range(6):
       skor2[i]=0.9*skor2[i]+(input==meta[i])-(input==rot[rot[meta[i]]])
   DNA[0]+=input
   DNA[1]+=output
   DNA[2]+=RNA[input+output]
   for i in range(3):
       j=min(21,len(DNA[2]))
       k=-1
       while j>1 and k<0:
             j-=1
             k=DNA[i].rfind(DNA[i][-j:],0,-1)
       prin[2*i]=DNA[0][j+k]
       prin[2*i+1]=rot[DNA[1][j+k]]
       k=DNA[i].rfind(DNA[i][-j:],0,j+k-1)
       prin[2*i]=mix[prin[2*i]+DNA[0][j+k]]
       prin[2*i+1]=mix[prin[2*i+1]+rot[DNA[1][j+k]]]
   for i in range(6,18):
       prin[i]=rot[prin[i-6]]
   for i in range(0,6,2):
       meta[i]=prin[skor1[i].index(max(skor1[i]))]
       meta[i+1]=rot[prin[skor1[i+1].index(max(skor1[i+1]))]]
output=rot[meta[skor2.index(max(skor2))]]
COREYSPROBS = {}
for coreysidx in range(len(meta)):
    coreyskey = rot[meta[coreysidx]]
    if coreyskey not in COREYSPROBS.keys():
        COREYSPROBS[coreyskey] = skor2[coreysidx]
    elif skor2[coreysidx] > COREYSPROBS[coreyskey]:
        COREYSPROBS[coreyskey] = skor2[coreysidx]
""", '<string>', 'exec')

code_io2fighting = compile(
    """
#Iocaine powder based AI

import random

# 2 different lengths of history, 3 kinds of history, both, mine, yours
# 3 different limit length of reverse learning
# 6 kinds of strategy based on Iocaine Powder
num_predictor = 27

if input=="":
    len_rfind = [20]
    limit = [10,20,60]
    beat = { "R":"P" , "P":"S", "S":"R"}
    not_lose = { "R":"PPR" , "P":"SSP" , "S":"RRS" } #50-50 chance
    my_his   =""
    your_his =""
    both_his =""
    list_predictor = [""]*num_predictor
    length = 0
    temp1 = { "PP":"1" , "PR":"2" , "PS":"3",
              "RP":"4" , "RR":"5", "RS":"6",
              "SP":"7" , "SR":"8", "SS":"9"}
    temp2 = { "1":"PP","2":"PR","3":"PS",
                "4":"RP","5":"RR","6":"RS",
                "7":"SP","8":"SR","9":"SS"} 
    who_win = { "PP": 0, "PR":1 , "PS":-1,
                "RP": -1,"RR":0, "RS":1,
                "SP": 1, "SR":-1, "SS":0}
    score_predictor = [0]*num_predictor
    output = random.choice("RPS")
    predictors = [output]*num_predictor
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
else:
    #update predictors
    if len(list_predictor[0])<5:
        front =0
    else:
        front =1
    for i in range (num_predictor):
        if predictors[i]==input:
            result ="1"
        else:
            result ="0"
        list_predictor[i] = list_predictor[i][front:5]+result #only 5 rounds before
    #history matching 1-6
    my_his += output
    your_his += input
    both_his += temp1[input+output]
    length +=1
    for i in range(1):
        len_size = min(length,len_rfind[i])
        j=len_size
        #both_his
        while j>=1 and not both_his[length-j:length] in both_his[0:length-1]:
            j-=1
        if j>=1:
            k = both_his.rfind(both_his[length-j:length],0,length-1)
            predictors[0+6*i] = your_his[j+k]
            predictors[1+6*i] = beat[my_his[j+k]]
        else:
            predictors[0+6*i] = random.choice("RPS")
            predictors[1+6*i] = random.choice("RPS")
        j=len_size
        #your_his
        while j>=1 and not your_his[length-j:length] in your_his[0:length-1]:
            j-=1
        if j>=1:
            k = your_his.rfind(your_his[length-j:length],0,length-1)
            predictors[2+6*i] = your_his[j+k]
            predictors[3+6*i] = beat[my_his[j+k]]
        else:
            predictors[2+6*i] = random.choice("RPS")
            predictors[3+6*i] = random.choice("RPS")
        j=len_size
        #my_his
        while j>=1 and not my_his[length-j:length] in my_his[0:length-1]:
            j-=1
        if j>=1:
            k = my_his.rfind(my_his[length-j:length],0,length-1)
            predictors[4+6*i] = your_his[j+k]
            predictors[5+6*i] = beat[my_his[j+k]]
        else:
            predictors[4+6*i] = random.choice("RPS")
            predictors[5+6*i] = random.choice("RPS")

    for i in range(3):
        temp =""
        search = temp1[(output+input)] #last round
        for start in range(2, min(limit[i],length) ):
            if search == both_his[length-start]:
                temp+=both_his[length-start+1]
        if(temp==""):
            predictors[6+i] = random.choice("RPS")
        else:
            collectR = {"P":0,"R":0,"S":0} #take win/lose from opponent into account
            for sdf in temp:
                next_move = temp2[sdf]
                if(who_win[next_move]==-1):
                    collectR[temp2[sdf][1]]+=3
                elif(who_win[next_move]==0):
                    collectR[temp2[sdf][1]]+=1
                elif(who_win[next_move]==1):
                    collectR[beat[temp2[sdf][0]]]+=1
            max1 = -1
            p1 =""
            for key in collectR:
                if(collectR[key]>max1):
                    max1 = collectR[key]
                    p1 += key
            predictors[6+i] = random.choice(p1)
    
    #rotate 9-27:
    for i in range(9,27):
        predictors[i] = beat[beat[predictors[i-9]]]
        
    #choose a predictor
    len_his = len(list_predictor[0])
    for i in range(num_predictor):
        sum = 0
        for j in range(len_his):
            if list_predictor[i][j]=="1":
                sum+=(j+1)*(j+1)
            else:
                sum-=(j+1)*(j+1)
        score_predictor[i] = sum
    max_score = max(score_predictor)
    #min_score = min(score_predictor)
    #c_temp = {"R":0,"P":0,"S":0}
    #for i in range (num_predictor):
        #if score_predictor[i]==max_score:
        #    c_temp[predictors[i]] +=1
        #if score_predictor[i]==min_score:
        #    c_temp[predictors[i]] -=1
    if max_score>0:
        predict = predictors[score_predictor.index(max_score)]
    else:
        predict = random.choice(your_his)
    output = random.choice(not_lose[predict])

    COREYSPROBS = {}
    for coreysidx in range(len(predictors)):
        coreyskey = beat[predictors[coreysidx]]
        if coreyskey not in COREYSPROBS.keys():
            COREYSPROBS[coreyskey] = score_predictor[coreysidx]
        elif score_predictor[coreysidx] > COREYSPROBS[coreyskey]:
            COREYSPROBS[coreyskey] = score_predictor[coreysidx]

""", '<string>', 'exec')

code_bayes14 = compile(
    """
# See http://overview.cc/RockPaperScissors for more information about rock, paper, scissors
# Extension to bayes13: Use also the csc function for singleopp and singlemy

from collections import defaultdict
import operator
import random

if input == "":
  score = {'RR': 0, 'PP': 0, 'SS': 0, 'PR': 1, 'RS': 1, 'SP': 1,'RP': -1, 'SR': -1, 'PS': -1,}
  cscore = {'RR': 'r', 'PP': 'r', 'SS': 'r', 'PR': 'b', 'RS': 'b', 'SP': 'b','RP': 'c', 'SR': 'c', 'PS': 'c',}
  beat = {'P': 'S', 'S': 'R', 'R': 'P'}
  cede = {'P': 'R', 'S': 'P', 'R': 'S'}
  rps = ['R', 'P', 'S']
  
  def counter_prob(probs):
    weighted_list = []
    for h in ['R', 'P', 'S']:
      weighted = 0
      for p in probs.keys():
        points = score[h+p]
        prob = probs[p]
        weighted += points * prob
      weighted_list.append((h, weighted))

    return max(weighted_list, key=operator.itemgetter(1))[0], dict(weighted_list)

  played_probs = defaultdict(lambda: 1)
  opp_probs = defaultdict(lambda: defaultdict(lambda: 1))
  my_probs = defaultdict(lambda: defaultdict(lambda: 1))
  both_probs = defaultdict(lambda: defaultdict(lambda: 1))

  singleopp_opp_probs = defaultdict(lambda: defaultdict(lambda: 1))
  singleopp_my_probs = defaultdict(lambda: defaultdict(lambda: 1))
  singleopp_both_probs = defaultdict(lambda: defaultdict(lambda: 1))

  singlemy_opp_probs = defaultdict(lambda: defaultdict(lambda: 1))
  singlemy_my_probs = defaultdict(lambda: defaultdict(lambda: 1))
  singlemy_both_probs = defaultdict(lambda: defaultdict(lambda: 1))

  opp2_probs = defaultdict(lambda: defaultdict(lambda: 1))
  my2_probs = defaultdict(lambda: defaultdict(lambda: 1))
  both2_probs = defaultdict(lambda: defaultdict(lambda: 1))

  singleopp_opp2_probs = defaultdict(lambda: defaultdict(lambda: 1))
  singleopp_my2_probs = defaultdict(lambda: defaultdict(lambda: 1))
  singleopp_both2_probs = defaultdict(lambda: defaultdict(lambda: 1))

  singlemy_opp2_probs = defaultdict(lambda: defaultdict(lambda: 1))
  singlemy_my2_probs = defaultdict(lambda: defaultdict(lambda: 1))
  singlemy_both2_probs = defaultdict(lambda: defaultdict(lambda: 1))

  win_probs = defaultdict(lambda: 1)
  lose_probs = defaultdict(lambda: 1)
  tie_probs = defaultdict(lambda: 1)

  singleopp_win_probs = defaultdict(lambda: 1)
  singleopp_lose_probs = defaultdict(lambda: 1)
  singleopp_tie_probs = defaultdict(lambda: 1)

  singlemy_win_probs = defaultdict(lambda: 1)
  singlemy_lose_probs = defaultdict(lambda: 1)
  singlemy_tie_probs = defaultdict(lambda: 1)

  opp_answers = {'c': 1, 'b': 1, 'r': 1}
  my_answers = {'c': 1, 'b': 1, 'r': 1}

  opp2_answers = {'c': 1, 'b': 1, 'r': 1}
  my2_answers = {'c': 1, 'b': 1, 'r': 1}  

  singleopp_opp_answers = {'c': 1, 'b': 1, 'r': 1}
  singleopp_my_answers = {'c': 1, 'b': 1, 'r': 1}

  singleopp_opp2_answers = {'c': 1, 'b': 1, 'r': 1}
  singleopp_my2_answers = {'c': 1, 'b': 1, 'r': 1}  

  singlemy_opp_answers = {'c': 1, 'b': 1, 'r': 1}
  singlemy_my_answers = {'c': 1, 'b': 1, 'r': 1}

  singlemy_opp2_answers = {'c': 1, 'b': 1, 'r': 1}
  singlemy_my2_answers = {'c': 1, 'b': 1, 'r': 1}

  patterndict = defaultdict(str)
  patterndict2 = defaultdict(str)
  opppatterndict = defaultdict(str)
  opppatterndict2 = defaultdict(str)
  mypatterndict = defaultdict(str)
  mypatterndict2 = defaultdict(str)

  csu = [0] * 6 # consecutive strategy usage
  csc = []  # consecutive strategy candidates
  singleopp_csu = [0] * 6 # consecutive strategy usage
  singleopp_csc = []  # consecutive strategy candidates
  singlemy_csu = [0] * 6 # consecutive strategy usage
  singlemy_csc = []  # consecutive strategy candidates

  output = random.choice(["R", "P", "S"])
  hist = "" 
  myhist = "" 
  opphist = "" 

  my = opp = my2 = opp2 =  ""
  singleopp_my = singleopp_opp = singleopp_my2 = singleopp_opp2 = ""
  singlemy_my = singlemy_opp = singlemy_my2 = singlemy_opp2 = ""

  sc = 0
  opp_strats = []
  singleopp_oppstrats = []
  singlemy_oppstrats = []
  COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
else:
  previous_opp_strats = opp_strats[:]
  previous_singleopp_oppstrats = singleopp_oppstrats[:]
  previous_singlemy_oppstrats = singlemy_oppstrats[:]
  previous_sc = sc

  sc = score[output + input]
  for i, c in enumerate(csc):
    if c == input:
      csu[i] += 1
    else:
      csu[i] = 0

  for i, c in enumerate(singleopp_csc):
    if c == input:
      singleopp_csu[i] += 1
    else:
      singleopp_csu[i] = 0

  for i, c in enumerate(singlemy_csc):
    if c == input:
      singlemy_csu[i] += 1
    else:
      singlemy_csu[i] = 0

  m = max(csu)
  opp_strats = [i for i, c in enumerate(csc) if csu[i] == m]

  m = max(singleopp_csu)
  singleopp_oppstrats = [i for i, c in enumerate(singleopp_csc) if singleopp_csu[i] == m]

  m = max(csu)
  singlemy_oppstrats = [i for i, c in enumerate(singlemy_csc) if singlemy_csu[i] == m]
  
  if previous_sc == 1:
    for s1 in previous_opp_strats:
      for s2 in opp_strats:
        win_probs[chr(s1)+chr(s2)] += 1

    for s1 in previous_singleopp_oppstrats:
      for s2 in singleopp_oppstrats:
        singleopp_win_probs[chr(s1)+chr(s2)] += 1

    for s1 in previous_singlemy_oppstrats:
      for s2 in singlemy_oppstrats:
        singlemy_win_probs[chr(s1)+chr(s2)] += 1
  
  if previous_sc == 0:
    for s1 in previous_opp_strats:
      for s2 in opp_strats:
        tie_probs[chr(s1)+chr(s2)] += 1

    for s1 in previous_singleopp_oppstrats:
      for s2 in singleopp_oppstrats:
        singleopp_tie_probs[chr(s1)+chr(s2)] += 1

    for s1 in previous_singlemy_oppstrats:
      for s2 in singlemy_oppstrats:
        singlemy_tie_probs[chr(s1)+chr(s2)] += 1

  if previous_sc == -1:
    for s1 in previous_opp_strats:
      for s2 in opp_strats:
        lose_probs[chr(s1)+chr(s2)] += 1
    for s1 in previous_singleopp_oppstrats:
      for s2 in singleopp_oppstrats:
        singleopp_lose_probs[chr(s1)+chr(s2)] += 1
    for s1 in previous_singlemy_oppstrats:
      for s2 in singlemy_oppstrats:
        singlemy_lose_probs[chr(s1)+chr(s2)] += 1

  if my and opp:
    opp_answers[cscore[input+opp]] += 1
    my_answers[cscore[input+my]] += 1
  if my2 and opp2:
    opp2_answers[cscore[input+opp2]] += 1
    my2_answers[cscore[input+my2]] += 1

  if singleopp_my and singleopp_opp:
    singleopp_opp_answers[cscore[input+singleopp_opp]] += 1
    singleopp_my_answers[cscore[input+singleopp_my]] += 1
  if singleopp_my2 and singleopp_opp2:
    singleopp_opp2_answers[cscore[input+singleopp_opp2]] += 1
    singleopp_my2_answers[cscore[input+singleopp_my2]] += 1

  if singlemy_my and singlemy_opp:
    singlemy_opp_answers[cscore[input+singlemy_opp]] += 1
    singlemy_my_answers[cscore[input+singlemy_my]] += 1
  if singlemy_my2 and singlemy_opp2:
    singlemy_opp2_answers[cscore[input+singlemy_opp2]] += 1
    singlemy_my2_answers[cscore[input+singlemy_my2]] += 1

  for length in range(min(10, len(hist)), 0, -2):
    pattern = patterndict[hist[-length:]]
    if pattern:
      for length2 in range(min(10, len(pattern)), 0, -2):
        patterndict2[pattern[-length2:]] += output + input
    patterndict[hist[-length:]] += output + input

  # singleopp
  for length in range(min(5, len(opphist)), 0, -1):
    pattern = opppatterndict[opphist[-length:]]
    if pattern:
      for length2 in range(min(10, len(pattern)), 0, -2):
        opppatterndict2[pattern[-length2:]] += output + input
    opppatterndict[opphist[-length:]] += output + input

  # singlemy
  for length in range(min(5, len(myhist)), 0, -1):
    pattern = mypatterndict[myhist[-length:]]
    if pattern:
      for length2 in range(min(10, len(pattern)), 0, -2):
        mypatterndict2[pattern[-length2:]] += output + input
    mypatterndict[myhist[-length:]] += output + input

  played_probs[input] += 1
  opp_probs[opp][input] += 1
  my_probs[my][input] += 1
  both_probs[my+opp][input] += 1

  opp2_probs[opp2][input] += 1
  my2_probs[my2][input] += 1
  both2_probs[my2+opp2][input] += 1

  hist += output + input
  myhist += output
  opphist += input

  my = opp = my2 = opp2 = ""
  singleopp_my = singleopp_opp = singleopp_my2 = singleopp_opp2 = ""
  singlemy_my = singlemy_opp = singlemy_my2 = singlemy_opp2 = ""

  for length in range(min(10, len(hist)), 0, -2):
    pattern = patterndict[hist[-length:]]
    if pattern != "":
      my = pattern[-2]
      opp = pattern[-1]
      for length2 in range(min(10, len(pattern)), 0, -2):
        pattern2 = patterndict2[pattern[-length2:]]
        if pattern2 != "":
          my2 = pattern2[-2]
          opp2 = pattern2[-1]
          break
      break

  # singleopp
  for length in range(min(5, len(opphist)), 0, -1):
    pattern = opppatterndict[opphist[-length:]]
    if pattern != "":
      singleopp_my = pattern[-2]
      singleopp_opp = pattern[-1]
      for length2 in range(min(10, len(pattern)), 0, -2):
        pattern2 = opppatterndict2[pattern[-length2:]]
        if pattern2 != "":
          singleopp_my2 = pattern2[-2]
          singleopp_opp2 = pattern2[-1]
          break
      break

  # singlemy
  for length in range(min(5, len(myhist)), 0, -1):
    pattern = mypatterndict[myhist[-length:]]
    if pattern != "":
      singlemy_my = pattern[-2]
      singlemy_opp = pattern[-1]
      for length2 in range(min(10, len(pattern)), 0, -2):
        pattern2 = mypatterndict2[pattern[-length2:]]
        if pattern2 != "":
          singlemy_my2 = pattern2[-2]
          singlemy_opp2 = pattern2[-1]
          break
      break

  probs = {}
  for hand in rps:
    probs[hand] = played_probs[hand]
        
  if my and opp:
    for hand in rps:
      probs[hand] *= opp_probs[opp][hand] * my_probs[my][hand] * both_probs[my+opp][hand]
      probs[hand] *= opp_answers[cscore[hand+opp]] * my_answers[cscore[hand+my]]


    csc = [opp, beat[opp], cede[opp], my, cede[my], beat[my]]
  
    strats_for_hand = {'R': [], 'P': [], 'S': []}
    for i, c in enumerate(csc):
      strats_for_hand[c].append(i)

    if sc == 1:
      pr = win_probs
    if sc == 0:
      pr = tie_probs
    if sc == -1:
      pr = lose_probs

    for hand in rps:
      for s1 in opp_strats:
        for s2 in strats_for_hand[hand]:
          probs[hand] *= pr[chr(s1)+chr(s2)]
  else:
    csc = []

  if singleopp_my and singleopp_opp:
    for hand in rps:
      probs[hand] *= singleopp_opp_probs[singleopp_opp][hand] * \
                     singleopp_my_probs[singleopp_my][hand] * \
                     singleopp_both_probs[singleopp_my+singleopp_opp][hand]
      probs[hand] *= singleopp_opp_answers[cscore[hand+singleopp_opp]] * singleopp_my_answers[cscore[hand+singleopp_my]]

    singleopp_csc = [singleopp_opp, beat[singleopp_opp], cede[singleopp_opp], singleopp_my, cede[singleopp_my], beat[singleopp_my]]
  
    strats_for_hand = {'R': [], 'P': [], 'S': []}
    for i, c in enumerate(singleopp_csc):
      strats_for_hand[c].append(i)

    if sc == 1:
      pr = singleopp_win_probs
    if sc == 0:
      pr = singleopp_tie_probs
    if sc == -1:
      pr = singleopp_lose_probs

    for hand in rps:
      for s1 in singleopp_oppstrats:
        for s2 in strats_for_hand[hand]:
          probs[hand] *= pr[chr(s1)+chr(s2)]
  else:
    singleopp_csc = []

  if singlemy_my and singlemy_opp:
    for hand in rps:
      probs[hand] *= singlemy_opp_probs[singlemy_opp][hand] * \
                     singlemy_my_probs[singlemy_my][hand] * \
                     singlemy_both_probs[singlemy_my+singlemy_opp][hand]
      probs[hand] *= singlemy_opp_answers[cscore[hand+singlemy_opp]] * singlemy_my_answers[cscore[hand+singlemy_my]]

    singlemy_csc = [singlemy_opp, beat[singlemy_opp], cede[singlemy_opp], singlemy_my, cede[singlemy_my], beat[singlemy_my]]
  
    strats_for_hand = {'R': [], 'P': [], 'S': []}
    for i, c in enumerate(singlemy_csc):
      strats_for_hand[c].append(i)

    if sc == 1:
      pr = singlemy_win_probs
    if sc == 0:
      pr = singlemy_tie_probs
    if sc == -1:
      pr = singlemy_lose_probs

    for hand in rps:
      for s1 in singlemy_oppstrats:
        for s2 in strats_for_hand[hand]:
          probs[hand] *= pr[chr(s1)+chr(s2)]
  else:
    singlemy_csc = []
                
  if my2 and opp2:
    for hand in rps:
      probs[hand] *= opp2_probs[opp2][hand] * my2_probs[my2][hand] * both2_probs[my2+opp2][hand]
      probs[hand] *= opp2_answers[cscore[hand+opp2]] * my2_answers[cscore[hand+my2]]

  if singleopp_my2 and singleopp_opp2:
    for hand in rps:
      probs[hand] *= singleopp_opp2_probs[singleopp_opp2][hand] *\
                     singleopp_my2_probs[singleopp_my2][hand] *\
                     singleopp_both2_probs[singleopp_my2+singleopp_opp2][hand]
      probs[hand] *= singleopp_opp2_answers[cscore[hand+singleopp_opp2]] * \
                     singleopp_my2_answers[cscore[hand+singleopp_my2]]

  if singlemy_my2 and singlemy_opp2:
    for hand in rps:
      probs[hand] *= singlemy_opp2_probs[singlemy_opp2][hand] *\
                     singlemy_my2_probs[singlemy_my2][hand] *\
                     singlemy_both2_probs[singlemy_my2+singlemy_opp2][hand]
      probs[hand] *= singlemy_opp2_answers[cscore[hand+singlemy_opp2]] * \
                     singlemy_my2_answers[cscore[hand+singlemy_my2]]

  output, COREYSPROBS = counter_prob(probs)
""", '<string>', 'exec')

code_skipblenderfixed = compile(
    """
    # https://web.archive.org/web/20161130225835/http://www.rpscontest.com/entry/5679128418189312
if input == "":

    import collections
    import random
    import secrets

    def random_index(ps):
        t = sum(ps)
        r = random.uniform(0, t)
        x = 0
        for i, p in enumerate(ps):
            x += p
            if r <= x:
                break
        return i

    class MarkovTree:
        def __init__(self, counts = None):
            self.counts = [0 for _ in range(3)]
            self.children = None
            self.total = 0

        def update_helper(self, h, i, p, d, skips):
            stop = False
            for j in range(p, len(h)):
                k = h[j]
                self.counts[i] += 2
                self.total += 2
                if stop or d >= 9 or skips >= 3:
                    return
                d += 1
                if self.children is None:
                    self.children = [None for _ in range(4)]
                    self.children[3] = MarkovTree()
                if self.children[k] is None:
                    self.children[k] = MarkovTree()
                    stop = True
                self.children[3].update_helper(h, i, j + 1, d, skips + 1)
                self = self.children[k]

        def update(self, h, i):
            self.update_helper(h, i, 0, 0, 0)

        def predict_helper(self, h, p, n0):
            for j in range(p, len(h)):
                k = h[j]
                for i, x in enumerate(self.counts):
                    n0[i] += x
                if self.children is None:
                    return
                self.children[3].predict_helper(h, j + 1, n0)
                child = self.children[k]
                if child is None:
                    return
                self = child

        def predict(self, h):
            n0 = [1, 1, 1]
            self.predict_helper(h, 0, n0)
            return n0

    R, P, S = 0, 1, 2
    index = {"R": R, "P": P, "S": S}
    name = ("R", "P", "S")
    beat = (P, S, R)

    tree = MarkovTree()

    history = collections.deque([])

else:

    i = index[input]
    j = index[output]

    tree.update(history, i)
    history.appendleft(i)
    history.appendleft(j)

counts = tree.predict(history)
hypotheses = [random_index(counts) for _ in range(3)]
scores = [0, 0, 0]
for i, _ in enumerate(scores):
    for h in hypotheses:
        if i == beat[h]:
            scores[i] += 1
        elif h == beat[i]:
            scores[i] -= 1
best = max(scores)
m = random.choice([i for i in range(3) if scores[i] == best])
output = name[m]
nonrandom_output = output
COREYSPROBS = {'R':scores[0],'P':scores[1],'S':scores[2]}
    
""", '<string>', 'exec')

code_ZASMC = compile(
    """
# greedy history pattern match
# greedy - use highest available order and most recent
# order 6 - should be slightly faster than order 7 but just as good
# use my hands, op hands, as well as both to predict next hands
# update only the orders that are >= the order last used to make the prediction
#   - something from PPM compression
# choose the strategy with the best score
# choose the meta strategy using an additional history matching corrector
#    which matches using past meta choices and the corresponding payoffs
#    even though I use a history matcher, it seems order 1 is the best
# a little bit of decay goes a long way
# this bot is mostly deterministic
# this beats bayes14 and bayes15 approximately 70% - 80% of the time, but it's
#    still not as effective and ties a lot more with other bots

# --------------------- initialization -----------------------------
if not input:
    import random, collections
    
    # micro-optimizations
    rchoice = random.choice
    randint = random.randint
    uniform = random.uniform
    
    # global constants and maps
    R, P, S = 0, 1, 2
    RPS = [R, P, S]
    T, W, L = 0, 1, 2
    PAYOFFS = [T, W, L]
    
    tonum = {'R':R, 'P':P, 'S':S}
    tostr = {R:'R', P:'P', S:'S'}
    
    scoreround = [[T, L, W], [W, T, L], [L, W, T]]
    frompayoff = [[R, P, S], [P, S, R], [S, R, P]]
    ties, beats, loses = frompayoff[T], frompayoff[W], frompayoff[L]
    
    # more specific variables
    topoints = [0, 1, -1]
    
    encode1hand = [1,2,3]
    decode1hand = [None, R, P, S]
    
    encode2hands = [[1,2,3], [4,5,6], [7,8,9]]
    decode2hands = [None,(R,R),(R,P),(R,S),(P,R),(P,P),(P,S),(S,R),(S,P),(S,S)]
    
    encodemeta = [[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12], [13, 14, 15, 16, 17, 18]]
    
    ORDER = 6
    CORRECTOR_ORDER = 1
    power3 = [0] + [3 ** i for i in range(ORDER)]
    power9 = [0] + [9 ** i for i in range(ORDER)]
    power18 = [0] + [18 ** i for i in range(CORRECTOR_ORDER)]
    
    my_history = collections.defaultdict(lambda: None)
    op_history = collections.defaultdict(lambda: None)
    both_history = collections.defaultdict(lambda: None)
    corrector_history = collections.defaultdict(lambda: None)
    corrector_history[0] = [0] * 6
    corrector_predict = corrector_history[0]
    
    my_last_used_order = 0
    op_last_used_order = 0
    both_last_used_order = 0
    corrector_last_used_order = 0
    
    DECAY = 0.94
    STRATEGIES = 18
    
    META_DECAY = 0.94
    META_STRATEGIES = 6
    
    # 0-2 is my my, 3-5 is my op, 6-8 is both op my, 9-11 is op op, 12-14 is both my, 15-17 is both op
    strategy_scores = [0.0] * STRATEGIES
    reverse_strategy_scores = [0.0] * STRATEGIES
    next_hands = [None] * STRATEGIES
    
    next_hands_meta = [None] * META_STRATEGIES
    
    meta_picks = []
    
    # bookkeeping
    my_hands_encoded = []
    op_hands_encoded = []
    both_hands_encoded = []
    hands_played = 0
    
    # first hand
    output = tostr[rchoice(RPS)]
    nonrandom_output = 'Q'
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
# --------------------- turn -----------------------------
else:
    # bookkeeping
    last_hand_index = hands_played
    hands_played += 1
    
    my_last_hand = tonum[output]
    my_last_hand_encoded = encode1hand[my_last_hand]
    
    op_last_hand = tonum[input]
    op_last_hand_encoded = encode1hand[op_last_hand]
    
    both_last_hand_encoded = encode2hands[my_last_hand][op_last_hand]
    
    last_payoff = scoreround[my_last_hand][op_last_hand]
    
    my_hands_encoded.append(my_last_hand_encoded)
    op_hands_encoded.append(op_last_hand_encoded)
    both_hands_encoded.append(both_last_hand_encoded)
    
    # decay the scores
    for i in range(STRATEGIES):
        strategy_scores[i] *= DECAY
        reverse_strategy_scores[i] *= DECAY
    
    # update the scores of the strategies if played at least 2 hands
    if hands_played > 1:
        for i in range(STRATEGIES):
            strategy_scores[i] += topoints[scoreround[next_hands[i]][op_last_hand]]
            reverse_strategy_scores[i] += topoints[scoreround[next_hands[i]][my_last_hand]]
        
        # update the corrector history
        last_meta_pick = encodemeta[last_payoff][pick_meta_strategy]
        meta_picks.append(last_meta_pick)
        
        corrector_update_index = 0
        if corrector_last_used_order == 0:
            for i in range(META_STRATEGIES):
                corrector_history[0][i] *= META_DECAY
                corrector_history[0][i] += topoints[scoreround[next_hands_meta[i]][op_last_hand]]
        
        corrector_predict = corrector_history[0]
        
        for order in range(1, CORRECTOR_ORDER+1 if hands_played-1 > CORRECTOR_ORDER else hands_played-1):
            corrector_predict_index = corrector_update_index * 18 + last_meta_pick
            corrector_update_index += meta_picks[-order-1] * power18[order]
            
            if order >= corrector_last_used_order:
                if corrector_history[corrector_update_index] is None:
                    corrector_history[corrector_update_index] = [0] * 6
                for i in range(META_STRATEGIES):
                    corrector_history[corrector_update_index][i] *= META_DECAY
                    corrector_history[corrector_update_index][i] += topoints[scoreround[next_hands_meta[i]][op_last_hand]]
            
            corrector_try_get = corrector_history[corrector_predict_index]
            
            if corrector_try_get is not None:
                corrector_predict = corrector_try_get
                corrector_last_used_order = order
        
    # update the history, order 0 as a special case
    my_update_index = 0
    op_update_index = 0
    both_update_index = 0
    if my_last_used_order == 0:
        my_history[0] = last_hand_index
    if op_last_used_order == 0:
        op_history[0] = last_hand_index
    if both_last_used_order == 0:
        both_history[0] = last_hand_index
    
    # start the prediction with the zeroth order
    my_predict = my_history[0]
    op_predict = op_history[0]
    both_predict = both_history[0]
    
    # update the higher orders and predict the next hand
    for order in range(1, ORDER+1 if hands_played > ORDER else hands_played):
        my_predict_index = my_update_index * 3 + my_last_hand_encoded
        op_predict_index = op_update_index * 3 + op_last_hand_encoded
        both_predict_index = both_update_index * 9 + both_last_hand_encoded
        
        my_update_index += my_hands_encoded[-order-1] * power3[order]
        op_update_index += op_hands_encoded[-order-1] * power3[order]
        both_update_index += both_hands_encoded[-order-1] * power9[order]
        
        if order >= my_last_used_order:
            my_history[my_update_index] = last_hand_index
        if order >= op_last_used_order:
            op_history[op_update_index] = last_hand_index
        if order >= both_last_used_order:
            both_history[both_update_index] = last_hand_index
        
        my_try_get = my_history[my_predict_index]
        op_try_get = op_history[op_predict_index]
        both_try_get = both_history[both_predict_index]
        
        if my_try_get is not None:
            my_predict = my_try_get
            my_last_used_order = order
        if op_try_get is not None:
            op_predict = op_try_get
            op_last_used_order = order
        if both_try_get is not None:
            both_predict = both_try_get
            both_last_used_order = order
    
    my_base_prediction = decode2hands[both_hands_encoded[my_predict]]
    op_base_prediction = decode2hands[both_hands_encoded[op_predict]]
    both_base_prediction = decode2hands[both_hands_encoded[both_predict]]
    
    # update each strategy's prediction
    for f in PAYOFFS:
        next_hands[f] = frompayoff[f][my_base_prediction[0]]
        next_hands[f+3] = frompayoff[f][my_base_prediction[1]]
        next_hands[f+6] = frompayoff[f][op_base_prediction[0]]
        next_hands[f+9] = frompayoff[f][op_base_prediction[1]]
        next_hands[f+12] = frompayoff[f][both_base_prediction[0]]
        next_hands[f+15] = frompayoff[f][both_base_prediction[1]]
    
    # use the strategy with the best score
    max_score = max(strategy_scores)
    max_list = [i for i in range(STRATEGIES) if strategy_scores[i] == max_score]
    pick_strategy = rchoice(max_list)
    
    # use the reverse strategy with the best score
    max_reverse_score = max(reverse_strategy_scores)
    max_reverse_list = [i for i in range(STRATEGIES) if reverse_strategy_scores[i] == max_reverse_score]
    pick_reverse_strategy = rchoice(max_reverse_list)
    
    # update the meta strategy
    for f in PAYOFFS:
        next_hands_meta[f] = frompayoff[f][next_hands[pick_strategy]]
        next_hands_meta[f+3] = frompayoff[f][next_hands[pick_reverse_strategy]]
    
    max_meta_score = max(corrector_predict)
    max_meta_list = [i for i in range(META_STRATEGIES) if corrector_predict[i] == max_meta_score]
    pick_meta_strategy = rchoice(max_meta_list)
    
    # play the next hand
    output = tostr[next_hands_meta[pick_meta_strategy]]
    COREYSPROBS = {}
    for coreysindex in range(len(corrector_predict)):
        coreyskey = tostr[next_hands_meta[coreysindex]]
        if coreyskey not in COREYSPROBS.keys():
            COREYSPROBS[coreyskey] = corrector_predict[coreysindex]
        elif corrector_predict[coreysindex] > COREYSPROBS[coreyskey]:
            COREYSPROBS[coreyskey] = corrector_predict[coreysindex]
    nonrandom_output = output
    
""", '<string>', 'exec')

code_flight825 = compile(
    """
import math,random
p=0.825
memory=8
if input=="":
    count=0
    wcount=0
    lcount=0
    prev=dict()
    hist=""
    abc=0
else:
    count+=1
    hist+=input
    n=1000-count
    if wcount-lcount>n-(math.sqrt(1+8*(1-p)*(n+1)*(n+1))-1)/2:
        abc=1
if (input=='R' and output=='P') or (input=='P' and output=='S') or (input=='S' and output=='R'):
    wcount+=1
else:
    lcount+=1
l=len(hist)
if l>=memory+2:
    word=str(hist[l-memory-2])
    for i in range(-memory-1,-2):
        word=word+str(hist[l+i])
    if not word in prev:
        prev[word]=dict()
    if hist[l-1] in prev[word]:
        prev[word][hist[l-1]]+=1
    else:
        prev[word][hist[l-1]]=1
if count<50:
    output = random.choice(["R","P","S"])
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
else:
    guess=str(hist[l-memory])
    for i in range(-memory+1,0):
        guess=guess+str(hist[l+i])
    if guess in prev:
        predict=max(prev[guess], key = lambda x: prev[guess].get(x) )
        output={'R':'P', 'P':'S', 'S':'R'}[predict]
        COREYSPROBS = {}
        for k in prev[guess]:
            COREYSPROBS[{'R':'P','P':'S','S':'R'}[k]] = prev[guess][k]
    else:
        output=random.choice(["R","P","S"])
        COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
if abc==1:
        output=random.choice(["R","P","S"])
        COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
hist+=output
    
""", '<string>', 'exec')

code_funbots = compile(
    """
import math,random
memory=8
strike=3
if input=="":
    wcount=0
    lcount=0
    count=0
    hist=""
    prev=dict()
    prev2=dict()
    guess=""
else:
    hist+=input
    count+=2
l=len(hist)
if (input=='R' and output=='P') or (input=='P' and output=='S') or (input=='S' and output=='R'):
    wcount+=1
elif (input=='P' and output=='R') or (input=='S' and output=='P') or (input=='R' and output=='S'):
    lcount+=1
    if count>=memory+2:
        if guess in prev2 and predict in prev2[guess]:
            prev2[guess][predict]-=1
if count>=memory+2:
    word=str(hist[l-memory-2])
    for i in range(-memory-1,-2):
        word=word+str(hist[l+i])
    if not word in prev:
        prev[word]=dict()
        prev2[word]=dict()
    if hist[count-1] in prev[word]:
        prev[word][hist[l-1]]+=1
    else:
        prev[word][hist[l-1]]=1
        prev2[word][hist[l-1]]=strike
if count<8 or wcount>60+lcount:
    output=random.choice(["R","P","S"])
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
else:
    guess=str(hist[count-memory])
    for i in range(-memory+1,0):
        guess=guess+str(hist[count+i])
    if guess in prev:
        predict=max(prev[guess], key = lambda x: prev[guess].get(x) )
        # COREYSPROBS = {}
        # for k in prev[guess]:
        #     COREYSPROBS[{'R':'P','P':'S','S':'R'}[k]] = prev[guess][k]
        if prev2[guess][predict]>0:
            output={'R':'P', 'P':'S', 'S':'R'}[predict]
        else:
            output=predict
    else:
        output=random.choice(["R","P","S"])
        COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}

hist+=output
    
""", '<string>', 'exec')

code_decay6 = compile(
    """
import random, math

#emphasize prediction by squaring

if input == "":
    history = []
    time_steps_random = 10
    cc_to_n = {'RR':0, 'RP':1, 'RS':2, 'PR':3, 'PP':4, 'PS':5, 'SR':6, 'SP':7, 'SS':8}
    n_to_cc = {0:'RR', 1:'RP', 2:'RS', 3:'PR', 4:'PP', 5:'PS', 6:'SR', 7:'SP', 8:'SS'}
    c_to_n = {'R':0, 'P':1, 'S':2}
    n_to_c = {0:'R', 1:'P', 2:'S'}
    n_to_score = {0:0, 1:-1, 2:1, 3:1, 4:0, 5:-1, 6:-1, 7:1, 8:0}
    match_length = 1
    decay = 0.9
else:
    history.append(cc_to_n[output+input])
if len(history) >= time_steps_random:
    longest = 0
    prediction = [1.0]*3
    for t in range(match_length, len(history)-1):
        if history[t-match_length:t] == history[-match_length:]:
            prediction_index = c_to_n[n_to_cc[history[t]][1]]
            prediction = [x * decay for x in prediction]
            prediction[prediction_index] += 1
    prediction = [x/sum(prediction) for x in prediction]
    prediction = [x*x for x in prediction]
    if random.random() < prediction[0] / sum(prediction):
        output = "P"
    elif random.random() < prediction[1] / sum(prediction[1:]):
        output = "S"
    else:
        output = "R"

    COREYSPROBS = {'P':prediction[0],'S':prediction[1],'R':prediction[2]}
else:
    output = random.choice(['R','P','S'])
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
    
""", '<string>', 'exec')

code_derp = compile(
    """
import random, math

#gets more random if predicting incorrectly
#gives up if losing midway

if input == "":
    padding = 7.0
    decay_model = 0.67
    decay_strategy = 0.95
    history = []
    weight_strategy = 1
    weight_random = 1
    prediction = ''
    my_prev = ''
    first_order = {}
    for i in ['R','P','S']:
        for j in ['R','P','S']:
            for k in ['R','P','S']:
                first_order[i+j+k] = padding
    win = ['RP', 'PS', 'SR']
    total_wins = 0
    t = 0
    nihilism = False
else:
    t += 1
    if (input+output) in win:
        total_wins += 1
    
    if prediction == input:
        weight_strategy += 1
    else:
        weight_random += 1
    weight_strategy *= decay_strategy
    weight_random *= decay_strategy
        
    your_prev = input
    history.append((my_prev, your_prev))
if len(history) >= 2:
    if t > 500 and total_wins < 166:
        nihilism = True
    if nihilism:
        output = random.choice(['R','P','S'])
        COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
    else:
        pred = [1,1,1]
        pred[0] = first_order[my_prev + your_prev + 'R']
        pred[1] = first_order[my_prev + your_prev + 'P']
        pred[2] = first_order[my_prev + your_prev + 'S']
        first_order[history[-2][0] + history[-2][1] + history[-1][1]] += 1
        first_order[my_prev + your_prev + 'R'] *= decay_model
        first_order[my_prev + your_prev + 'P'] *= decay_model
        first_order[my_prev + your_prev + 'S'] *= decay_model

        pred = [x/sum(pred) for x in pred]
        pred = [math.pow(x,2) for x in pred]

        COREYSPROBS = {'R':pred[0],'P':pred[1],'S':pred[2]}

        if pred[0] > pred[1] and pred[0] > pred[2]:
            prediction = 'R'
        elif pred[1] > pred[2]:
            prediction = 'P'
        else:
            prediction = 'S'

        if random.random() < weight_strategy / (weight_strategy+weight_random):
            if random.random() < pred[0] / sum(pred):
                output = "P"
            elif random.random() < pred[1] / sum(pred[1:]):
                output = "S"
            else:
                output = "R"
        else:
            output = random.choice(['R','P','S'])
else:
    output = random.choice(['R','P','S'])
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
    
my_prev = output
    
""", '<string>', 'exec')

code_metaioc = compile(
    """
from random import *

def findMatchMe(history_me, history_yu, depth):
    prefix = history_me[-depth:]
    i = history_me[:-1].rfind(prefix)
    if i > -1:
        return history_yu[i+depth]
    else:
        return None

def findMatchYu(history_yu, depth):
    prefix = history_yu[-depth:]
    i = history_yu[:-1].rfind(prefix)
    if i > -1:
        return history_yu[i+depth]
    else:
        return None

def findMatchUs(history_us, depth):
    prefix = history_us[-depth*2:]
    i = history_us[:-2].rfind(prefix)
    if i > -1:
        return history_us[i+depth*2].upper()
    else:
        return None

if input == "":
    shift = {None:None, 'R':'P', 'P':'S', 'S':'R'}
    initial_random_moves = 10
    decay = 0.6
    meta_decay = 0.6
    history_me, history_yu, history_us = '', '', ''
    markov_depth = 13
    weight_random = 1
    weights = [[[1 for k in range(markov_depth)] for j in range(3)] for i in range(3)]
    predictions = [[[None for k in range(markov_depth)] for j in range(3)] for i in range(3)]
    meta_prediction = [None for j in range(3)]
    meta_weight = [1 for j in range(3)]
    chosen_prediction = None
    meta_chosen_prediction = None
    output = choice(['R','P','S'])
    chosen_i, chosen_j, chosen_k = -1, -1, -1
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
else:
    #store histories
    history_me += output
    history_yu += input
    history_us += input.lower() + output
    
    #decay models and update weights based on input observation
    weight_random *= decay
    if chosen_prediction != None:
        if input != chosen_prediction:
            weight_random += 1
        for i in range(3):
            for j in range(3):
                for k in range(markov_depth):
                    weights[i][j][k] *= decay
                    if input == predictions[i][j][k]:
                        weights[i][j][k] += 1

    #decay meta model
    if meta_chosen_prediction != None:
        for i in range(3):
            meta_weight[i] *= meta_decay
            if meta_chosen_prediction == meta_prediction[i]:
                meta_weight[i] += 1
                
    #make new predictions
    prediction = None
    for i in range(3):
        for j in range(3):
            for k in range(markov_depth):
                predictions[i][j][k] = None
                if i == 0:
                    prediction = findMatchMe(history_me, history_yu, k+1)
                elif i == 1:
                    prediction = findMatchYu(history_yu, k+1)
                else:
                    prediction = findMatchUs(history_us, k+1)
                for j2 in range(j): #iocane-shift
                    prediction = shift[shift[prediction]]
                if prediction != None:
                    predictions[i][j][k] = prediction
    
    #choose highest weight (possibly random)
    chosen_prediction = None
    mx = weight_random
    for i in range(3):
        for j in range(3):
            for k in range(markov_depth):
                if predictions[i][j][k] != None and weights[i][j][k] > mx:
                    mx = weights[i][j][k]
                    chosen_prediction = predictions[i][j][k]

    #meta-iocane
    meta_chosen_prediction = None
    mx = 0
    for i in range(3):
        meta_prediction[i] = chosen_prediction
        for i2 in range(i):
            meta_prediction[i] = shift[shift[meta_prediction[i]]]
        if meta_weight[i] > mx:
            mx = meta_weight[i]
            meta_chosen_prediction = meta_prediction[i]

    #choose action
    if meta_chosen_prediction == None:
        output = choice(['R','P','S'])
    else:
        output = shift[meta_chosen_prediction]
    
""", '<string>', 'exec')

code_notorious = compile(
    """
import random, math


if input == "":
    padding = 8.0
    decay_model = 0.67
    decay_strategy = 0.95
    history = []
    weight_strategy = 1
    weight_random = 1
    prediction = ''
    my_prev = ''
    first_order = {}
    for N in ['R','P','S']:
        for O in ['R','P','S']:
            for T in ['R','P','S']:
                first_order[N+O+T] = padding
else:
    if prediction == input:
        weight_strategy += 1
    else:
        weight_random += 1
    weight_strategy *= decay_strategy
    weight_random *= decay_strategy
        
    your_prev = input
    history.append((my_prev, your_prev))
if len(history) >= 2:
    pred = [1,1,1]
    pred[0] = first_order[my_prev + your_prev + 'R']
    pred[1] = first_order[my_prev + your_prev + 'P']
    pred[2] = first_order[my_prev + your_prev + 'S']
    first_order[history[-2][0] + history[-2][1] + history[-1][1]] += 1
    first_order[my_prev + your_prev + 'R'] *= decay_model
    first_order[my_prev + your_prev + 'P'] *= decay_model
    first_order[my_prev + your_prev + 'S'] *= decay_model

    pred = [x/sum(pred) for x in pred]
    pred = [math.pow(x,2) for x in pred]

    COREYSPROBS = {'R':pred[0],'P':pred[1],'S':pred[2]}

    if pred[0] > pred[1] and pred[0] > pred[2]:
        prediction = 'R'
    elif pred[1] > pred[2]:
        prediction = 'P'
    else:
        prediction = 'S'

    if random.random() < weight_strategy / (weight_strategy+weight_random):
        if random.random() < pred[0] / sum(pred):
            output = "P"
        elif random.random() < pred[1] / sum(pred[1:]):
            output = "S"
        else:
            output = "R"
    else:
        output = random.choice(['R','P','S'])
else:
    output = random.choice(['R','P','S'])
    COREYSPROBS = {'R':.33,'P':.33,'S':.33}

my_prev = output

    
""", '<string>', 'exec')

code_john2 = compile(
    """
##same as John debugged again, just different weights
import random
from time import sleep

class Igralec:
    ime = 'vpisi ime'
    def __init__(self):
        self.lama = 0.9 #lama = lambda
        self.prevm = 0
        self.prevh = 0
        self.prevm2 = 0
        self.prevh2 = 0
        self.hist = [[[20.0 for x in range(3)] for x in range(3)] for x in range(3)]
        #hist[MOJ PREJ%u0160NJI][NJEGOV PREJ%u0160NJI][REZULTAT]
    def update(self):
        for i in range(3):
            self.hist[self.prevm2][self.prevh2][i] *= self.lama
            if self.prevh == i:  self.hist[self.prevm2][self.prevh2][i] += 1
    def inflate(self, s):
        a = 1.0/sum(s)
        s = [x*a for x in s]
        return s
    def toSum(self, s):
        #iz seznama %u0161tevil naredi seznam, ki je pru %u010Dlen
        #pru %u010Dlen, drug %u010Dlen je drug %u010Dlen plus pru %u010Dlen,
        #tret %u010Dlen je pru plus drug plus tret %u010Dlen...
        for i in range(1,len(s)):
            s[i] = s[i] + s[i-1]
        return s
    def getHisNext(self):
        ver = self.toSum(self.inflate(list(self.hist[self.prevm][self.prevh])))
        hisNext = random.random()
        if hisNext < ver[0]:
            return 0
        elif hisNext < ver[1]:
            return 1
        else:
            return 2
    def poteza(self, tocke, prev): #for competition
        if prev == 'XX' or prev == ('X', 'X'):
            a = random.choice(['K', 'S', 'P'])
            self.prevm = self.stToin(a)
            return a
        self.prevh2 = self.prevh
        self.prevh = self.stToin(prev[1])
        ### kle se neha prev, delamo naprej
        self.update()
        self.prevm2 = self.prevm
        self.prevm = self.beat(self.getHisNext())
        return self.inTost(self.prevm)
    def rpscontINIT(self):
        return self.sloToEng(self.poteza(0, 'XX'))
    def rpscont(self, pr):
        return self.sloToEng(self.poteza(0, 'X' + self.engToSlo(pr)))
    ### dit-like funkcije ###
    def inTost(self, a):
        if a == 0:
            return 'K'
        elif a == 1:
            return 'S'
        else:
            return 'P'
    def sloToEng(self, a):
        if a == 'K':
            return 'R'
        else:
            return a #P == P and S == S
    def engToSlo(self, a):
        if a== 'R':
            return 'K'
        else:
            return a
    def stToin(self, a):
        if a== 'K':
            return 0
        elif a == 'S':
            return 1
        else:
            return 2
    def beat(self, a):
        if a == 0:
            return 2
        elif a == 1:
            return 0
        else:
            return 1
    ###konc dit-like funkcij####
    
#test %u010De se%u0161tevanje deluje
# a = Igralec()
# mp2 = ''
# mr = 'K'
# mp = a.poteza(0, 'XX')
# from time import sleep
# for i in range(1000):
    # #print 'mm', mp2, mp
    # mp2 = mp
    # mp = a.poteza(0, mp+mr)
    # mr = mp2
    # sleep(0.1)
    # print 'delam',
# print 1/0

#konc testa
if input == '':
    a = Igralec()
    output = a.rpscontINIT()
    #print output
else:
    output = a.rpscont(input)
    #print output

    
""", '<string>', 'exec')

code_cereal4 = compile(
    '''
# See http://overview.cc/RockPaperScissors for more information about rock, paper, scissors
import random
import math
import collections

if input == "":
    rps = ['R', 'P', 'S']
    beat = {'R': 'P', 'P': 'S', 'S': 'R'}
    cede = {'R': 'S', 'P': 'R', 'S': 'P'}
    hist = ""
    output = random.choice(rps)
    patterns = collections.defaultdict(lambda: 10)
    rnd = .0
    startround = repeated = countered = 0
    played = [0, 0, 0]
    nocoupons = 1
    longcoupons = 1.0
    hist = ""
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
elif rnd == 0:
    rnd += 1.1
    hist += output + input
    opplastmove = output
    mymost = [output]
    mmycounter = 0
    mylastmove = output = random.choice(rps)
else:
    if cede[input] in mymost:
        mmycounter += 1
    played[rps.index(input)] = 1
    for i in range(1, int(min(1+len(hist)/2,5))):
        patterns[hist[-i*2:]+input] += 1
        patterns[hist[-i*2:]+output.lower()] += 1

    if min(played) == 1:
        if rnd - startround > 8:
            longcoupons += 1
        nocoupons += 1
        startround = rnd
        played = [0, 0, 0]

    hist += output + input
    probs = [1.0, 1.0, 1.0]

    myprobs = [1.0, 1.0, 1.0]
    for i in range(1, int(min(1+len(hist)/2,5))):
        probs = [p*patterns[hist[-i*2:]+h] for p,h in zip(probs,rps)]

    for i in range(int(min(1+len(hist)/2,6)),0, -1):
        tmp = [patterns[hist[-i*2:]+h.lower()] for h in rps]
        if sum(tmp) > 30:
            myprobs = tmp
            break

    mymost = [e for e, p in zip(rps, myprobs) if p == max(myprobs)]

    probs = [(p/max(probs))**0.03 for p in probs]

    if mmycounter/rnd > 0.41:
        for e in mymost:
            probs[rps.index(beat[e])] += .01/(1.41-(mmycounter/rnd))**6

    if rnd - startround > 8 and longcoupons/nocoupons > 0.117 and sum(played)==2:
       probs[played.index(0)] -= 2*longcoupons/nocoupons 
    if opplastmove == input:
        repeated += 1
    if beat[mylastmove] == input:
        countered += 1
    if repeated / rnd > 0.3:
        probs[rps.index(input)] += 0.01 / (0.999-repeated/rnd)**2
    if countered / rnd > 0.33:
        probs[rps.index(beat[output])] += 0.01 / (0.999- countered/rnd)**2
    vals = list(map(lambda x: random.uniform(0, x), probs))
    opplastmove = input
    mylastmove = output
    output = beat[rps[vals.index(max(vals))]]

    COREYSPROBS = {}
    for coreysidx in range(len(rps)):
        coreyskey = beat[rps[coreysidx]]
        if coreyskey not in COREYSPROBS.keys():
            COREYSPROBS[coreyskey] = vals[coreysidx]
        elif vals[coreysidx] > COREYSPROBS[coreyskey]:
            COREYSPROBS[coreyskey] = vals[coreysidx]

    rnd += 1

    
''', '<string>', 'exec')

code_zdgsteal = compile(
    '''
#!/usr/bin/python
# -*- coding: utf-8 -*-
# Name: zai_switch_markov1_ml
# Author: zdg
# Email: rpscontest.b73@gishpuppy.com
# the email is disposable in case it gets spammed
#
# let's try out some bayes inference with 1-st order markov model wrappers on
#    various strategies as the hypotheses
# uses last diff as well as last payoff to transition for the 1-st order
# uses ML i.e. maximum likelihood learning by minimizing the entropy
# also have some ad-hoc decay which helps a lot

# --------------------- initialization -----------------------------

if not input:
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
    import random
    import collections
    import math

    # micro-optimizations

    rchoice = random.choice
    log = math.log

    # global constants and maps
    # using lists and dictionaries because function call and arithmetic is slow

    (R, P, S) = (0, 1, 2)
    RPS = [R, P, S]
    (T, W, L) = (R, P, S)
    PAYOFFS = RPS
    tr = {
        'R': R,
        'P': P,
        'S': S,
        R: 'R',
        P: 'P',
        S: 'S',
        }
    sub = [[T, L, W], [W, T, L], [L, W, T]]
    add = [[R, P, S], [P, S, R], [S, R, P]]
    (ties, beats, loses) = (add[T], add[W], add[L])

    pts = [0, 1, -1]
    near = [1, 0, 0]

    enc1 = [1, 2, 3]
    dec1 = [None, R, P, S]

    enc2 = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
    dec2 = [
        None,
        (R, R),
        (R, P),
        (R, S),
        (P, R),
        (P, P),
        (P, S),
        (S, R),
        (S, P),
        (S, S),
        ]

    seed = rchoice(RPS)


    # entropy of a 3 vector - doesn't need to be normalized

    def entropy(vec):
        (v1, v2, v3) = vec

        # force coerce to float

        total = v1 + v2 + v3 + 0.0

        # normalize

        (p1, p2, p3) = (v1 / total, v2 / total, v3 / total)
        if total > 0.0:
            return -((p1 * log(p1) if v1 > 0.0 else 0.0)) - ((p2
                    * log(p2) if v2 > 0.0 else 0.0)) - ((p3
                    * log(p3) if v3 > 0.0 else 0.0))
        else:

            # entropy of nothing is undefined, but here it's useful to define it
            #    as the same as completely random

            return 1.0


    def pick_max(vec):
        max_val = max(vec)
        max_list = [i for i in range(len(vec)) if vec[i] == max_val]
        return rchoice(max_list)


    # calculate the hand with the best expected value against the given op hand
    # random only in case of ties

    def expected(vec):
        expected_payoffs = [vec[S] - vec[P], vec[R] - vec[S], vec[P]
                            - vec[R]]
        max_expected = max(expected_payoffs)
        max_list = [i for i in RPS if expected_payoffs[i]
                    == max_expected]
        return rchoice(max_list)


    def normalize(vec):
        factor = 1.0 / sum(vec)
        for i in range(len(vec)):
            vec[i] *= factor


    # greedy history pattern matcher
    # ORDER is the largest context size
    # BASE is the base of the numerical encoding
    # encodes sequences of numbers from 1...BASE as a BASE-adic number
    # encodes the empty sequence as 0
    # apparently this encoding is called a bijective base-BASE system on wikipedia

    class GHPM:

        def __init__(self, ORDER, BASE):
            self.ORDER = ORDER
            self.BASE = BASE
            self.powers = [0] + [BASE ** i for i in range(ORDER)]
            self.hist = []
            self.contexts = collections.defaultdict(lambda : None)
            self.pred = None
            self.preds = [None] * (ORDER + 1)

        def update(self, next_val, up_fun):
            self.hist.append(next_val)

            # update the history, order 0 as a special case

            up_ix = 0
            self.contexts[0] = up_fun(self.contexts[0])

            # start the prediction with the zeroth order

            self.pred = self.contexts[0]

            # update the higher orders and prediction

            elems = len(self.hist)
            for order in range(1, (self.ORDER + 1 if elems
                                > self.ORDER else elems)):
                pred_ix = up_ix * self.BASE + next_val

                up_ix += self.hist[-order - 1] * self.powers[order]
                self.contexts[up_ix] = up_fun(self.contexts[up_ix])

                try_get = self.contexts[pred_ix]
                self.preds[order] = try_get
                if try_get is not None:
                    self.pred = try_get


    NUM_BOTS = 9
    BOTS = range(NUM_BOTS)
    DECAY = 0.98
    next_hands = [seed for _ in BOTS]

    # contexts = [[0.0, 0.0, 0.0] for _ in BOTS]
    # contexts = [[1.0, 1.0, 1.0] for _ in BOTS]

    # 1st order transition matrix
    # use both last diff and last payoff for each bot

    markov = [[[1.0 for _ in RPS] for _ in range(9)] for _ in BOTS]

    # initialize history matching strategies

    my_ghpm = GHPM(6, 3)
    op_ghpm = GHPM(6, 3)
    both_ghpm = GHPM(6, 9)

    # first hand is completely random - no reason to do otherwise

    next_hand = seed
    output = tr[next_hand]

    bot_diffs = [[] for _ in BOTS]
    bot_payoffs = [[] for _ in BOTS]

    # bookkeeping

    hands = 1
    last_ix = 0
    score = 0
else:

# --------------------- turn -----------------------------

    last_my = tr[output]
    last_op = tr[input]
    last_payoff = sub[last_my][last_op]

    # keep track of how bots have been playing

    for b in BOTS:
        bot_diffs[b].append(sub[last_op][next_hands[b]])
        bot_payoffs[b].append(sub[last_my][next_hands[b]])

    # update the zeroth order markov models
    # for b in BOTS:
        # contexts[b][T] *= DECAY
        # contexts[b][W] *= DECAY
        # contexts[b][L] *= DECAY
        # contexts[b][bot_diffs[b][-1]] += 1.0

    # update the first order markov models

    if hands >= 2:
        for b in BOTS:
            (last_last_diff, last_diff) = (bot_diffs[b][-2],
                    bot_diffs[b][-1])
            last_last_payoff = bot_payoffs[b][-2]
            last_last_state = enc2[last_last_diff][last_last_payoff] - 1

            markov[b][last_last_state][T] *= DECAY
            markov[b][last_last_state][W] *= DECAY
            markov[b][last_last_state][L] *= DECAY
            markov[b][last_last_state][last_diff] += 1.0

    # use first order markov to calculate each bot's next state distribution

    next_dist = [None] * NUM_BOTS
    for b in BOTS:
        last_state = enc2[bot_diffs[b][-1]][bot_payoffs[b][-1]] - 1
        next_dist[b] = markov[b][last_state]

    # update the history matchers

    my_ghpm.update(enc1[last_my], lambda _: last_ix)
    op_ghpm.update(enc1[last_op], lambda _: last_ix)
    both_ghpm.update(enc2[last_op][last_my], lambda _: last_ix)

    # update predictions

    next_hands[1] = last_op
    next_hands[2] = last_my

    # hpm

    both_hist = both_ghpm.hist

    # my hist

    (pred_op, pred_my) = dec2[both_hist[my_ghpm.pred]]
    next_hands[3] = pred_op
    next_hands[4] = pred_my

    # op hist

    (pred_op, pred_my) = dec2[both_hist[op_ghpm.pred]]
    next_hands[5] = pred_op
    next_hands[6] = pred_my

    # both hist

    (pred_op, pred_my) = dec2[both_hist[both_ghpm.pred]]
    next_hands[7] = pred_op
    next_hands[8] = pred_my

    # find the most likely bot i.e. one with least entropy

    entropies = [-entropy(next_dist[b]) for b in BOTS]
    most_likely = pick_max(entropies)

    pred_next = [None] * 3
    for h in RPS:
        pred_next[add[h][next_hands[most_likely]]] = \
            next_dist[most_likely][h]

    t = sum(pred_next)
    r = random.uniform(0, t)
    x = 0
    for (i, p) in enumerate(pred_next):
        x += p
        if r <= x:
            break
    output = tr[beats[i]]
    COREYSPROBS = {'R':pred_next[2],'P':pred_next[0],'S':pred_next[1]}

    # bookkeeping

    hands += 1
    last_ix += 1
    score += pts[last_payoff]

    # if hands % 100 == 0:
        # print markov
        # print next_dist

    
''', '<string>', 'exec')

code_ZCC2 = compile(
    '''
xrange = range
imap = lambda *args, **kwargs: list(map(*args, **kwargs))
# this mixes the predictions of the two strategies instead of just switches

# --------------------- initialization -----------------------------
if not input:
   import random
   import math
   import itertools, operator, functools
   
   # constants
   ROUNDS = 1000
   R, P, S = 0, 1, 2
   RPS_NUM = (R, P, S)
   RPS_STR = 'RPS'
   WIN, TIE, LOSE = 1, 0, -1
   
   def to_num(h):
      return RPS_STR.index(h)

   def to_str(i):
      return RPS_STR[i]

   def play(h1, h2):
      return (h1 - h2 + 4) % 3 - 1

   def diff(h1, h2):
      return (h1 - h2 + 3) % 3

   def undiff(h, d):
      return (h + d + 3) % 3

   def beats(h):
      return undiff(h, WIN)

   def loses(h):
      return undiff(h, LOSE)

   def ties(h):
      return undiff(h, TIE)

   def pick_random(lst):
      i = random.randint(0, len(lst) - 1)
      return (i, lst[i])

   def my_max(val1, val2):
      return val1 if val1[1] > val2[1] else val2

   def pick_max(lst):
      return reduce(my_max, enumerate(lst))

   def pick_weighted(lst, sumlst=None):
      r = random.uniform(0.0, sumlst or sum(lst))
      acc = 0.0
      for i, w in enumerate(lst):
         acc += w
         if r <= acc:
            return (i, w)

   def skew(lst):
      m = max(float(max(lst)), 1.0)
      # m = min(lst) / 2.0
      for i, x in enumerate(lst):
         lst[i] = (x / m)
         # lst[i] = x - m
   
   def shift(lst):
      m = min(min(lst), 0)
      return [(x - m) for x in lst]
   
   def random_hand():
      return random.choice(RPS_NUM)
   
   COUNT_ORDER_MAX = 7
   POWERS3 = tuple(3 ** x for x in xrange(COUNT_ORDER_MAX+1))
   
   # little endian ternary number
   def ternary(vec):
      return sum(imap(operator.mul, vec, POWERS3))

   def emptycounts(n):
      return tuple(tuple([0.0] * 3 for x in RPS_NUM) for i in xrange(n))

   def weigh_match(hands_played):
      return 2 ** (hands_played / 14.0)

   def weigh_order(weight, order):
      return weight * 2 ** (order)
   
   op_counts = tuple(emptycounts(POWERS3[n]) for n in xrange(COUNT_ORDER_MAX+1))
   my_counts = tuple(emptycounts(POWERS3[n]) for n in xrange(COUNT_ORDER_MAX+1))
   
   STRATEGIES = 2
   DECAY = 0.9
   strategy_scores = [0.0] * STRATEGIES
   next_hand_weights = [[0.0] * 3 for x in xrange(STRATEGIES)]
   count_random = 0
   
   my_hands = []
   op_hands = []
   
   output = to_str(random_hand())
   my_hands.append(to_num(output))
   COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
   
# --------------------- turn -----------------------------
else:
   op_hands.append(to_num(input))
   
   hands_played = len(my_hands)
   my_last_hand = my_hands[-1]
   op_last_hand = op_hands[-1]
   my_last_score = play(my_last_hand, op_last_hand)
   op_last_score = play(op_last_hand, my_last_hand)
   
   for i in xrange(STRATEGIES):
      # score the strategies
      last_score = 0.0
      for j in RPS_NUM:
         last_score += next_hand_weights[i][j] * play(j, op_last_hand)
      strategy_scores[i] *= DECAY
      strategy_scores[i] += last_score
   
   # update the counts
   for order in xrange(min(hands_played - 1, COUNT_ORDER_MAX) + 1):
      # update counts for my hands
      my_ix = ternary(my_hands[-order-1:-1])
      my_counts[order][my_ix][my_last_score][my_last_hand] += weigh_match(hands_played)
      # update counts for op hands
      op_ix = ternary(op_hands[-order-1:-1])
      op_counts[order][op_ix][op_last_score][op_last_hand] += weigh_match(hands_played)
   
   # use counts to determine the next move
   next_hand_weights = [[0.0] * 3 for x in xrange(STRATEGIES)]
   for order in xrange(min(hands_played, COUNT_ORDER_MAX) + 1):
      # use my and op hands to determine the next move
      my_ix = ternary(my_hands[-order:] if order != 0 else [])
      my_curr_counts = my_counts[order][my_ix]
      op_ix = ternary(op_hands[-order:] if order != 0 else [])
      op_curr_counts = op_counts[order][op_ix]
      for i in RPS_NUM:
         next_hand_weights[0][i] += weigh_order(my_curr_counts[LOSE][beats(i)], order)
         next_hand_weights[1][i] += weigh_order(op_curr_counts[WIN][loses(i)], order)
         # next_hand_weights[2][i] += weigh_order(op_curr_counts[WIN][loses(i)], order) + weigh_order(op_curr_counts[TIE][loses(i)], order) + weigh_order(op_curr_counts[LOSE][loses(i)], order)
         # next_hand_weights[2][i] += weigh_order(my_curr_counts[TIE][loses(i)], order)
         # next_hand_weights[3][i] += weigh_order(op_curr_counts[TIE][loses(i)], order)
         # next_hand_weights[2][i] += weigh_order(my_curr_counts[WIN][ties(i)], order)
         # next_hand_weights[3][i] += weigh_order(op_curr_counts[LOSE][ties(i)], order)
   
   next_hand_weights_total = [0.0] * 3
   for i in xrange(STRATEGIES):
      skew(next_hand_weights[i])
      if strategy_scores[i] < 0:
         for j in RPS_NUM:
            next_hand_weights_total[j] += next_hand_weights[i][beats(j)] * (-strategy_scores[i])
      else:
         for j in RPS_NUM:
            next_hand_weights_total[j] += next_hand_weights[i][ties(j)] * (strategy_scores[i])
   
   # shifted_weights = [max(p, 0) for p in next_hand_weights_total]
   # shifted_weights = shift(next_hand_weights_total)
   shifted_weights = (next_hand_weights_total)
   COREYSPROBS = {'R':shifted_weights[0],'P':shifted_weights[1],'S':shifted_weights[2]}
   if sum(shifted_weights) <= 0:
      # count_random += 1
      output = to_str(random_hand())
   else:
      output = to_str(ties(pick_weighted(shifted_weights)[0]))
   # if hands_played % 99 == 0:
      # print(strategy_scores)
   my_hands.append(to_num(output))
   
   # if hands_played == 999:
      # print(count_random)

    
''', '<string>', 'exec')

code_ZMH3 = compile(
    '''
xrange = range
# mixes the strategies that try to play in the opponent's shoes

# --------------------- initialization -----------------------------
if not input:
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
    import random, math
    import itertools, operator, collections
    
    R, P, S = 0, 1, 2
    RPS = [R, P, S]
    T, W, L = R, P, S
    PAYOFFS = [T, W, L]
    to_win = [W, T, L]
    convert = {'R':R, 'P':P, 'S':S, R:'R', P:'P', S:'S'}
    subh = [[T, L, W], [W, T, L], [L, W, T]]
    addh = [[R, P, S], [P, S, R], [S, R, P]]
    ties, beats, loses = addh[T], addh[W], addh[L]
    POWER3 = [3 ** n for n in xrange(10)]
    
    def empty_vec(n):
        return [0.0] * n

    def uniform_vec(n):
        return [1.0 / n] * n
    
    def mix_scores(u, v, d):
        e = 1.0 - d
        for i in xrange(len(u)):
            u[i] = u[i] * d + v[i] * e
    
    def predictability(u, sqrt=math.sqrt):
        uniform = 1.0 / len(u)
        acc = 0.0
        for x in u:
            acc += (x - uniform) ** 2
        return sqrt(acc)

    def normalize(u):
        factor = 1.0 / sum(u)
        for i in xrange(len(u)):
            u[i] *= factor

    def pick_weighted(v):
        u = random.random()
        acc = 0.0
        for i, p in enumerate(v):
            acc += p
            if u <= acc:
                return (i, p)
    
    # def score_counts(counts):
        # score = 0
        # for s in PAYOFFS:
            # score += counts[s] * s
        # return score
    
    # mixes multiple strategies
    class Mixer(object):
        def __init__(self, decay_function, dropswitch, *bots):
            self.bots = bots
            self.bot_num = len(self.bots)
            self.decay_function = decay_function
            self.dropswitch = dropswitch
            self.next_hands = [None] * self.bot_num
            self.scores = [None] * self.bot_num
            self.COREYSPROBS = [-1,-1,-1]
      
        def update(self):
            # update scores
            op_last_hand = op_hands[-1]
            for i in xrange(self.bot_num):
                last_hand = self.next_hands[i]
                if last_hand is None:
                    continue
                payoff = [last_hand[ties[op_last_hand]], last_hand[beats[op_last_hand]], last_hand[loses[op_last_hand]]]
                if payoff[L] > payoff[W] and self.dropswitch:
                    self.scores[i] = None
                else:
                    if self.scores[i] is None:
                        self.scores[i] = uniform_vec(3)
                    mix_scores(self.scores[i], payoff, self.decay_function(len(op_hands)))
         
            # compute next hand
            next_hand = empty_vec(3)
            for i in xrange(self.bot_num):
                bot_next_hand = self.bots[i].next_hand
                self.next_hands[i] = bot_next_hand
                if bot_next_hand is None:
                    continue
                curr_score = self.scores[i]
                if curr_score is None:
                    continue
                nonrandomness = predictability(curr_score)
                # nonrandomness = 1.0
                for s in PAYOFFS:
                    pscore = curr_score[s] * nonrandomness
                    offset = addh[L][s]
                    for h in RPS:
                        next_hand[h] += bot_next_hand[addh[offset][h]] * pscore
            self.COREYSPROBS = next_hand
            if sum(next_hand) <= 0.1:
                self.next_hand = pick_weighted(uniform_vec(3))[0]
            else:
                normalize(next_hand)
                self.next_hand = pick_weighted(next_hand)[0]
    
    class History(object):
        def __init__(self, order, offset, *handss):
            self.order = order
            self.offset = offset
            self.handss = handss
            self.handss_num = len(handss)
            self.all = []
            self.counts = collections.defaultdict(list)
        
        def update(self):
            # update the all list
            last_hands = [self.handss[i][-1] for i in xrange(self.handss_num)]
            code = 0
            for i in xrange(self.handss_num):
                code += last_hands[i] * POWER3[i]
            self.all.append(code)
            hands_len = len(self.all)
            
            # update for the previous hand
            for order in xrange(min(self.order, hands_len - 1)):
                index = tuple(self.all[-order-2:-1])
                self.counts[index].append(code)
            
            # setup the next one and predict
            prediction = [0] * 3
            for order in xrange(min(self.order, hands_len)):
                index = tuple(self.all[-order-1:])
                past = self.counts[index]
                if past:
                    mine, ops = self.decode3(past[-1])
                    if subh[mine][int(ops)] == W:
                        prediction[mine] = 1.0
            if sum(prediction) < 0.01:
                self.next_hand = None
            else:
                normalize(prediction)
                offsetted = [0.0] * 3
                offsetted[addh[self.offset][R]] = prediction[R]
                offsetted[addh[self.offset][P]] = prediction[P]
                offsetted[addh[self.offset][S]] = prediction[S]
                self.next_hand = offsetted
        
        def decode3(self, code):
            hands = []
            for i in xrange(self.handss_num):
                hands.append(code % 3)
                code /= 3
            return hands
    
    class ConstantBot(object):
        def __init__(self, op_moves, my_moves, offset):
            self.op_moves = op_moves
            self.my_moves = my_moves
            self.offset = offset
            next_hand_p = [0.0] * 3
            next_hand_p[addh[offset][my_moves[-1]]] = 1.0
            self.next_hand = next_hand_p
        
        def update(self):
            pass
    
    class BeatLastBot(object):
        def __init__(self, op_moves, my_moves, offset):
            self.op_moves = op_moves
            self.my_moves = my_moves
            self.offset = offset
        
        def update(self):
            next_hand_p = [0.0] * 3
            next_hand_p[addh[self.offset][self.op_moves[-1]]] = 1.0
            self.next_hand = next_hand_p
    
    op_hands = []
    my_hands = []
    payoffs = []
    
    output = convert[random.choice(RPS)]
# --------------------- turn -----------------------------
else:
    last_input, last_output = convert[input], convert[output]
    my_hands.append(last_output)
    op_hands.append(last_input)
    payoffs.append(subh[last_output][last_input])
    hands_played = len(op_hands)
    
    # bot initialization after first turn
    if hands_played == 1:
        # constantbot = ConstantBot(op_hands, my_hands, T)
        # constantbot2 = ConstantBot(my_hands, op_hands, W)
        beatlastbot = BeatLastBot(op_hands, my_hands, T)
        beatlastbot2 = BeatLastBot(my_hands, op_hands, W)
        hist = History(5, T, my_hands, op_hands)
        hist2 = History(5, W, op_hands, my_hands)
        
        mixer = Mixer(lambda x:0.9, False, beatlastbot, beatlastbot2, hist, hist2)
    
    # constantbot.update()
    # constantbot2.update()
    beatlastbot.update()
    beatlastbot2.update()
    hist.update()
    hist2.update()
    mixer.update()
    
    output = convert[mixer.next_hand]
    COREYSPROBS = mixer.COREYSPROBS
    COREYSPROBS = {'R':COREYSPROBS[0],'P':COREYSPROBS[1],'S':COREYSPROBS[2]}

    
''', '<string>', 'exec')

code_zaibayes = compile(
    '''
xrange = range
# Name: zai_mix_markov1_bayes
# Author: zdg
# Email: rpscontest.b73@gishpuppy.com
# the email is disposable in case it gets spammed
# 
# let's try out some bayes inference with 1-st order markov model wrappers on
#    various strategies as the hypotheses
# uses approximate bayes inference i.e. weighted hypotheses by likelihood
# also have some ad-hoc decay which helps a lot

# --------------------- initialization -----------------------------
if not input:
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
    import random, collections, math
    
    # micro-optimizations
    rchoice = random.choice
    log = math.log
    exp = math.exp
    
    # global constants and maps
    # using lists and dictionaries because function call and arithmetic is slow
    R, P, S = 0, 1, 2
    RPS = [R, P, S]
    T, W, L = R, P, S
    PAYOFFS = RPS
    tr = {'R':R, 'P':P, 'S':S, R:'R', P:'P', S:'S'}
    sub = [[T, L, W], [W, T, L], [L, W, T]]
    add = [[R, P, S], [P, S, R], [S, R, P]]
    ties, beats, loses = add[T], add[W], add[L]
    
    pts = [0, 1, -1]
    near = [1, 0, 0]
    
    enc1 = [1,2,3]
    dec1 = [None, R, P, S]
    
    enc2 = [[1,2,3], [4,5,6], [7,8,9]]
    dec2 = [None,(R,R),(R,P),(R,S),(P,R),(P,P),(P,S),(S,R),(S,P),(S,S)]
    
    seed = rchoice(RPS)
    
    def pick_max(vec):
        max_val = max(vec)
        max_list = [i for i in xrange(len(vec)) if vec[i] == max_val]
        return rchoice(max_list)
    
    # calculate the hand with the best expected value against the given op hand
    # random only in case of ties
    def expected(vec):
        expected_payoffs = [vec[S] - vec[P], vec[R] - vec[S], vec[P] - vec[R]]
        max_expected = max(expected_payoffs)
        max_list = [i for i in RPS if expected_payoffs[i] == max_expected]
        return rchoice(max_list)
    
    def normalize(vec):
        factor = 1.0 / sum(vec)
        for i in xrange(len(vec)):
            vec[i] *= factor
    
    def normalized(vec):
        factor = 1.0 / sum(vec)
        return [v * factor for v in vec]
    
    # greedy history pattern matcher
    # ORDER is the largest context size
    # BASE is the base of the numerical encoding
    # encodes sequences of numbers from 1...BASE as a BASE-adic number
    # encodes the empty sequence as 0
    # apparently this encoding is called a bijective base-BASE system on wikipedia
    class GHPM:
        def __init__(self, ORDER, BASE):
            self.ORDER = ORDER
            self.BASE = BASE
            self.powers = [0] + [BASE ** i for i in xrange(ORDER)]
            self.hist = []
            self.contexts = collections.defaultdict(lambda: None)
            self.pred = None
            self.preds = [None] * (ORDER+1)
        
        def update(self, next_val, up_fun):
            self.hist.append(next_val)
            # update the history, order 0 as a special case
            up_ix = 0
            self.contexts[0] = up_fun(self.contexts[0])
            
            # start the prediction with the zeroth order
            self.pred = self.contexts[0]
            
            # update the higher orders and prediction
            elems = len(self.hist)
            for order in xrange(1, self.ORDER+1 if elems > self.ORDER else elems):
                pred_ix = up_ix * self.BASE + next_val
                
                up_ix += self.hist[-order-1] * self.powers[order]
                self.contexts[up_ix] = up_fun(self.contexts[up_ix])
                
                try_get = self.contexts[pred_ix]
                self.preds[order] = try_get
                if try_get is not None:
                    self.pred = try_get
    
    class ConstantBot:
        def __init__(self, h, reversed=False):
            self.pred = h
        
        def update(self, op_hist, my_hist):
            pass
    
    class LastBot:
        def __init__(self, h, reversed=False):
            self.pred = h
            self.reversed = reversed
        
        def update(self, op_hist, my_hist):
            self.pred = op_hist[-1] if not self.reversed else my_hist[-1]
    
    # doesn't update wrapped ghpm
    class GHPMBot:
        def __init__(self, h, ghpm, reversed=False):
            self.pred = h
            self.ghpm = ghpm
            self.reversed = reversed
        
        def update(self, op_hist, my_hist):
            self.pred = (op_hist if not self.reversed else my_hist)[self.ghpm.pred]
    
    # this wraps around a deterministic bot using 0-th order markov (counts)
    class Markov0Hypothesis:
        def __init__(self, bot, decay, reversed=False):
            self.reversed = reversed
            # uniform prior
            self.pred_diff_dist = [1.0] * 3
            self.pred_dist_norm = [1.0 / 3.0] * 3
            self.recency_factor = 1.0 / decay
            self.counting_weight = 1.0
            
            # wraps over a deterministic bot
            self.bot = bot
        
        def update(self, op_hist, my_hist):
            if self.reversed:
                op_hist, my_hist = my_hist, op_hist
            # update probabilities for last time
            last_diff = sub[op_hist[-1]][self.bot.pred]
            
            self.counting_weight *= self.recency_factor
            self.pred_diff_dist[last_diff] += self.counting_weight
            
            # update wrapped bot
            self.bot.update(op_hist, my_hist)
            
            # predict next distribution
            pred_dist_norm = [None] * 3
            for d in PAYOFFS:
                pred_dist_norm[add[d][self.bot.pred]] = self.pred_diff_dist[d]
            self.pred_dist_norm = normalized(pred_dist_norm)
    
    # wraps around a deterministic bot using 1-st order markov
    # uses last op and last payoff
    class Markov1Hypothesis:
        def __init__(self, bot, decay, reversed=False):
            self.reversed = reversed
            # uniform prior
            self.pred_diff_matrix = [[3.0] * 3 for _ in xrange(9)]
            self.pred_dist_norm = [1.0 / 3.0] * 3
            self.recency_factor = 1.0 / decay
            self.counting_weight = 1.0
            
            # wraps over a deterministic bot
            self.bot = bot
            self.diffs = []
        
        def update(self, op_hist, my_hist):
            if self.reversed:
                op_hist, my_hist = my_hist, op_hist
            
            # update diffs for last time
            self.diffs.append(sub[op_hist[-1]][self.bot.pred])
            
            # update matrix for last time
            self.counting_weight *= self.recency_factor
            if hands >= 2:
                last_last_diff, last_diff = self.diffs[-2], self.diffs[-1]
                last_last_payoff = sub[my_hist[-2]][op_hist[-2]]
                last_last_state = enc2[last_last_diff][last_last_payoff]-1
                self.pred_diff_matrix[last_last_state][last_diff] += self.counting_weight
            
            # update underlying bot
            self.bot.update(op_hist, my_hist)
            
            # use first order markov to calculate next diff distribution
            pred_dist_norm = [None] * 3
            last_diff = self.diffs[-1]
            last_payoff = sub[my_hist[-1]][op_hist[-1]]
            last_state = enc2[last_diff][last_payoff]-1
            for d in PAYOFFS:
                pred_dist_norm[add[d][self.bot.pred]] = self.pred_diff_matrix[last_state][d]
            self.pred_dist_norm = normalized(pred_dist_norm)
    
    # mixes multiple hypotheses together
    class BayesMixer:
        def __init__(self, hypotheses, decay, reversed=False):
            self.reversed = reversed
            # uniform prior
            self.pred_dist_norm = [1.0 / 3.0] * 3
            self.decay_factor = decay
            self.pred = expected(self.pred_dist_norm)
            
            self.hypotheses = hypotheses
            self.range = range(len(hypotheses))
            
            # log (P ( history | bot))
            self.log_data_probs = [0.0 for _ in self.hypotheses]
        
        def update(self, op_hist, my_hist):
            if self.reversed:
                op_hist, my_hist = my_hist, op_hist
            # update probabilities for last time
            for i in self.range:
                data_prob = self.hypotheses[i].pred_dist_norm[op_hist[-1]]
                self.log_data_probs[i] *= self.decay_factor
                self.log_data_probs[i] += log(data_prob)
            
            # update hypotheses
            for h in self.hypotheses:
                h.update(op_hist, my_hist)
            
            # approximate likelihoods
            max_factor = max(self.log_data_probs)
    
            # P ( bot | history ) =  P ( history | bot) * (some normalizing factor)
            likelihoods = [exp(p - max_factor) for p in self.log_data_probs]
            
            # mix according to likelihood
            pred_dist_norm = [0.0, 0.0, 0.0]
            for i in self.range:
                for h in RPS:
                    pred_dist_norm[h] += self.hypotheses[i].pred_dist_norm[h] * likelihoods[i]
            
            self.pred_dist_norm = normalized(pred_dist_norm)
            self.pred = expected(self.pred_dist_norm)
            
    # 1st order transition matrix
    # use both last diff and last payoff for each bot
    # markov1 = [[[3.0, 3.0, 3.0] for _ in xrange(9)] for _ in BOTS]
    
    # initialize history matching strategies
    my_ghpm = GHPM(6, 3)
    op_ghpm = GHPM(6, 3)
    both_ghpm = GHPM(6, 9)
    
    # initialize deterministic bots
    constant_bot = ConstantBot(seed)
    last_op_bot = LastBot(seed)
    last_my_bot = LastBot(seed, reversed=True)
    
    op_ghpm_bot = GHPMBot(seed, op_ghpm)
    rev_op_ghpm_bot = GHPMBot(seed, op_ghpm, reversed=True)
    
    my_ghpm_bot = GHPMBot(seed, my_ghpm)
    rev_my_ghpm_bot = GHPMBot(seed, my_ghpm, reversed=True)
    
    both_ghpm_bot = GHPMBot(seed, both_ghpm)
    rev_both_ghpm_bot = GHPMBot(seed, both_ghpm, reversed=True)
    
    bots = [
        constant_bot, last_op_bot, last_my_bot,
        op_ghpm_bot, rev_op_ghpm_bot, my_ghpm_bot, rev_my_ghpm_bot,
        both_ghpm_bot, rev_both_ghpm_bot
    ]
    
    # initialize the hypotheses
    hypotheses = [Markov1Hypothesis(b, 0.98) for b in bots]
    
    # initialize bayes mixer
    bayes_mixer = BayesMixer(hypotheses, 0.98)
    # orig_modified = Markov1Hypothesis(bayes_mixer, 0.98)
    
    mixer = bayes_mixer
    
    # first hand is completely random - no reason to do otherwise
    next_hand = expected(mixer.pred_dist_norm)
    output = tr[next_hand]
    
    # bot_diffs = [[] for _ in BOTS]
    # bot_payoffs = [[] for _ in BOTS]
    
    # bookkeeping
    op_hist = []
    my_hist = []
    hands = 1
    last_ix = 0
    score = 0
# --------------------- turn -----------------------------
else:
    last_op = tr[input]
    last_my = tr[output]
    last_payoff = sub[last_my][last_op]
    op_hist.append(last_op)
    my_hist.append(last_my)
    
    # update the history matchers
    my_ghpm.update(enc1[last_my], lambda _:last_ix)
    op_ghpm.update(enc1[last_op], lambda _:last_ix)
    both_ghpm.update(enc2[last_op][last_my], lambda _:last_ix)
    
    mixer.update(op_hist, my_hist)
    
    COREYSPROBS = {'R':mixer.pred_dist_norm[2],'P':mixer.pred_dist_norm[0],'S':mixer.pred_dist_norm[1]}
    next_hand = expected(mixer.pred_dist_norm)
    
    output = tr[next_hand]
    # bookkeeping
    hands += 1
    last_ix += 1
    score += pts[last_payoff]
    
    # if hands % 100 == 0:
    # if hands < 10:
        # print

    
''', '<string>', 'exec')

code_zaimarkov = compile(
    '''
xrange = range
# Name: zai_switch_markov1_ml
# Author: zdg
# Email: rpscontest.b73@gishpuppy.com
# the email is disposable in case it gets spammed
# 
# let's try out some bayes inference with 1-st order markov model wrappers on
#    various strategies as the hypotheses
# uses last diff as well as last payoff to transition for the 1-st order
# uses ML i.e. maximum likelihood learning by minimizing the entropy
# also have some ad-hoc decay which helps a lot

# --------------------- initialization -----------------------------
if not input:
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
    import random, collections, math
    
    # micro-optimizations
    rchoice = random.choice
    log = math.log
    
    # global constants and maps
    # using lists and dictionaries because function call and arithmetic is slow
    R, P, S = 0, 1, 2
    RPS = [R, P, S]
    T, W, L = R, P, S
    PAYOFFS = RPS
    tr = {'R':R, 'P':P, 'S':S, R:'R', P:'P', S:'S'}
    sub = [[T, L, W], [W, T, L], [L, W, T]]
    add = [[R, P, S], [P, S, R], [S, R, P]]
    ties, beats, loses = add[T], add[W], add[L]
    
    pts = [0, 1, -1]
    near = [1, 0, 0]
    
    enc1 = [1,2,3]
    dec1 = [None, R, P, S]
    
    enc2 = [[1,2,3], [4,5,6], [7,8,9]]
    dec2 = [None,(R,R),(R,P),(R,S),(P,R),(P,P),(P,S),(S,R),(S,P),(S,S)]
    
    seed = rchoice(RPS)
    
    # entropy of a 3 vector - doesn't need to be normalized
    def entropy(vec):
        v1, v2, v3 = vec
        # force coerce to float
        total = v1 + v2 + v3 + 0.0
        # normalize
        p1, p2, p3 = v1 / total, v2 / total, v3 / total
        if total > 0.0:
            return (-((p1 * log(p1)) if v1 > 0.0 else 0.0)
                - ((p2 * log(p2)) if v2 > 0.0 else 0.0)
                - ((p3 * log(p3)) if v3 > 0.0 else 0.0))
        else:
            # entropy of nothing is undefined, but here it's useful to define it
            #    as the same as completely random
            return 1.0
    
    def pick_max(vec):
        max_val = max(vec)
        max_list = [i for i in xrange(len(vec)) if vec[i] == max_val]
        return rchoice(max_list)
    
    # calculate the hand with the best expected value against the given op hand
    # random only in case of ties
    def expected(vec):
        expected_payoffs = [vec[S] - vec[P], vec[R] - vec[S], vec[P] - vec[R]]
        max_expected = max(expected_payoffs)
        max_list = [i for i in RPS if expected_payoffs[i] == max_expected]
        return rchoice(max_list)
    
    def normalize(vec):
        factor = 1.0 / sum(vec)
        for i in xrange(len(vec)):
            vec[i] *= factor
    
    # greedy history pattern matcher
    # ORDER is the largest context size
    # BASE is the base of the numerical encoding
    # encodes sequences of numbers from 1...BASE as a BASE-adic number
    # encodes the empty sequence as 0
    # apparently this encoding is called a bijective base-BASE system on wikipedia
    class GHPM:
        def __init__(self, ORDER, BASE):
            self.ORDER = ORDER
            self.BASE = BASE
            self.powers = [0] + [BASE ** i for i in xrange(ORDER)]
            self.hist = []
            self.contexts = collections.defaultdict(lambda: None)
            self.pred = None
            self.preds = [None] * (ORDER+1)
        
        def update(self, next_val, up_fun):
            self.hist.append(next_val)
            # update the history, order 0 as a special case
            up_ix = 0
            self.contexts[0] = up_fun(self.contexts[0])
            
            # start the prediction with the zeroth order
            self.pred = self.contexts[0]
            
            # update the higher orders and prediction
            elems = len(self.hist)
            for order in xrange(1, self.ORDER+1 if elems > self.ORDER else elems):
                pred_ix = up_ix * self.BASE + next_val
                
                up_ix += self.hist[-order-1] * self.powers[order]
                self.contexts[up_ix] = up_fun(self.contexts[up_ix])
                
                try_get = self.contexts[pred_ix]
                self.preds[order] = try_get
                if try_get is not None:
                    self.pred = try_get
    
    NUM_BOTS = 9
    BOTS = range(NUM_BOTS)
    DECAY = 0.98
    next_hands = [seed for _ in BOTS]
    # contexts = [[0.0, 0.0, 0.0] for _ in BOTS]
    # contexts = [[1.0, 1.0, 1.0] for _ in BOTS]
    
    # 1st order transition matrix
    # use both last diff and last payoff for each bot
    markov = [[[1.0 for _ in RPS] for _ in xrange(9)] for _ in BOTS]
    
    # initialize history matching strategies
    my_ghpm = GHPM(6, 3)
    op_ghpm = GHPM(6, 3)
    both_ghpm = GHPM(6, 9)
    
    # first hand is completely random - no reason to do otherwise
    next_hand = seed
    output = tr[next_hand]
    
    bot_diffs = [[] for _ in BOTS]
    bot_payoffs = [[] for _ in BOTS]
    
    # bookkeeping
    hands = 1
    last_ix = 0
    score = 0
# --------------------- turn -----------------------------
else:
    last_my = tr[output]
    last_op = tr[input]
    last_payoff = sub[last_my][last_op]
    
    # keep track of how bots have been playing
    for b in BOTS:
        bot_diffs[b].append(sub[last_op][next_hands[b]])
        bot_payoffs[b].append(sub[last_my][next_hands[b]])
    
    # update the zeroth order markov models
    # for b in BOTS:
        # contexts[b][T] *= DECAY
        # contexts[b][W] *= DECAY
        # contexts[b][L] *= DECAY
        # contexts[b][bot_diffs[b][-1]] += 1.0
    
    # update the first order markov models
    if hands >= 2:
        for b in BOTS:
            last_last_diff, last_diff = bot_diffs[b][-2], bot_diffs[b][-1]
            last_last_payoff = bot_payoffs[b][-2]
            last_last_state = enc2[last_last_diff][last_last_payoff]-1
            
            markov[b][last_last_state][T] *= DECAY
            markov[b][last_last_state][W] *= DECAY
            markov[b][last_last_state][L] *= DECAY
            markov[b][last_last_state][last_diff] += 1.0
    
    # use first order markov to calculate each bot's next state distribution
    next_dist = [None] * NUM_BOTS
    for b in BOTS:
        last_state = enc2[bot_diffs[b][-1]][bot_payoffs[b][-1]]-1
        next_dist[b] = markov[b][last_state]
    
    # update the history matchers
    my_ghpm.update(enc1[last_my], lambda _:last_ix)
    op_ghpm.update(enc1[last_op], lambda _:last_ix)
    both_ghpm.update(enc2[last_op][last_my], lambda _:last_ix)
    
    # update predictions
    next_hands[1] = last_op
    next_hands[2] = last_my
    
    # hpm
    both_hist = both_ghpm.hist
    # my hist
    pred_op, pred_my = dec2[both_hist[my_ghpm.pred]]
    next_hands[3] = pred_op
    next_hands[4] = pred_my
    # op hist
    pred_op, pred_my = dec2[both_hist[op_ghpm.pred]]
    next_hands[5] = pred_op
    next_hands[6] = pred_my
    # both hist
    pred_op, pred_my = dec2[both_hist[both_ghpm.pred]]
    next_hands[7] = pred_op
    next_hands[8] = pred_my
    
    # find the most likely bot i.e. one with least entropy
    entropies = [-entropy(next_dist[b]) for b in BOTS]
    most_likely = pick_max(entropies)
    
    pred_next = [None] * 3
    for h in RPS:
        pred_next[add[h][next_hands[most_likely]]] = next_dist[most_likely][h]

    next_hand = expected(pred_next)
    
    output = tr[next_hand]
    COREYSPROBS = {'R':pred_next[2],'P':pred_next[0],'S':pred_next[1]}
    # bookkeeping
    hands += 1
    last_ix += 1
    score += pts[last_payoff]
    
    # if hands % 100 == 0:
        # print markov
        # print next_dist
        # print

    
''', '<string>', 'exec')

code_zaidfa3 = compile(
    '''
xrange = range
# uses a higher order markov chain to transition between the 6 ways of
# using the beat last strategy

# --------------------- initialization -----------------------------
if not input:
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
    import random, math
    import itertools, operator, collections
    
    R, P, S = 0, 1, 2
    RPS = [R, P, S]
    T, W, L = R, P, S
    PAYOFFS = [T, W, L]
    scorep = [0, 1, -1]
    to_win = [W, T, L]
    convert = {'R':R, 'P':P, 'S':S, R:'R', P:'P', S:'S'}
    subh = [[T, L, W], [W, T, L], [L, W, T]]
    addh = [[R, P, S], [P, S, R], [S, R, P]]
    ties, beats, loses = addh[T], addh[W], addh[L]
    
    def pick_max(vec):
        maxix = 0
        accmax = vec[0]
        for i in xrange(1, len(vec)):
            if vec[i] >= accmax:
                accmax = vec[i]
                maxix = i
        return maxix
    
    def normalize(u):
        factor = 1.0 / sum(u)
        for i in xrange(len(u)):
            u[i] *= factor

    def pick_weighted(v):
        u = random.random()
        acc = 0.0
        for i, p in enumerate(v):
            acc += p
            if u < acc:
                return (i, p)
    
    class MarkovN(object):
        def __init__(self, order, op_moves, state_history, bot_list):
            self.order = order
            self.state_history = state_history
            self.op_moves = op_moves
            self.bot_list = bot_list
            self.dict = collections.defaultdict(lambda:[0,0,0,0,0,0])
        
        def update(self):
            # last turn's state
            last_state = self.state_history[-1]
            
            # op's last move
            last_op_move = self.op_moves[-1]
            
            # the highest order that can be updated
            highest_prev_order = min(len(self.state_history) - 1, self.order)
            
            # update the transitions for the bots that won last turn
            for bot_index, bot in enumerate(self.bot_list):
                if bot.next_hand == beats[last_op_move]:
                    # up to and including highest_prev_order
                    for order in xrange(highest_prev_order + 1):
                        # up to second last one
                        upper_bound = len(self.state_history) - 1
                        lower_bound = upper_bound - order
                        index = tuple(self.state_history[lower_bound:upper_bound]);
                        
                        # update last time's info
                        self.dict[index][bot_index] += 1
            
            # now find highest order that's not all 0
            highest_curr_order = min(len(self.state_history), self.order)
            for order in xrange(highest_curr_order, -1, -1):
                lower_bound = len(self.state_history) - order
                index = tuple(self.state_history[lower_bound:])
                counts = self.dict[index]
                if sum(counts) > 0:
                    self.counts = counts
                    break
                
    
    class LastBot(object):
        def __init__(self, op_moves, offset):
            self.op_moves = op_moves
            self.offset = offset
        
        def update(self):
            self.next_hand = addh[self.offset][self.op_moves[-1]]
    
    op_hands = []
    my_hands = []
    payoffs = []
    op_payoffs = []
    
    output = convert[random.choice(RPS)]
    
    
    bots = [
        LastBot(op_hands, W),
        LastBot(my_hands, L),
        LastBot(op_hands, T),
        LastBot(my_hands, W),
        LastBot(op_hands, L),
        LastBot(my_hands, T)]
    mode = 0
    
    states = []
    
    markov = MarkovN(6, op_hands, states, bots)
# --------------------- turn -----------------------------
else:
    last_input, last_output = convert[input], convert[output]
    my_hands.append(last_output)
    op_hands.append(last_input)
    payoffs.append(subh[last_output][last_input])
    op_payoffs.append(subh[last_input][last_output])
    hands_played = len(op_hands)
    
    states.append(mode)
    
    if hands_played > 1:
        markov.update()
        
        for b in bots:
            b.update()
        
        nextstatep = markov.counts[:]
        normalize(nextstatep)
        mode = pick_weighted(nextstatep)[0]
        next_hand = bots[mode].next_hand

        COREYSPROBS = {}
        for coreyindex in range(len(bots)):
            key = convert[bots[coreyindex].next_hand]
            if key not in COREYSPROBS.keys():
                COREYSPROBS[key] = nextstatep[coreyindex]
            elif nextstatep[coreyindex] > COREYSPROBS[key]:
                COREYSPROBS[key] = nextstatep[coreyindex]
    else:
        for b in bots:
            b.update()
        next_hand = bots[mode].next_hand
        COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
    
    output = convert[next_hand]
    
    # output = convert[random.choice(RPS)]

    
''', '<string>', 'exec')

code_lucker = compile(
    """
import random

num_predictors =27
num_meta= 18

if input =="":
    len_rfind = [20]
    limit = [10,20,60]
    beat = { "P":"S" , "R":"P" , "S":"R" }
    not_lose = { "R":"PR", "P":"SP", "S":"RS" } 
    your_his =""
    my_his = ""
    both_his=""
    both_his2=""
    length =0
    score1=[3]*num_predictors
    score2=[3]*num_predictors
    score3=[3]*num_predictors
    score4=[3]*num_predictors
    score5=[3]*num_predictors
    score6=[3]*num_predictors
    metascore=[3]*num_meta
    temp1 = { "PP":"1","PR":"2","PS":"3",
              "RP":"4","RR":"5","RS":"6",
              "SP":"7","SR":"8","SS":"9"}
    temp2 = { "1":"PP","2":"PR","3":"PS",
                "4":"RP","5":"RR","6":"RS",
                "7":"SP","8":"SR","9":"SS"} 
    who_win = { "PP": 0, "PR":1 , "PS":-1,
                "RP": -1,"RR":0, "RS":1,
                "SP": 1, "SR":-1, "SS":0}
    index = { "P":0, "R":1, "S":2 }
    chance =[0]*num_predictors
    chance2 =[0]*num_predictors
    output = random.choice("RPS")
    predictors = [output]*num_predictors
    metapredictors = [output]*num_meta
    COREYSPROBS = {'R':0.33, 'P':0.33, 'S':0.33}
else:
    #calculate score
    for i in range(num_predictors):
        #meta 1
        score1[i]*=0.8
        if input==predictors[i]:
            score1[i]+=3
        else:
            score1[i]-=3
        #meta 2
        if input==predictors[i]:
            score2[i]+=3
        else:
            score2[i]=0
        #meta 3
        score3[i]*=0.8
        if output==predictors[i]:
            score3[i]+=3
        else:
           score3[i]-=3
        #meta 4
        if output==predictors[i]:
            score4[i]+=3
        else:
            score4[i]=0
        #meta 5
        score5[i]*=0.8
        if input==predictors[i]:
            score5[i]+=3
        else:
            if chance[i]==1:
                chance[i]=0
                score5[i]-=3
            else:
                chance[i]=1
                score5[i]=0
        #meta 6
        score6[i]*=0.8
        if output==predictors[i]:
            score6[i]+=3
        else:
            if chance2[i]==1:
                chance2[i]=0
                score6[i]-=3
            else:
                chance2[i]=1
                score6[i]=0
    #calculate metascore
    for i in range(num_meta):
        metascore[i]*=0.9
        if input==metapredictors[i]:
            metascore[i]+=3
        else:
            metascore[i]=0
    #Predictors
    #if length>1:
    #    output=beat[predict]
    your_his+=input
    my_his+=output
    both_his+=temp1[(input+output)]
    both_his2+=temp1[(output+input)]
    length+=1

    #history matching
    for i in range(1):
        len_size = min(length,len_rfind[i])
        j=len_size
        #both_his
        while j>=1 and not both_his[length-j:length] in both_his[0:length-1]:
            j-=1
        if j>=1:
            k = both_his.rfind(both_his[length-j:length],0,length-1)
            predictors[0+6*i] = your_his[j+k]
            predictors[1+6*i] = beat[my_his[j+k]]
        else:
            predictors[0+6*i] = random.choice("RPS")
            predictors[1+6*i] = random.choice("RPS")
        j=len_size
        #your_his
        while j>=1 and not your_his[length-j:length] in your_his[0:length-1]:
            j-=1
        if j>=1:
            k = your_his.rfind(your_his[length-j:length],0,length-1)
            predictors[2+6*i] = your_his[j+k]
            predictors[3+6*i] = beat[my_his[j+k]]
        else:
            predictors[2+6*i] = random.choice("RPS")
            predictors[3+6*i] = random.choice("RPS")
        j=len_size
        #my_his
        while j>=1 and not my_his[length-j:length] in my_his[0:length-1]:
            j-=1
        if j>=1:
            k = my_his.rfind(my_his[length-j:length],0,length-1)
            predictors[4+6*i] = your_his[j+k]
            predictors[5+6*i] = beat[my_his[j+k]]
        else:
            predictors[4+6*i] = random.choice("RPS")
            predictors[5+6*i] = random.choice("RPS")
    
    #Reverse
    for i in range(3):
        temp =""
        search = temp1[(output+input)] #last round
        for start in range(2, min(limit[i],length) ):
            if search == both_his2[length-start]:
                temp+=both_his2[length-start+1]
        if(temp==""):
            predictors[6+i] = random.choice("RPS")
        else:
            collectR = {"P":0,"R":0,"S":0} #take win/lose from opponent into account
            for sdf in temp:
                next_move = temp2[sdf]
                if(who_win[next_move]==-1):
                    collectR[temp2[sdf][1]]+=3
                elif(who_win[next_move]==0):
                    collectR[temp2[sdf][1]]+=1
                elif(who_win[next_move]==1):
                    collectR[beat[temp2[sdf][0]]]+=1
            max1 = -1
            p1 =""
            for key in collectR:
                if(collectR[key]>max1):
                    max1 = collectR[key]
                    p1 += key
            predictors[6+i] = random.choice(p1)
    
    for i in range(9,27):
        predictors[i]=beat[beat[predictors[i-9]]]
    #find prediction for each meta
    metapredictors[0]=predictors[score1.index(max(score1))]
    metapredictors[1]=predictors[score2.index(max(score2))]
    metapredictors[2]=beat[predictors[score3.index(max(score3))]]
    metapredictors[3]=beat[predictors[score4.index(max(score4))]]
    metapredictors[4]=predictors[score5.index(max(score5))]
    metapredictors[5]=beat[predictors[score6.index(max(score6))]]
    for i in range(6,18):
        metapredictors[i] = beat[metapredictors[i-6]]
    
    predict = metapredictors[metascore.index(max(metascore))]
    output = beat[predict]

    COREYSPROBS = {}
    for coreysidx in range(len(metapredictors)):
        coreyskey = beat[metapredictors[coreysidx]]
        if coreyskey not in COREYSPROBS.keys():
            COREYSPROBS[coreyskey] = metascore[coreysidx]
        elif metascore[coreysidx] > COREYSPROBS[coreyskey]:
            COREYSPROBS[coreyskey] = metascore[coreysidx]

    #output = random.choice(not_lose[predict])
""", '<string>', 'exec')

code_dilu1 = compile(
    """
import random
numPre = 30
numMeta = 6
if not input:
    limit = 8
    beat={'R':'P','P':'S','S':'R'}
    moves=['','','','']
    pScore=[[5]*numPre,[5]*numPre,[5]*numPre,[5]*numPre,[5]*numPre,[5]*numPre]
    centrifuge={'RP':0,'PS':1,'SR':2,'PR':3,'SP':4,'RS':5,'RR':6,'PP':7,'SS':8}
    centripete={'R':0,'P':1,'S':2}
    soma = [0,0,0,0,0,0,0,0,0];
    rps = [1,1,1];
    a="RPS"
    best = [0,0,0];
    length=0
    p=[random.choice("RPS")]*numPre
    m=[random.choice("RPS")]*numMeta
    mScore=[5,2,5,2,4,2]
else:
    for i in range(numPre):
        pp = p[i]
        bpp = beat[pp]
        bbpp = beat[bpp]
        pScore[0][i]=0.9*pScore[0][i]+((input==pp)-(input==bbpp))*3
        pScore[1][i]=0.9*pScore[1][i]+((output==pp)-(output==bbpp))*3
        pScore[2][i]=0.87*pScore[2][i]+(input==pp)*3.3-(input==bpp)*1.2-(input==bbpp)*2.3
        pScore[3][i]=0.87*pScore[3][i]+(output==pp)*3.3-(output==bpp)*1.2-(output==bbpp)*2.3
        pScore[4][i]=(pScore[4][i]+(input==pp)*3)*(1-(input==bbpp))
        pScore[5][i]=(pScore[5][i]+(output==pp)*3)*(1-(output==bbpp))
    for i in range(numMeta):
        mScore[i]=0.96*(mScore[i]+(input==m[i])-(input==beat[beat[m[i]]]))
    soma[centrifuge[input+output]] +=1;
    rps[centripete[input]] +=1;
    moves[0]+=str(centrifuge[input+output])
    moves[1]+=input
    moves[2]+=output
    length+=1
    for y in range(3):
        j=min([length,limit])
        while j>=1 and not moves[y][length-j:length] in moves[y][0:length-1]:
            j-=1
        i = moves[y].rfind(moves[y][length-j:length],0,length-1)
        p[0+2*y] = moves[1][j+i] 
        p[1+2*y] = beat[moves[2][j+i]]
    j=min([length,limit])
    while j>=2 and not moves[0][length-j:length-1] in moves[0][0:length-2]:
        j-=1
    i = moves[0].rfind(moves[0][length-j:length-1],0,length-2)
    if j+i>=length:
        p[6] = p[7] = random.choice("RPS")
    else:
        p[6] = moves[1][j+i] 
        p[7] = beat[moves[2][j+i]]
        
    best[0] = soma[centrifuge[output+'R']]*rps[0]/rps[centripete[output]]
    best[1] = soma[centrifuge[output+'P']]*rps[1]/rps[centripete[output]]
    best[2] = soma[centrifuge[output+'S']]*rps[2]/rps[centripete[output]]
    p[8] = p[9] = a[best.index(max(best))]
    
    for i in range(10,numPre):
        p[i]=beat[beat[p[i-10]]]
        
    for i in range(0,numMeta,2):
        m[i]=       p[pScore[i  ].index(max(pScore[i  ]))]
        m[i+1]=beat[p[pScore[i+1].index(max(pScore[i+1]))]]
output = beat[m[mScore.index(max(mScore))]]
if max(mScore)<0.07 or random.randint(3,40)>length:
    output=beat[random.choice("RPS")]
COREYSPROBS = {}
for coreysidx in range(len(m)):
    coreyskey = beat[m[coreysidx]]
    if coreyskey not in COREYSPROBS.keys():
        COREYSPROBS[coreyskey] = mScore[coreysidx]
    elif mScore[coreysidx] > COREYSPROBS[coreyskey]:
        COREYSPROBS[coreyskey] = mScore[coreysidx]
""", '<string>', 'exec')

code_iocaine = compile(
    '''

import random

if input == '':
    ages = [1000, 100, 10, 5, 2, 1]
    
    def recall(age, hist):
        """Looking at the last 'age' points in 'hist', finds the
        last point with the longest similarity to the current point,
        returning 0 if none found."""
        end, length = 0, 0
        for past in range(1, min(age + 1, len(hist) - 1)):
            if length >= len(hist) - past: break
            for i in range(-1 - length, 0):
                if hist[i - past] != hist[i]: break
            else:
                for length in range(length + 1, len(hist) - past):
                    if hist[-past - length - 1] != hist[-length - 1]: break
                else: length += 1
                end = len(hist) - past
        return end

    def beat(i):
        return (i + 1) % 3
    def loseto(i):
        return (i - 1) % 3

    class Stats:
        """Maintains three running counts and returns the highest count based
             on any given time horizon and threshold."""
        def __init__(self):
            self.sum = [[0, 0, 0]]
        def add(self, move, score):
            self.sum[-1][move] += score
        def advance(self):
            self.sum.append(self.sum[-1])
        def max(self, age, default, score):
            if age >= len(self.sum): diff = self.sum[-1]
            else: diff = [self.sum[-1][i] - self.sum[-1 - age][i] for i in range(3)]
            m = max(diff)
            if m > score: return diff.index(m), m
            return default, score

    class Predictor:
        """The basic iocaine second- and triple-guesser.    Maintains stats on the
             past benefits of trusting or second- or triple-guessing a given strategy,
             and returns the prediction of that strategy (or the second- or triple-
             guess) if past stats are deviating from zero farther than the supplied
             "best" guess so far."""
        def __init__(self):
            self.stats = Stats()
            self.lastguess = -1
        def addguess(self, lastmove, guess):
            if lastmove != -1:
                diff = (lastmove - self.prediction) % 3
                self.stats.add(beat(diff), 1)
                self.stats.add(loseto(diff), -1)
                self.stats.advance()
            self.prediction = guess
        def bestguess(self, age, best):
            bestdiff = self.stats.max(age, (best[0] - self.prediction) % 3, best[1])
            return (bestdiff[0] + self.prediction) % 3, bestdiff[1]

    class Iocaine:

        def __init__(self):
            """Build second-guessers for 50 strategies: 36 history-based strategies,
                 12 simple frequency-based strategies, the constant-move strategy, and
                 the basic random-number-generator strategy.    Also build 6 meta second
                 guessers to evaluate 6 different time horizons on which to score
                 the 50 strategies' second-guesses."""
            self.predictors = []
            self.predict_history = self.predictor((len(ages), 2, 3))
            self.predict_frequency = self.predictor((len(ages), 2))
            self.predict_fixed = self.predictor()
            self.predict_random = self.predictor()
            self.predict_meta = [Predictor() for a in range(len(ages))]
            self.stats = [Stats() for i in range(2)]
            self.histories = [[], [], []]

        def predictor(self, dims=None):
            """Returns a nested array of predictor objects, of the given dimensions."""
            if dims: return [self.predictor(dims[1:]) for i in range(dims[0])]
            self.predictors.append(Predictor())
            return self.predictors[-1]

        def move(self, them):
            """The main iocaine "move" function."""

            # histories[0] stores our moves (last one already previously decided);
            # histories[1] stores their moves (last one just now being supplied to us);
            # histories[2] stores pairs of our and their last moves.
            # stats[0] and stats[1] are running counters our recent moves and theirs.
            if them != -1:
                self.histories[1].append(them)
                self.histories[2].append((self.histories[0][-1], them))
                for watch in range(2):
                    self.stats[watch].add(self.histories[watch][-1], 1)

            # Execute the basic RNG strategy and the fixed-move strategy.
            rand = random.randrange(3)
            self.predict_random.addguess(them, rand)
            self.predict_fixed.addguess(them, 0)

            # Execute the history and frequency stratgies.
            for a, age in enumerate(ages):
                # For each time window, there are three ways to recall a similar time:
                # (0) by history of my moves; (1) their moves; or (2) pairs of moves.
                # Set "best" to these three timeframes (zero if no matching time).
                best = [recall(age, hist) for hist in self.histories]
                for mimic in range(2):
                    # For each similar historical moment, there are two ways to anticipate
                    # the future: by mimicing what their move was; or mimicing what my
                    # move was.    If there were no similar moments, just move randomly.
                    for watch, when in enumerate(best):
                        if not when: move = rand
                        else: move = self.histories[mimic][when]
                        self.predict_history[a][mimic][watch].addguess(them, move)
                    # Also we can anticipate the future by expecting it to be the same
                    # as the most frequent past (either counting their moves or my moves).
                    mostfreq, score = self.stats[mimic].max(age, rand, -1)
                    self.predict_frequency[a][mimic].addguess(them, mostfreq)

            # All the predictors have been updated, but we have not yet scored them
            # and chosen a winner for this round.    There are several timeframes
            # on which we can score second-guessing, and we don't know timeframe will
            # do best.    So score all 50 predictors on all 6 timeframes, and record
            # the best 6 predictions in meta predictors, one for each timeframe.
            for meta, age in enumerate(ages):
                best = (-1, -1)
                for predictor in self.predictors:
                    best = predictor.bestguess(age, best)
                self.predict_meta[meta].addguess(them, best[0])

            # Finally choose the best meta prediction from the final six, scoring
            # these against each other on the whole-game timeframe. 
            best = (-1, -1)
            for meta in range(len(ages)):
                best = self.predict_meta[meta].bestguess(len(self.histories[0]) , best) 

            # We've picked a next move.    Record our move in histories[0] for next time.
            self.histories[0].append(best[0])

            # And return it.
            return best[0]
            
    iocaine = Iocaine()
    act = iocaine.move(-1)
  

else:
    act = iocaine.move({'R':0,'P':1,'S':2}[input])
   
        
    
output = 'RPS'[act]
''', '<string>', 'exec')

code_greenberg = compile(
    '''

import random
rps_to_text = ('rock','paper','scissors')

if input == '':
    def player(my_moves, opp_moves):
        import random
        from operator import itemgetter
        rps_to_text = ('rock','paper','scissors')
        rps_to_num  = {'rock':0, 'paper':1, 'scissors':2}
        wins_with = (1,2,0)      #superior
        best_without = (2,0,1)   #inferior

        lengths = (10, 20, 30, 40, 49, 0)
        p_random = random.choice([0,1,2])  #called 'guess' in iocaine

        TRIALS = 1000
        score_table =((0,-1,1),(1,0,-1),(-1,1,0))
        T = len(opp_moves)  #so T is number of trials completed

        def min_index(values):
            return min(enumerate(values), key=itemgetter(1))[0]

        def max_index(values):
            return max(enumerate(values), key=itemgetter(1))[0]

        def find_best_prediction(l):  # l = len
            bs = -TRIALS
            bp = 0
            if player.p_random_score > bs:
                bs = player.p_random_score
                bp = p_random
            for i in range(3):
                for j in range(24):
                    for k in range(4):
                        new_bs = player.p_full_score[T%50][j][k][i] - (player.p_full_score[(50+T-l)%50][j][k][i] if l else 0)
                        if new_bs > bs:
                            bs = new_bs
                            bp = (player.p_full[j][k] + i) % 3
                    for k in range(2):
                        new_bs = player.r_full_score[T%50][j][k][i] - (player.r_full_score[(50+T-l)%50][j][k][i] if l else 0)
                        if new_bs > bs:
                            bs = new_bs
                            bp = (player.r_full[j][k] + i) % 3
                for j in range(2):
                    for k in range(2):
                        new_bs = player.p_freq_score[T%50][j][k][i] - (player.p_freq_score[(50+T-l)%50][j][k][i] if l else 0)
                        if new_bs > bs:
                            bs = new_bs
                            bp = (player.p_freq[j][k] + i) % 3
                        new_bs = player.r_freq_score[T%50][j][k][i] - (player.r_freq_score[(50+T-l)%50][j][k][i] if l else 0)
                        if new_bs > bs:
                            bs = new_bs
                            bp = (player.r_freq[j][k] + i) % 3
            return bp


        if not my_moves:
            player.opp_history = [0]  #pad to match up with 1-based move indexing in original
            player.my_history = [0]
            player.gear = [[0] for _ in range(24)]
            # init()
            player.p_random_score = 0
            player.p_full_score = [[[[0 for i in range(3)] for k in range(4)] for j in range(24)] for l in range(50)]
            player.r_full_score = [[[[0 for i in range(3)] for k in range(2)] for j in range(24)] for l in range(50)]
            player.p_freq_score = [[[[0 for i in range(3)] for k in range(2)] for j in range(2)] for l in range(50)]
            player.r_freq_score = [[[[0 for i in range(3)] for k in range(2)] for j in range(2)] for l in range(50)]
            player.s_len = [0] * 6

            player.p_full = [[0,0,0,0] for _ in range(24)]
            player.r_full = [[0,0] for _ in range(24)]
        else:
            player.my_history.append(rps_to_num[my_moves[-1]])
            player.opp_history.append(rps_to_num[opp_moves[-1]])
            # update_scores()
            player.p_random_score += score_table[p_random][player.opp_history[-1]]
            player.p_full_score[T%50] = [[[player.p_full_score[(T+49)%50][j][k][i] + score_table[(player.p_full[j][k] + i) % 3][player.opp_history[-1]] for i in range(3)] for k in range(4)] for j in range(24)]
            player.r_full_score[T%50] = [[[player.r_full_score[(T+49)%50][j][k][i] + score_table[(player.r_full[j][k] + i) % 3][player.opp_history[-1]] for i in range(3)] for k in range(2)] for j in range(24)]
            player.p_freq_score[T%50] = [[[player.p_freq_score[(T+49)%50][j][k][i] + score_table[(player.p_freq[j][k] + i) % 3][player.opp_history[-1]] for i in range(3)] for k in range(2)] for j in range(2)]
            player.r_freq_score[T%50] = [[[player.r_freq_score[(T+49)%50][j][k][i] + score_table[(player.r_freq[j][k] + i) % 3][player.opp_history[-1]] for i in range(3)] for k in range(2)] for j in range(2)]
            player.s_len = [s + score_table[p][player.opp_history[-1]] for s,p in zip(player.s_len,player.p_len)]


        # update_history_hash()
        if not my_moves:
            player.my_history_hash = [[0],[0],[0],[0]]
            player.opp_history_hash = [[0],[0],[0],[0]]
        else:
            player.my_history_hash[0].append(player.my_history[-1])
            player.opp_history_hash[0].append(player.opp_history[-1])
            for i in range(1,4):
                player.my_history_hash[i].append(player.my_history_hash[i-1][-1] * 3 + player.my_history[-1])
                player.opp_history_hash[i].append(player.opp_history_hash[i-1][-1] * 3 + player.opp_history[-1])


        #make_predictions()

        for i in range(24):
            player.gear[i].append((3 + player.opp_history[-1] - player.p_full[i][2]) % 3)
            if T > 1:
                player.gear[i][T] += 3 * player.gear[i][T-1]
            player.gear[i][T] %= 9 # clearly there are 9 different gears, but original code only allocated 3 gear_freq's
                                   # code apparently worked, but got lucky with undefined behavior
                                   # I fixed by allocating gear_freq with length = 9
        if not my_moves:
            player.freq = [[0,0,0],[0,0,0]]
            value = [[0,0,0],[0,0,0]]
        else:
            player.freq[0][player.my_history[-1]] += 1
            player.freq[1][player.opp_history[-1]] += 1
            value = [[(1000 * (player.freq[i][2] - player.freq[i][1])) / float(T),
                      (1000 * (player.freq[i][0] - player.freq[i][2])) / float(T),
                      (1000 * (player.freq[i][1] - player.freq[i][0])) / float(T)] for i in range(2)]
        player.p_freq = [[wins_with[max_index(player.freq[i])], wins_with[max_index(value[i])]] for i in range(2)]
        player.r_freq = [[best_without[min_index(player.freq[i])], best_without[min_index(value[i])]] for i in range(2)]

        f = [[[[0,0,0] for k in range(4)] for j in range(2)] for i in range(3)]
        t = [[[0,0,0,0] for j in range(2)] for i in range(3)]

        m_len = [[0 for _ in range(T)] for i in range(3)]

        for i in range(T-1,0,-1):
            m_len[0][i] = 4
            for j in range(4):
                if player.my_history_hash[j][i] != player.my_history_hash[j][T]:
                    m_len[0][i] = j
                    break
            for j in range(4):
                if player.opp_history_hash[j][i] != player.opp_history_hash[j][T]:
                    m_len[1][i] = j
                    break
            for j in range(4):
                if player.my_history_hash[j][i] != player.my_history_hash[j][T] or player.opp_history_hash[j][i] != player.opp_history_hash[j][T]:
                    m_len[2][i] = j
                    break

        for i in range(T-1,0,-1):
            for j in range(3):
                for k in range(m_len[j][i]):
                    f[j][0][k][player.my_history[i+1]] += 1
                    f[j][1][k][player.opp_history[i+1]] += 1
                    t[j][0][k] += 1
                    t[j][1][k] += 1

                    if t[j][0][k] == 1:
                        player.p_full[j*8 + 0*4 + k][0] = wins_with[player.my_history[i+1]]
                    if t[j][1][k] == 1:
                        player.p_full[j*8 + 1*4 + k][0] = wins_with[player.opp_history[i+1]]
                    if t[j][0][k] == 3:
                        player.p_full[j*8 + 0*4 + k][1] = wins_with[max_index(f[j][0][k])]
                        player.r_full[j*8 + 0*4 + k][0] = best_without[min_index(f[j][0][k])]
                    if t[j][1][k] == 3:
                        player.p_full[j*8 + 1*4 + k][1] = wins_with[max_index(f[j][1][k])]
                        player.r_full[j*8 + 1*4 + k][0] = best_without[min_index(f[j][1][k])]

        for j in range(3):
            for k in range(4):
                player.p_full[j*8 + 0*4 + k][2] = wins_with[max_index(f[j][0][k])]
                player.r_full[j*8 + 0*4 + k][1] = best_without[min_index(f[j][0][k])]

                player.p_full[j*8 + 1*4 + k][2] = wins_with[max_index(f[j][1][k])]
                player.r_full[j*8 + 1*4 + k][1] = best_without[min_index(f[j][1][k])]

        for j in range(24):
            gear_freq = [0] * 9 # was [0,0,0] because original code incorrectly only allocated array length 3

            for i in range(T-1,0,-1):
                if player.gear[j][i] == player.gear[j][T]:
                    gear_freq[player.gear[j][i+1]] += 1

            #original source allocated to 9 positions of gear_freq array, but only allocated first three
            #also, only looked at first 3 to find the max_index
            #unclear whether to seek max index over all 9 gear_freq's or just first 3 (as original code)
            player.p_full[j][3] = (player.p_full[j][1] + max_index(gear_freq)) % 3

        # end make_predictions()

        player.p_len = [find_best_prediction(l) for l in lengths]

        COREYSPROBS = {}
        for coreysidx in range(len(player.p_len)):
            coreyskey = 'RPS'[rps_to_num[rps_to_text[player.p_len[coreysidx]]]]
            if coreyskey not in COREYSPROBS.keys():
                COREYSPROBS[coreyskey] = player.s_len[coreysidx]
            elif player.s_len[coreysidx] > COREYSPROBS[coreyskey]:
                COREYSPROBS[coreyskey] = player.s_len[coreysidx]

        return rps_to_num[rps_to_text[player.p_len[max_index(player.s_len)]]], COREYSPROBS

    opponent_hist, my_hist = [], []

else:
    my_hist.append(rps_to_text[act])
    opponent_hist.append(rps_to_text[{'R':0,'P':1,'S':2}[input]])
    
act, COREYSPROBS = player(my_hist, opponent_hist)
        
    
output = 'RPS'[act]
''', '<string>', 'exec')

code_rps30 = compile(
    '''

import random

nwin = 0
ntie = 0
nloss = 0
iter = 0
epsilon = 0.2
if not input:
    score = {
        ('R', 'R'): 0, ('R', 'P'): -1, ('R', 'S'): 1,
        ('P', 'R'): 1, ('P', 'P'): 0, ('P', 'S'): -1,
        ('S', 'R'): -1, ('S', 'P'): 1, ('S', 'S'): 0
    }
    Q = dict()
    lr = 0.9
    limits = [5, 15, 30, 50, 69, 100, 200]
    beat = {'R': 'P', 'P': 'S', 'S': 'R'}
    urmoves = ""
    mymoves = ""
    DNAmoves = ""
    output = random.choice(['R', 'P', 'S'])
    nuclease = {'RP': 'a', 'PS': 'b', 'SR': 'c', 'PR': 'd', 'SP': 'e', 'RS': 'f', 'RR': 'g', 'PP': 'h', 'SS': 'i'}
    length = 0
    output = random.choice('RPS')
    newstate = tuple()
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
else:
    urmoves += input
    mymoves += output
    DNAmoves += nuclease[input + output]
    length += 1

    state = newstate
    newstate = []
    for z in range(1):
        limit = min([length, limits[z]])
        j = limit
        while j >= 1 and not DNAmoves[length - j:length] in DNAmoves[0:length - 1]:
            j -= 1
        if j >= 1:
            i = DNAmoves.rfind(DNAmoves[length - j:length], 0, length - 1)  # You seem to be playing based on our moves
            newstate.append(urmoves[j + i])
            newstate.append(mymoves[j + i])
        j = limit
        while j >= 1 and not urmoves[length - j:length] in urmoves[0:length - 1]:
            j -= 1
        if j >= 1:
            i = urmoves.rfind(urmoves[length - j:length], 0, length - 1)  # You seem to be playing based on your moves
            newstate.append(urmoves[j+i])
            newstate.append(mymoves[j+i])
        j = limit
        while j >= 1 and not mymoves[length - j:length] in mymoves[0:length - 1]:
            j -= 1
        if j >= 1:
            i = mymoves.rfind(mymoves[length - j:length], 0, length - 1)  # You seem to be playing based on my moves
            newstate.append(urmoves[j+i])
            newstate.append(mymoves[j+i])

    newstate = tuple(newstate)
    action = output
    if score[(output, input)] == 1: nloss += 1
    elif score[(output, input)] == 0: ntie += 1
    elif score[(output, input)] == -1: nwin += 1

    reward = score[(action, input)]
    maxvalue = max(Q.get((newstate, a), 0) for a in 'RPS')
    Q[(state, action)] = Q.get((state, action), 0) + lr * (reward + 0.5 * maxvalue - Q.get((state, action), 0))
    succ = [Q.get((newstate, a), 0) for a in 'RPS']
    optimal_actions = ['RPS'[x] for x in range(len(succ)) if succ[x] == max(succ)]
    output = random.choice(optimal_actions) #if random.random() > epsilon else random.choice('RPS')
    COREYSPROBS = {'R':succ[0],'P':succ[1],'S':succ[2]}
''', '<string>', 'exec')

code_dsad = compile(
    """
xrange = range
import random
SIZE = 15
WEIGHT_FACTOR = 7.

from functools import cmp_to_key
def cmp(a, b):
    if a < b:
        return -1
    elif a == b:
        return 0
    else:
        return 1

class HistoryNode(object):
    def __init__(self, parent=None):
        if parent is not None:
            self.depth = parent.depth + 1
        else:
            self.depth = 0

        self.children = {'RR': None, 'RS': None, 'RP': None, 'SR': None, 'SS': None,
                         'SP': None, 'PR': None, 'PS': None, 'PP': None}

        self.distribution = {'RR': 0, 'RS': 0, 'RP': 0, 'SR': 0, 'SS': 0, 'SP': 0,
                             'PR': 0, 'PS': 0, 'PP': 0}

    def new_move(self, input):
        last_move = input[0:2]

        if len(input) > 2:
            if self.children[last_move] is None:
                self.children[last_move] = HistoryNode(self)
            self.children[last_move].new_move(input[2:])
        else:
            self.distribution[last_move] = self.distribution[last_move] * 0.975 + 1

    def predict(self, input):
        if len(input) > 0:
            last_move = input[0:2]
            if self.children[last_move] is not None:
                return self.children[last_move].predict(input[2:])
            else:
                return None
        else:
            return self.distribution


class HistoryTree(object):
    def __init__(self):
        self.root = HistoryNode()

        self.input = ''

    def new_move(self, move):
        self.input += move
        if len(self.input) > SIZE * 2:
            self.input = self.input[-SIZE * 2:]

        for i in xrange(2, len(self.input) + 1, 2):
            self.root.new_move(self.input[-i:])

    def predict(self):
        results = {'R':0, 'S':0, 'P':0}
        for i in xrange(2, len(self.input) + 1, 2):
            res = self.root.predict(self.input[-i:])
            if res is not None:
                for key in res:
                    results[key[1]] += res[key] * (WEIGHT_FACTOR ** i)

        d = results
        e = list(d.keys())
        
        e.sort(key=cmp_to_key(lambda a, b: cmp(d[a], d[b])))
        return e[-1]
    
    def predict_i(self, i):
        results = {'R':0, 'S':0, 'P':0}
        for depth in xrange(i, -1, -1):
            res = self.root.predict(self.input[-depth * 2:])
            if res is not None:
                break
        
        if res is None:
            return random.choice(["R", "P", "S"])
        else:
            for key in res:
                results[key[1]] += res[key]
            
        d = results
        e = list(d.keys())
        e.sort(key=cmp_to_key(lambda a, b: cmp(d[a], d[b])))
        return e[-1]


if input == '':
    history_tree_me = HistoryTree()
    history_tree_him = HistoryTree()
    output = random.choice(["R", "P", "S"])
    meta_predictor = [output] * (6 * SIZE + 6)
    metascore = [0] * (6 * SIZE + 6)
    win = ['RS', 'SP', 'PR']
    lost = ['SR', 'PS', 'RP']
    prediction_me_i = [output] * SIZE
    prediction_him_i = [output] * SIZE
    possible_moves = ['R', 'P', 'S']
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
else:
    history_tree_me.new_move(output + input)
    history_tree_him.new_move(input + output)
        
    for idx in xrange(len(metascore)):
        if meta_predictor[idx] + input in win:
            metascore[idx] = metascore[idx] * 0.9 + 1
        elif meta_predictor[idx] + input in lost:
            metascore[idx] = metascore[idx] * 0.9 - 1
        else:
            metascore[idx] = metascore[idx] * 0.9 - 0.34
            
        
    prediction_me = history_tree_me.predict()
    prediction_him = history_tree_him.predict()
    for j in xrange(SIZE):
        prediction_me_i[j] = history_tree_me.predict_i(j)
        prediction_him_i[j] = history_tree_him.predict_i(j)
        
    for i in xrange(3):
        move_me = possible_moves[(possible_moves.index(prediction_me) + i) % 3]
        move_him = possible_moves[(possible_moves.index(prediction_him) + i) % 3]
        meta_predictor[i] = move_me
        meta_predictor[i + 3] = move_him
        for j in xrange(SIZE):
            move_me = possible_moves[(possible_moves.index(prediction_me_i[j]) + i) % 3]
            move_him = possible_moves[(possible_moves.index(prediction_him_i[j]) + i) % 3]
            meta_predictor[i + (j + 1) * 6] = move_me
            meta_predictor[i + 3 + (j + 1) * 6] = move_him
        
    
    best_predictor = metascore.index(max(metascore))

    play_random = True
    for score in metascore:
        if score > 2.7:
            play_random = False
        
    play_random = False
    if play_random:
        output = random.choice(["R", "P", "S"])
    else:
        output = meta_predictor[best_predictor]
""", '<string>', 'exec')

code_copymodel = compile(
    """
import random
class MatchModel:
    def __init__(self, order):
        self.order = order
        self.mask = 1 << (order << 1)
        self.ctx = 0
        self.hist = [3 for i in range(0, self.mask)]
        self.mask -= 1
    def predict(self):
        tmp = self.ctx
        for i in range(0, self.order):
            fnd = self.hist[tmp]
            if fnd != 3: return fnd
            tmp >>= 2
        return random.randint(0, 2)
    def update(self, val):
        tmp = self.ctx
        for i in range(0, self.order):
            self.hist[tmp] = val
            tmp >>= 2
    def next(self, val): self.ctx = (self.ctx << 2) + val + 1
    def done(self): self.ctx &= self.mask
    def upSgl(self, cval, hval):
        self.update(hval)
        self.next(cval)
        self.done()
    def upDbl(self, cv0, cv1, hval):
        self.update(hval)
        self.next(cv0)
        self.next(cv1)
        self.done()
class ModelPicker:
    def __init__(self, count, decay):
        self.count = count
        self.preds = [0 for i in range(0, count)]
        self.weigh = [0.0 for i in range(0, count)]
        self.decay = decay
    def setpred(self, idx, val): self.preds[idx] = val
    def setpreds(self, idx, vals):
        vl = len(vals)
        for vi in range(0, vl):
            self.preds[idx] = vals[vi]
            idx += 1
    def predict(self):
        idx = 0
        mxv = -10000.0
        tmv = 0
        for i in range(0, self.count):
            tmv = self.weigh[i]
            if tmv > mxv:
                mxv = tmv
                idx = i
        return self.preds[idx]
    def update(self, val):
        val *= 3
        for i in range(0, self.count):
            self.weigh[i] = (self.weigh[i] * self.decay) + VALUE[val + self.preds[i]]
if input == "":
    VALUE = [1,-1,0,0,1,-1,-1,0,1]
    WINS = [1,2,0]
    LOSS = [2,0,1]
    NVAL = {"R":0,"P":1,"S":2,"":0}
    CVAL = ["R", "P", "S"]
    cUU = MatchModel(8)
    cUT = MatchModel(8)
    cTU = MatchModel(8)
    cTT = MatchModel(8)
    cBU = MatchModel(8)
    cBT = MatchModel(8)
    sse = ModelPicker(28, 0.93)
    tval = random.randint(0, 2)
    uval = random.randint(0, 2)
else:
    tval = NVAL[input]
    cUU.upSgl(uval, uval)
    cUT.upSgl(uval, tval)
    cTU.upSgl(tval, uval)
    cTT.upSgl(tval, tval)
    cBU.upDbl(uval, tval, uval)
    cBT.upDbl(uval, tval, tval)
    sse.update(tval)
sse.preds[0] = cUU.predict()
sse.preds[1] = cUT.predict()
sse.preds[2] = cTU.predict()
sse.preds[3] = cTT.predict()
sse.preds[4] = cBU.predict()
sse.preds[5] = cBT.predict()
sse.preds[6] = tval
sse.preds[7] = uval
for i in range(0, 8):
    sse.preds[i + 8] = WINS[sse.preds[i]]
    sse.preds[i + 16] = WINS[sse.preds[i + 8]]
sse.setpreds(24, [0, 1, 2, random.randint(0, 2)])
uval = WINS[sse.predict()]
output = CVAL[uval]
""", '<string>', 'exec')

code_BothCM_CH10RP6 = compile(
    """
#Created by David Catt on December 13, 2014
import random
class CntModel:
    def __init__(self, decay):
        self.decay = decay
        self.prob = [0.0, 0.0, 0.0]
    def predict(self):
        if self.prob[0] > self.prob[1]:
            if self.prob[0] > self.prob[2]:
                return 0
            else:
                return 2
        else:
            if self.prob[1] > self.prob[2]:
                return 1
            else:
                return 2
    def update(self, val):
        self.prob[0] *= self.decay
        self.prob[1] *= self.decay
        self.prob[2] *= self.decay
        self.prob[val] += 1.0
class CtxModel:
    def __init__(self, order, decay):
        self.order = order
        self.decay = decay
        self.mask = (3 ** order) << 2
        self.prob = [0.0 if (i & 3) < 3 else random.randint(0, 2) for i in range(0, self.mask)]
        self.ctx = 0
    def predict(self):
        tvl = 0
        tpr = 0.0
        prb = 0.0
        p0 = self.prob[self.ctx]
        p1 = self.prob[self.ctx+1]
        p2 = self.prob[self.ctx+2]
        pm = 0
        if p1 > p0:
            if p1 > p2:
                pm = 1
            else:
                pm = 2
        elif p2 > p0:
            pm = 2
        rv = random.random() * (p0 + p1 + p2)
        if rv <= p0:
            return pm, self.prob[self.ctx + 3], 0
        elif rv <= p1:
            return pm, self.prob[self.ctx + 3], 1
        else:
            return pm, self.prob[self.ctx + 3], 2
    def update(self, cvl, val):
        self.prob[self.ctx] *= self.decay
        self.prob[self.ctx + 1] *= self.decay
        self.prob[self.ctx + 2] *= self.decay
        self.prob[self.ctx + 3] = val
        self.prob[self.ctx + val] += 1.0
        self.ctx = ((self.ctx * 3) + (cvl << 2)) % self.mask
    def nxtctx(self, cvl):
        self.ctx = ((self.ctx * 3) + (cvl << 2)) % self.mask
class ModelSwitch:
    def __init__(self, count, decay):
        self.value = [1,0,-1,-1,1,0,0,-1,1]
        self.count = count
        self.decay = decay
        self.weigh = [0.0] * count
        self.pred = [0] * count
    def setvalue(self, idx, val):
        self.pred[idx] = val
    def predict(self):
        return self.pred[self.weigh.index(max(self.weigh))]
    def update(self, val):
        val *= 3
        for i in range(0, self.count):
            self.weigh[i] = (self.weigh[i] * self.decay) + self.value[val + self.pred[i]]

if input == "":
    winner = [1,2,0]
    nval = {"R":0,"P":1,"S":2,"":0}
    cval = ["R","P","S"]
    mdls = [CtxModel((i % 6) + 1, 0.93) if i < 24 else CtxModel((((i - 24) % 4) + 1) * 2, 0.93) for i in range(0, 32)]
    zmdl0 = CntModel(0.93)
    zmdl1 = CntModel(0.93)
    msse = ModelSwitch(301, 0.93)
    lval = 0
else:
    val = nval[input]
    for i in range(0, 6):
        mdls[i].update(val, val)
        mdls[i+6].update(lval, val)
        mdls[i+12].update(val, lval)
        mdls[i+18].update(lval, lval)
    for i in range(24, 28):
        mdls[i].nxtctx(val)
        mdls[i].update(lval, val)
        mdls[i+4].nxtctx(val)
        mdls[i+4].update(lval, lval)
    zmdl0.update(val)
    zmdl1.update(lval)
    msse.update(val)
pv = 0
for i in range(0, 32):
    p1,p2,p3 = mdls[i].predict()
    msse.setvalue(i, p1)
    p1 = winner[p1]
    msse.setvalue(i + 32, p1)
    p1 = winner[p1]
    msse.setvalue(i + 64, p1)
    msse.setvalue(i + 96, p2)
    p2 = winner[p2]
    msse.setvalue(i + 128, p2)
    p2 = winner[p2]
    msse.setvalue(i + 160, p2)
    msse.setvalue(i + 192, p3)
    p3 = winner[p3]
    msse.setvalue(i + 224, p3)
    p3 = winner[p3]
    msse.setvalue(i + 256, p3)
pv = zmdl0.predict()
msse.setvalue(288, pv)
pv = winner[pv]
msse.setvalue(289, pv)
pv = winner[pv]
msse.setvalue(290, pv)
pv = zmdl1.predict()
msse.setvalue(291, pv)
pv = winner[pv]
msse.setvalue(292, pv)
pv = winner[pv]
msse.setvalue(293, pv)
pv = nval[input]
msse.setvalue(294, pv)
pv = winner[pv]
msse.setvalue(295, pv)
pv = winner[pv]
msse.setvalue(296, pv)
pv = lval
msse.setvalue(297, pv)
pv = winner[pv]
msse.setvalue(298, pv)
pv = winner[pv]
msse.setvalue(299, pv)
msse.setvalue(300, random.randint(0, 2))
lval = winner[msse.predict()]
output = cval[lval]
""", '<string>', 'exec')

code_kera = compile(
    """
def calcOutput(hist):
  q = [0 for k in range(3)]
  for i in range(0,3):
    q[i] = 1*w[i][c*6];
    for k in range(0,c*6):
      q[i] = q[i] + hist[k] * w[i][k]
  return q

def cvtHand(a):
  if a == 0:
    return 'R'
  if a == 1:
    return 'P'
  if a == 2:
    return 'S'

def cvtNum(a):
  if a == 'R':
    return 0
  if a == 'P':
    return 1
  if a == 'S':
    return 2

def calcHand(q):
  max = -99999;
  maxId = -1;
  for i in range(0,3):
    if max < q[i]:
      max = q[i]
      maxId = i
  return cvtHand(maxId)

def calcNextHist(op):
  nextHist = []
  nextHist.extend(hist)
  del nextHist[0:6]
  oph = [0 for k in range(3)]
  oph[cvtNum(op)] = 1
  nextHist = nextHist+oph+[0 for k in range(3)]
  return nextHist

def maxA(tmp):
  ret = tmp[0]
  for i in range(1,len(tmp)):
    if ret < tmp[i]:
      ret = tmp[i]
  return ret

def calcTeacher(op):
  teacher = [0 for k in range(3)]
  nHist = calcNextHist(op)
  winHandNum = (cvtNum(op)+1)%3
  loseHandNum = (cvtNum(op)+2)%3
  for i in range(0,3):
    nHist[c*6-3+i] = 1
    teacher[i] = g*maxA(calcOutput(nHist))
    nHist[c*6-3+i] = 0
    if winHandNum == i:
      teacher[i] += 1
    if loseHandNum == i:
      teacher[i] -= 1
  return teacher

def learn(op):
  dw = 0.000001
  idw = 1.0 / dw
  teacher = calcTeacher(op)
  out = calcOutput(hist)
  for i in range(0,3):
    oriError = pow(teacher[i] - out[i],2)/2
    error = pow(teacher[i] - out[i]+dw,2)/2
    w[i][c*6] += (error-oriError) * idw * r
    for j in range(0, c*6):
        w[i][j] += hist[j] * (error-oriError) * idw * r


def updateHist(op, my, h):
  oph = [0 for k in range(3)]
  myh = [0 for k in range(3)]
  del h[0:6]
  oph[cvtNum(op)] = 1
  myh[cvtNum(my)] = 1
  for i in range(0,3):
    h.append(oph[i])
  for i in range(0,3):
    h.append(myh[i])

if input == '':
  c = 7
  g = 0.5
  r = 1.0/(c*2+1) * 1.0
  w = [[0 for k in range(c*6+1)]for j in range(3)]
  hist = [0 for k in range(6*c)]
else:
  learn(input)
  updateHist(input, oldOutput, hist)

jjjj = calcOutput(hist)
output = calcHand(jjjj)
oldOutput = output

COREYSPROBS = {'R':jjjj[0],'P':jjjj[1],'S':jjjj[2]}
""", '<string>', 'exec')

code_noob = compile(
    """
xrange = range
#!/usr/bin/python
# -*- coding: utf-8 -*-
if input == '':
    import random
    import collections
    import math
    rchoice = random.choice
    randint = random.randint
    runit = random.random
    log = math.log
    sqrt = math.sqrt
    (R, P, S) = (0, 1, 2)
    RPS = [R, P, S]
    (T, W, L) = (R, P, S)
    PAYOFFS = RPS
    tr = {
        'R': R,
        'P': P,
        'S': S,
        R: 'R',
        P: 'P',
        S: 'S',
        }
    sub = [[T, L, W], [W, T, L], [L, W, T]]
    add = [[R, P, S], [P, S, R], [S, R, P]]
    (ties, beats, loses) = (add[T], add[W], add[L])

    pts = [0, 1, -1]
    near = [1, 0, 0]

    enc1 = [1, 2, 3]
    dec1 = [None, R, P, S]

    enc2 = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
    dec2 = [
        None,
        (R, R),
        (R, P),
        (R, S),
        (P, R),
        (P, P),
        (P, S),
        (S, R),
        (S, P),
        (S, S),
        ]


    def pick_max(vec):
        max_val = max(vec)
        max_list = [i for i in xrange(len(vec)) if vec[i] == max_val]
        return rchoice(max_list)


    def pick_weighted(vec):
        total = sum(vec) + 0.0
        u = runit() * total
        acc = 0.0
        for i in xrange(len(vec)):
            acc += vec[i]
            if u < acc:
                return i
        else:
            return randint(0, len(vec) - 1)


    def expected(vec):
        expected_payoffs = [vec[S] - vec[P], vec[R] - vec[S], vec[P]
                            - vec[R]]
        max_expected = max(expected_payoffs)
        max_list = [i for i in xrange(3) if expected_payoffs[i]
                    == max_expected]

        return rchoice(max_list)


    class GHPM:

        def __init__(self, ORDER, BASE):
            self.ORDER = ORDER
            self.BASE = BASE
            self.powers = [0] + [BASE ** i for i in xrange(ORDER)]
            self.hist = []
            self.contexts = collections.defaultdict(lambda : None)
            self.pred = None

        def update(self, next_val, up_val):
            self.hist.append(next_val)

            # update the history, order 0 as a special case

            up_ix = 0
            self.contexts[0] = up_val

            self.pred = self.contexts[0]
            elems = len(self.hist)
            for order in xrange(1, (self.ORDER + 1 if elems
                                > self.ORDER else elems)):
                pred_ix = up_ix * self.BASE + next_val

                up_ix += self.hist[-order - 1] * self.powers[order]
                self.contexts[up_ix] = up_val

                try_get = self.contexts[pred_ix]
                if try_get is not None:
                    self.pred = try_get


    STRATEGIES = range(27)
    my_ghpm = GHPM(6, 3)
    op_ghpm = GHPM(6, 3)
    both_ghpm = GHPM(6, 9)

    DECAY = 0.98
    sscores = [0 for _ in STRATEGIES]
    rev_sscores = [0 for _ in STRATEGIES]
    next_hands = [None for _ in STRATEGIES]

    PERIOD = 1
    META_DECAY = 0.98
    META_STRATEGIES = range(6)
    meta_sscores = [0 for _ in META_STRATEGIES]
    meta_next = [None for _ in META_STRATEGIES]
    meta_pick = 0

    next_hands[0] = R
    output = tr[rchoice(RPS)]
    hands = 1
    last_ix = 0
    score = 0
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
else:

    last_my = tr[output]
    last_op = tr[input]
    last_payoff = sub[last_my][last_op]

    my_ghpm.update(enc1[last_my], last_ix)
    op_ghpm.update(enc1[last_op], last_ix)
    both_ghpm.update(enc2[last_my][last_op], last_ix)

    if hands > 1:

        # update the scores

        for i in STRATEGIES:
            sscores[i] *= DECAY

            # sscores[i] += pts[sub[next_hands[i]][last_op]]

            sscores[i] += near[sub[next_hands[i]][last_op]]

            rev_sscores[i] *= DECAY

            # rev_sscores[i] += pts[sub[next_hands[i]][last_my]]

            rev_sscores[i] += near[sub[next_hands[i]][last_my]]

        for i in META_STRATEGIES:
            meta_sscores[i] *= META_DECAY
            meta_sscores[i] += pts[sub[meta_next[i]][last_op]]

    # update next_hands
    # constant needs no update

    # use only last

    next_hands[3] = last_my
    next_hands[6] = last_op

    # the bigger strategies

    both_hist = both_ghpm.hist

    (pred_op, pred_my) = dec2[both_hist[my_ghpm.pred]]
    next_hands[9] = pred_op
    next_hands[12] = pred_my

    (pred_op, pred_my) = dec2[both_hist[op_ghpm.pred]]
    next_hands[15] = pred_op
    next_hands[18] = pred_my

    (pred_op, pred_my) = dec2[both_hist[both_ghpm.pred]]
    next_hands[21] = pred_op
    next_hands[24] = pred_my

    for i in STRATEGIES:
        if i % 3 != 0:
            next_hands[i] = beats[next_hands[i - 1]]
    next_prob = [0, 0, 0]
    for i in STRATEGIES:
        next_prob[next_hands[i]] += sscores[i]
    next_my = expected(next_prob)
    rev_next_prob = [0, 0, 0]
    for i in STRATEGIES:
        rev_next_prob[next_hands[i]] += rev_sscores[i]
    rev_next_my = expected(rev_next_prob)
    meta_next[0] = next_my
    meta_next[3] = rev_next_my

    for i in META_STRATEGIES:
        if i % 3 != 0:
            meta_next[i] = beats[meta_next[i - 1]]

    meta_pick = pick_max(meta_sscores)
    meta_next_my = meta_next[meta_pick]
    output = tr[meta_next_my]
    hands += 1
    last_ix += 1
    score += pts[last_payoff]

    COREYSPROBS = {}
    for coreysidx in range(len(meta_sscores)):
        coreyskey = 'RPS'[meta_next[coreysidx]]
        if coreyskey not in COREYSPROBS.keys():
            COREYSPROBS[coreyskey] = meta_sscores[coreysidx]
        elif meta_sscores[coreysidx] > COREYSPROBS[coreyskey]:
            COREYSPROBS[coreyskey] = meta_sscores[coreysidx]

""", '<string>', 'exec')


code_spinoza13 = compile(
    '''
# https://www.kaggle.com/riccardosanson/rps-simple-rfind-agent
if not input:
    max_limit = 13  # can be modified
    add_rotations = True

    # number of predictors
    numPre = 6
    if add_rotations:
        numPre *= 3

    # number of meta-predictors
    numMeta = 4
    if add_rotations:
        numMeta *= 3

    # saves history
    moves = ['', '', '']

    beat = {'R':'P', 'P':'S', 'S':'R'}
    dna =  {'RP':0, 'PS':1, 'SR':2,
            'PR':3, 'SP':4, 'RS':5,
            'RR':6, 'PP':7, 'SS':8}

    p = ["P"]*numPre
    m = ["P"]*numMeta
    pScore = [[0]*numPre for i in range(8)]
    mScore = [0]*numMeta

    length = 0
    threat = 0
    output = "P"
    step = 0
    output = beat[output]
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
elif step ==1:
    output = beat[output]
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
else:
    outcome = (beat[input]==output) - (input==beat[output])
    threat = 0.9*threat - 0.1*outcome

    # refresh pScore
    for i in range(numPre):
        pp = p[i]
        bpp = beat[pp]
        bbpp = beat[bpp]
        pScore[0][i] = 0.9*pScore[0][i] + 0.1*((input==pp)-(input==bbpp))
        pScore[1][i] = 0.9*pScore[1][i] + 0.1*((output==pp)-(output==bbpp))
        pScore[2][i] = 0.8*pScore[2][i] + 0.3*((input==pp)-(input==bbpp)) + 0.1*(length % 3 - 1)
        pScore[3][i] = 0.8*pScore[3][i] + 0.3*((output==pp)-(output==bbpp)) + 0.1*(length % 3 - 1)

    # refresh mScore
    for i in range(numMeta):
        mScore[i] = 0.9*mScore[i] + 0.1*((input==m[i])-(input==beat[beat[m[i]]])) + 0.05*(length % 5 - 2)

    # refresh moves
    moves[0] += str(dna[input+output])
    moves[1] += input
    moves[2] += output

    # refresh length
    length += 1

    # new predictors
    limit = min([length,max_limit])
    for y in range(3):    # my moves, his, and both
        j = limit
        while j>=1 and not moves[y][length-j:length] in moves[y][0:length-1]:
            j-=1
        if j>=1:
            i = moves[y].rfind(moves[y][length-j:length],0,length-1)
            p[0+2*y] = moves[1][j+i] 
            p[1+2*y] = beat[moves[2][j+i]]

    # rotations of predictors
    if add_rotations:
        for i in range(int(numPre/3),numPre):
            p[i]=beat[beat[p[i-int(numPre/3)]]]

    # new meta
    for i in range(0,4,2):
        m[i] = p[pScore[i].index(max(pScore[i]))]
        m[i+1] = beat[p[pScore[i+1].index(max(pScore[i+1]))]]

    # rotations of meta
    if add_rotations:
        for i in range(4,12):
            m[i]=beat[beat[m[i-4]]]

    # - - -
    
    output = beat[m[mScore.index(max(mScore))]]

    COREYSPROBS = {}
    for coreysidx in range(len(m)):
        coreyskey = beat[m[coreysidx]]
        if coreyskey not in COREYSPROBS.keys():
            COREYSPROBS[coreyskey] = mScore[coreysidx]
        elif mScore[coreysidx] > COREYSPROBS[coreyskey]:
            COREYSPROBS[coreyskey] = mScore[coreysidx]

    if threat > 0.4:
        # ah take this!
        output = beat[beat[output]]
    
step += 1

''', '<string>', 'exec')

code_spinoza23 = compile(
    '''
# https://www.kaggle.com/riccardosanson/rps-simple-rfind-agent
if not input:
    max_limit = 23  # can be modified
    add_rotations = True

    # number of predictors
    numPre = 6
    if add_rotations:
        numPre *= 3

    # number of meta-predictors
    numMeta = 4
    if add_rotations:
        numMeta *= 3

    # saves history
    moves = ['', '', '']

    beat = {'R':'P', 'P':'S', 'S':'R'}
    dna =  {'RP':0, 'PS':1, 'SR':2,
            'PR':3, 'SP':4, 'RS':5,
            'RR':6, 'PP':7, 'SS':8}

    p = ["P"]*numPre
    m = ["P"]*numMeta
    pScore = [[0]*numPre for i in range(8)]
    mScore = [0]*numMeta

    length = 0
    threat = 0
    output = "P"
    step = 0
    output = beat[output]
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
elif step ==1:
    output = beat[output]
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
else:
    outcome = (beat[input]==output) - (input==beat[output])
    threat = 0.9*threat - 0.1*outcome

    # refresh pScore
    for i in range(numPre):
        pp = p[i]
        bpp = beat[pp]
        bbpp = beat[bpp]
        pScore[0][i] = 0.9*pScore[0][i] + 0.1*((input==pp)-(input==bbpp))
        pScore[1][i] = 0.9*pScore[1][i] + 0.1*((output==pp)-(output==bbpp))
        pScore[2][i] = 0.8*pScore[2][i] + 0.3*((input==pp)-(input==bbpp)) + 0.1*(length % 3 - 1)
        pScore[3][i] = 0.8*pScore[3][i] + 0.3*((output==pp)-(output==bbpp)) + 0.1*(length % 3 - 1)

    # refresh mScore
    for i in range(numMeta):
        mScore[i] = 0.9*mScore[i] + 0.1*((input==m[i])-(input==beat[beat[m[i]]])) + 0.05*(length % 5 - 2)

    # refresh moves
    moves[0] += str(dna[input+output])
    moves[1] += input
    moves[2] += output

    # refresh length
    length += 1

    # new predictors
    limit = min([length,max_limit])
    for y in range(3):    # my moves, his, and both
        j = limit
        while j>=1 and not moves[y][length-j:length] in moves[y][0:length-1]:
            j-=1
        if j>=1:
            i = moves[y].rfind(moves[y][length-j:length],0,length-1)
            p[0+2*y] = moves[1][j+i] 
            p[1+2*y] = beat[moves[2][j+i]]

    # rotations of predictors
    if add_rotations:
        for i in range(int(numPre/3),numPre):
            p[i]=beat[beat[p[i-int(numPre/3)]]]

    # new meta
    for i in range(0,4,2):
        m[i] = p[pScore[i].index(max(pScore[i]))]
        m[i+1] = beat[p[pScore[i+1].index(max(pScore[i+1]))]]

    # rotations of meta
    if add_rotations:
        for i in range(4,12):
            m[i]=beat[beat[m[i-4]]]

    # - - -
    
    output = beat[m[mScore.index(max(mScore))]]
    COREYSPROBS = {}
    for coreysidx in range(len(m)):
        coreyskey = beat[m[coreysidx]]
        if coreyskey not in COREYSPROBS.keys():
            COREYSPROBS[coreyskey] = mScore[coreysidx]
        elif mScore[coreysidx] > COREYSPROBS[coreyskey]:
            COREYSPROBS[coreyskey] = mScore[coreysidx]

    if threat > 0.4:
        # ah take this!
        output = beat[beat[output]]
    
step += 1

''', '<string>', 'exec')

code_pi = compile(
    '''
import re

PI = "3.14159 26535 89793 23846 26433 83279 50288 41971 69399 37510 58209 74944 59230 78164 06286 20899 86280 34825 34211 70679 82148 08651 32823 06647 09384 46095 50582 23172 53594 08128 48111 74502 84102 70193 85211 05559 64462 29489 54930 38196 44288 10975 66593 34461 28475 64823 37867 83165 27120 19091 45648 56692 34603 48610 45432 66482 13393 60726 02491 41273 72458 70066 06315 58817 48815 20920 96282 92540 91715 36436 78925 90360 01133 05305 48820 46652 13841 46951 94151 16094 33057 27036 57595 91953 09218 61173 81932 61179 31051 18548 07446 23799 62749 56735 18857 52724 89122 79381 83011 94912 98336 73362 44065 66430 86021 39494 63952 24737 19070 21798 60943 70277 05392 17176 29317 67523 84674 81846 76694 05132 00056 81271 45263 56082 77857 71342 75778 96091 73637 17872 14684 40901 22495 34301 46549 58537 10507 92279 68925 89235 42019 95611 21290 21960 86403 44181 59813 62977 47713 09960 51870 72113 49999 99837 29780 49951 05973 17328 16096 31859 50244 59455 34690 83026 42522 30825 33446 85035 26193 11881 71010 00313 78387 52886 58753 32083 81420 61717 76691 47303 59825 34904 28755 46873 11595 62863 88235 37875 93751 95778 18577 80532 17122 68066 13001 92787 66111 95909 21642 01989 38095 25720 10654 85863 27886 59361 53381 82796 82303 01952 03530 18529 68995 77362 25994 13891 24972 17752 83479 13151 55748 57242 45415 06959 50829 53311 68617 27855 88907 50983 81754 63746 49393 19255 06040 09277 01671 13900 98488 24012 85836 16035 63707 66010 47101 81942 95559 61989 46767 83744 94482 55379 77472 68471 04047 53464 62080 46684 25906 94912 93313 67702 89891 52104 75216 20569 66024 05803 81501 93511 25338 24300 35587 64024 74964 73263 91419 92726 04269 92279 67823 54781 63600 93417 21641 21992 45863 15030 28618 29745 55706 74983 85054 94588 58692 69956 90927 21079 75093 02955 32116 53449 87202 75596 02364 80665 49911 98818 34797 75356 63698 07426 54252 78625 51818 41757 46728 90977 77279 38000 81647 06001 61452 49192 17321 72147 72350 14144 19735 68548 16136 11573 52552 13347 57418 49468 43852 33239 07394 14333 45477 62416 86251 89835 69485 56209 92192 22184 27255 02542 56887 67179 04946 01653 46680 49886 27232 79178 60857 84383 82796 79766 81454 10095 38837 86360 95068 00642 25125 20511 73929 84896 08412 84886 26945 60424 19652 85022 21066 11863 06744 27862 20391 94945 04712 37137 86960 95636 43719 17287 46776 46575 73962 41389 08658 32645 99581 33904 78027 59009 94657 64078 95126 94683 98352 59570 98258 22620 52248 94077 26719 47826 84826 01476 99090 26401 36394 43745 53050 68203 49625 24517 49399 65143 14298 09190 65925 09372 21696 46151 57098 58387 41059 78859 59772 97549 89301 61753 92846 81382 68683 86894 27741 55991 85592 52459 53959 43104 99725 24680 84598 72736 44695 84865 38367 36222 62609 91246 08051 24388 43904 51244 13654 97627 80797 71569 14359 97700 12961 60894 41694 86855 58484 06353 42207 22258 28488 64815 84560 28506 01684 27394 52267 46767 88952 52138 52254 99546 66727 82398 64565 96116 35488 62305 77456 49803 55936 34568 17432 41125 15076 06947 94510 96596 09402 52288 79710 89314 56691 36867 22874 89405 60101 50330 86179 28680 92087 47609 17824 93858 90097 14909 67598 52613 65549 78189 31297 84821 68299 89487 22658 80485 75640 14270 47755 51323 79641 45152 37462 34364 54285 84447 95265 86782 10511 41354 73573 95231 13427 16610 21359 69536 23144 29524 84937 18711 01457 65403 59027 99344 03742 00731 05785 39062 19838 74478 08478 48968 33214 45713 86875 19435 06430 21845 31910 48481 00537 06146 80674 91927 81911 97939 95206 14196 63428 75444 06437 45123 71819 21799 98391 01591 95618 14675 14269 12397 48940 90718 64942 31961 56794 52080 95146 55022 52316 03881 93014 20937 62137 85595 66389 37787 08303 90697 92077 34672 21825 62599 66150 14215 03068 03844 77345 49202 60541 46659 25201 49744 28507 32518 66600 21324 34088 19071 04863 31734 64965 14539 05796 26856 10055 08106 65879 69981 63574 73638 40525 71459 10289 70641 40110 97120 62804 39039 75951 56771 57700 42033 78699 36007 23055 87631 76359 42187 31251 47120 53292 81918 26186 12586 73215 79198 41484 88291 64470 60957 52706 95722 09175 67116 72291 09816 90915 28017 35067 12748 58322 28718 35209 35396 57251 21083 57915 13698 82091 44421 00675 10334 67110 31412 67111 36990 86585 16398 31501 97016 51511 68517 14376 57618 35155 65088 49099 89859 98238 73455 28331 63550 76479 18535 89322 61854 89632 13293 30898 57064 20467 52590 70915 48141 65498 59461 63718 02709 81994 30992 44889 57571 28289 05923 23326 09729 97120 84433 57326 54893 82391 19325 97463 66730 58360 41428 13883 03203 82490 37589 85243 74417 02913 27656 18093 77344 40307 07469 21120 19130 20330 38019 76211 01100 44929 32151 60842 44485 96376 69838 95228 68478 31235 52658 21314 49576 85726 24334 41893 03968 64262 43410 77322 69780 28073 18915 44110 10446 82325 27162 01052 65227 21116 60396 66557 30925 47110 55785 37634 66820 65310 98965 26918 62056 47693 12570 58635 66201 85581 00729 36065 98764 86117 91045 33488 50346 11365 76867 53249 44166 80396 26579 78771 85560 84552 96541 26654 08530 61434 44318 58676 97514 56614 06800 70023 78776 59134 40171 27494 70420 56223 05389 94561 31407 11270 00407 85473 32699 39081 45466 46458 80797 27082 66830 63432 85878 56983 05235 80893 30657 57406 79545 71637 75254 20211 49557 61581 40025 01262 28594 13021 64715 50979 25923 09907 96547 37612 55176 56751 35751 78296 66454 77917 45011 29961 48903 04639 94713 29621 07340 43751 89573 59614 58901 93897 13111 79042 97828 56475 03203 19869 15140 28708 08599 04801 09412 14722 13179 47647 77262 24142 54854 54033 21571 85306 14228 81375 85043 06332 17518 29798 66223 71721 59160 77166 92547 48738 98665 49494 50114 65406 28433 66393 79003 97692 65672 14638 53067 36096 57120 91807 63832 71664 16274 88880 07869 25602 90228 47210 40317 21186 08204 19000 42296 61711 96377 92133 75751 14959 50156 60496 31862 94726 54736 42523 08177 03675 15906 73502 35072 83540 56704 03867 43513 62222 47715 89150 49530 98444 89333 09634 08780 76932 59939 78054 19341 44737 74418 42631 29860 80998 88687 41326 04721 56951 62396 58645 73021 63159 81931 95167 35381 29741 67729 47867 24229 24654 36680 09806 76928 23828 06899 64004 82435 40370 14163 14965 89794 09243 23789 69070 69779 42236 25082 21688 95738 37986 23001 59377 64716 51228 93578 60158 81617 55782 97352 33446 04281 51262 72037 34314 65319 77774 16031 99066 55418 76397 92933 44195 21541 34189 94854 44734 56738 31624 99341 91318 14809 27777 10386 38773 43177 20754 56545 32207 77092 12019 05166 09628 04909 26360 19759 88281 61332 31666 36528 61932 66863 36062 73567 63035 44776 28035 04507 77235 54710 58595 48702 79081 43562 40145 17180 62464 36267 94561 27531 81340 78330 33625 42327 83944 97538 24372 05835 31147 71199 26063 81334 67768 79695 97030 98339 13077 10987 04085 91337 46414 42822 77263 46594 70474 58784 77872 01927 71528 07317 67907 70715 72134 44730 60570 07334 92436 93113 83504 93163 12840 42512 19256 51798 06941 13528 01314 70130 47816 43788 51852 90928 54520 11658 39341 96562 13491 43415 95625 86586 55705 52690 49652 09858 03385 07224 26482 93972 85847 83163 05777 75606 88876 44624 82468 57926 03953 52773 48030 48029 00587 60758 25104 74709 16439 61362 67604 49256 27420 42083 20856 61190 62545 43372 13153 59584 50687 72460 29016 18766 79524 06163 42522 57719 54291 62991 93064 55377 99140 37340 43287 52628 88963 99587 94757 29174 64263 57455 25407 90914 51357 11136 94109 11939 32519 10760 20825 20261 87985 31887 70584 29725 91677 81314 96990 09019 21169 71737 27847 68472 68608 49003 37702 42429 16513 00500 51683 23364 35038 95170 29893 92233 45172 20138 12806 96501 17844 08745 19601 21228 59937 16231 30171 14448 46409 03890 64495 44400 61986 90754 85160 26327 50529 83491 87407 86680 88183 38510 22833 45085 04860 82503 93021 33219 71551 84306 35455 00766 82829 49304 13776 55279 39751 75461 39539 84683 39363 83047 46119 96653 85815 38420 56853 38621 86725 23340 28308 71123 28278 92125 07712 62946 32295 63989 89893 58211 67456 27010 21835 64622 01349 67151 88190 97303 81198 00497 34072 39610 36854 06643 19395 09790 19069 96395 52453 00545 05806 85501 95673 02292 19139 33918 56803 44903 98205 95510 02263 53536 19204 19947 45538 59381 02343 95544 95977 83779 02374 21617 27111 72364 34354 39478 22181 85286 24085 14006 66044 33258 88569 86705 43154 70696 57474 58550 33232 33421 07301 54594 05165 53790 68662 73337 99585 11562 57843 22988 27372 31989 87571 41595 78111 96358 33005 94087 30681 21602 87649 62867 44604 77464 91599 50549 73742 56269 01049 03778 19868 35938 14657 41268 04925 64879 85561 45372 34786 73303 90468 83834 36346 55379 49864 19270 56387 29317 48723 32083 76011 23029 91136 79386 27089 43879 93620 16295 15413 37142 48928 30722 01269 01475 46684 76535 76164 77379 46752 00490 75715 55278 19653 62132 39264 06160 13635 81559 07422 02020 31872 77605 27721 90055 61484 25551 87925 30343 51398 44253 22341 57623 36106 42506 39049 75008 65627 10953 59194 65897 51413 10348 22769 30624 74353 63256 91607 81547 81811 52843 66795 70611 08615 33150 44521 27473 92454 49454 23682 88606 13408 41486 37767 00961 20715 12491 40430 27253 86076 48236 34143 34623 51897 57664 52164 13767 96903 14950 19108 57598 44239 19862 91642 19399 49072 36234 64684 41173 94032 65918 40443 78051 33389 45257 42399 50829 65912 28508 55582 15725 03107 12570 12668 30240 29295 25220 11872 67675 62204 15420 51618 41634 84756 51699 98116 14101 00299 60783 86909 29160 30288 40026 91041 40792 88621 50784 24516 70908 70006 99282 12066 04183 71806 53556 72525 32567 53286 12910 42487 76182 58297 65157 95984 70356 22262 93486 00341 58722 98053 49896 50226 29174 87882 02734 20922 22453 39856 26476 69149 05562 84250 39127 57710 28402 79980 66365 82548 89264 88025 45661 01729 67026 64076 55904 29099 45681 50652 65305 37182 94127 03369 31378 51786 09040 70866 71149 65583 43434 76933 85781 71138 64558 73678 12301 45876 87126 60348 91390 95620 09939 36103 10291 61615 28813 84379 09904 23174 73363 94804 57593 14931 40529 76347 57481 19356 70911 01377 51721 00803 15590 24853 09066 92037 67192 20332 29094 33467 68514 22144 77379 39375 17034 43661 99104 03375 11173 54719 18550 46449 02636 55128 16228 82446 25759 16333 03910 72253 83742 18214 08835 08657 39177 15096 82887 47826 56995 99574 49066 17583 44137 52239 70968 34080 05355 98491 75417 38188 39994 46974 86762 65516 58276 58483 58845 31427 75687 90029 09517 02835 29716 34456 21296 40435 23117 60066 51012 41200 65975 58512 76178 58382 92041 97484 42360 80071 93045 76189 32349 22927 96501 98751 87212 72675 07981 25547 09589 04556 35792 12210 33346 69749 92356 30254 94780 24901 14195 21238 28153 09114 07907 38602 51522 74299 58180 72471 62591 66854 51333 12394 80494 70791 19153 26734 30282 44186 04142 63639 54800 04480 02670 49624 82017 92896 47669 75831 83271 31425 17029 69234 88962 76684 40323 26092 75249 60357 99646 92565 04936 81836 09003 23809 29345 95889 70695 36534 94060 34021 66544 37558 90045 63288 22505 45255 64056 44824 65151 87547 11962 18443 96582 53375 43885 69094 11303 15095 26179 37800 29741 20766 51479 39425 90298 96959 46995 56576 12186 56196 73378 62362 56125 21632 08628 69222 10327 48892 18654 36480 22967 80705 76561 51446 32046 92790 68212 07388 37781 42335 62823 60896 32080 68222 46801 22482 61177 18589 63814 09183 90367 36722 20888 32151 37556 00372 79839 40041 52970 02878 30766 70944 47456 01345 56417 25437 09069 79396 12257 14298 94671 54357 84687 88614 44581 23145 93571 98492 25284 71605 04922 12424 70141 21478 05734 55105 00801 90869 96033 02763 47870 81081 75450 11930 71412 23390 86639 38339 52942 57869 05076 43100 63835 19834 38934 15961 31854 34754 64955 69781 03829 30971 64651 43840 70070 73604 11237 35998 43452 25161 05070 27056 23526 60127 64848 30840 76118 30130 52793 20542 74628 65403 60367 45328 65105 70658 74882 25698 15793 67897 66974 22057 50596 83440 86973 50201 41020 67235 85020 07245 22563 26513 41055 92401 90274 21624 84391 40359 98953 53945 90944 07046 91209 14093 87001 26456 00162 37428 80210 92764 57931 06579 22955 24988 72758 46101 26483 69998 92256 95968 81592 05600 10165 52563 7567"
PI = re.sub('[^1-9]', '', PI)

if not input:
    step = 0
output = int(PI[step]) % 3
output = 'RPS'[output]
step += 1
COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}

''', '<string>', 'exec')

code_sunocos = compile(
    '''
    # https://www.kaggle.com/sunocos/very-simple-method
import random
import numpy as np
import secrets

if not input:
    recs = []
    output = 'RPS'[random.randint(0,2)]
else:
    recs.append({'R':0,'P':1,'S':2}[input])
    count = len(recs)
    hand_count = [recs.count(0), recs.count(1), recs.count(2)]
    #hand_ratio = [hand_count[0], hand_count[0]+hand_count[1], count] / count

    candidates = []
    for i, ratios in enumerate(hand_count):
        if ratios == np.max(hand_count):
            candidates.append(i)
            
    output = secrets.choice(candidates)
    output = (output + 1) % 3
    output = 'RPS'[output]

''', '<string>', 'exec')

code_rf = compile(
    """
import random
import numpy as np
from sklearn.ensemble import RandomForestClassifier

if not input:
    actions =  np.empty((0,0), dtype = int)
    observations =  np.empty((0,0), dtype = int)
    total_reward = 0
    output = random.randint(0,2)
    actions = np.append(actions , [output])
    step = 0
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
elif step == 1:
    output = random.randint(0,2)
    actions = np.append(actions , [output])
    observations = np.append(observations , [{'R':0,'P':1,'S':2}[input]])
    # Keep track of score
    winner = int((3 + actions[-1] - {'R':0,'P':1,'S':2}[input]) % 3);
    if winner == 1:
        total_reward = total_reward + 1
    elif winner == 2:
        total_reward = total_reward - 1
    COREYSPROBS = {'R':0.33,'P':0.33,'S':0.33}
else:
    # Get Observation to make the tables (actions & obervations) even.
    observations = np.append(observations , [{'R':0,'P':1,'S':2}[input]])
    
    # Prepare Data for training
    # :-1 as we dont have feedback yet.
    X_train = np.vstack((actions[:-1], observations[:-1])).T
    
    # Create Y by rolling observations to bring future a step earlier 
    shifted_observations = np.roll(observations, -1)
    
    # trim rolled & last element from rolled observations
    y_train = shifted_observations[:-1].T
    
    # Set the history period. Long chains here will need a lot of time
    if len(X_train) > 25:
        random_window_size = 10 + 5 #random.randint(0,10)
        X_train = X_train[-random_window_size:]
        y_train = y_train[-random_window_size:]
   
    # Train a classifier model
    model = RandomForestClassifier(n_estimators=25)
    model.fit(X_train, y_train)

    # Predict
    X_test = np.empty((0,0), dtype = int)
    X_test = np.append(X_test, [int(actions[-1]), {'R':0,'P':1,'S':2}[input]])
    prediction = model.predict(X_test.reshape(1, -1))
    prediction2 = model.predict_proba(X_test.reshape(1, -1))[0]

    COREYSPROBS = {}
    for coreysidx, pred in enumerate(np.unique(y_train)):
        coreyskey = 'RPS'[(pred + 1)%3]
        COREYSPROBS[coreyskey] = prediction2[coreysidx]

    # Keep track of score
    winner = int((3 + actions[-1] - {'R':0,'P':1,'S':2}[input]) % 3);
    if winner == 1:
        total_reward = total_reward + 1
    elif winner == 2:
        total_reward = total_reward - 1
   
    # Prepare action
    output = int((prediction + 1) % 3)
    
    # If losing a bit then change strategy and break the patterns by playing a bit random
    if total_reward < -2:
        win_tie = random.choice([0,1])
        output = int((prediction + win_tie) % 3)

    # Update actions
    actions = np.append(actions , [output])
   
step += 1
output = 'RPS'[output]
""", '<string>', 'exec')

code_ant = compile(
    """
import operator
import numpy as np
import cmath
from typing import List
from collections import namedtuple
import traceback
import sys


basis = np.array(
    [1, cmath.exp(2j * cmath.pi * 1 / 3), cmath.exp(2j * cmath.pi * 2 / 3)]
)


HistMatchResult = namedtuple("HistMatchResult", "idx length")


def find_all_longest(seq, max_len=None) -> List[HistMatchResult]:
    #Find all indices where end of `seq` matches some past.
    result = []

    i_search_start = len(seq) - 2

    while i_search_start > 0:
        i_sub = -1
        i_search = i_search_start
        length = 0

        while i_search >= 0 and seq[i_sub] == seq[i_search]:
            length += 1
            i_sub -= 1
            i_search -= 1

            if max_len is not None and length > max_len:
                break

        if length > 0:
            result.append(HistMatchResult(i_search_start + 1, length))

        i_search_start -= 1

    result = sorted(result, key=operator.attrgetter("length"), reverse=True)

    return result


def probs_to_complex(p):
    return p @ basis


def _fix_probs(probs):
    #Put probs back into triangle. Sometimes this happens due to rounding errors or if you
    #use complex numbers which are outside the triangle.
    if min(probs) < 0:
        probs -= min(probs)

    probs /= sum(probs)

    return probs


def complex_to_probs(z):
    probs = (2 * np.array(z * basis.conjugate()).real + 1) / 3
    probs = _fix_probs(probs)
    return probs


def z_from_action(action):
    return basis[action]


def sample_from_z(z):
    probs = complex_to_probs(z)
    #np.random.seed(99)
    #print('MINE:',probs)
    return np.random.choice(3, p=probs)


def bound(z):
    return probs_to_complex(complex_to_probs(z))


def norm(z):
    return bound(z / abs(z))


class Pred:
    def __init__(self, *, alpha, min_abs=0.01):
        self.offset = 0
        self.alpha = alpha
        self.last_feat = None
        self.min_abs = min_abs

    def train(self, target):
        if self.last_feat is not None:
            offset = target / self.last_feat

            self.offset = (1 - self.alpha) * self.offset + self.alpha * offset

    def predict(self, feat):
        #feat is an arbitrary feature with a probability on 0,1,2
        #anything which could be useful anchor to start with some kind of sensible direction
        if abs(feat) <= self.min_abs:
            self.last_feat = None
            return 0

        feat = norm(feat)

        # offset = mean(target - feat)
        # so here we see something like: result = feat + mean(target - feat)
        # which seem natural and accounts for the correlation between target and feat
        # all RPSContest bots do no more than that, just in a hidden way
        
        result = feat * self.offset

        self.last_feat = feat

        return result
    
    
class BaseAgent:
    def __init__(self):
        self.my_hist = [] # The moves that I make
        self.opp_hist = [] # The moves my opponent makes
        self.my_opp_hist = [] # list of tuples of (my_move, opponents_move)
        self.outcome_hist = [] # 0 for tie, 1 for win, -1 for loss
        self.step = 0

    def __call__(self, input):
        try:
            if self.step == 0:
                action = np.random.choice(3)
                self.my_hist.append(action)
                self.step += 1
                return action, [0.33, 0.33, 0.33]
                
            self.step += 1

            opp = int({'R':0,'P':1,'S':2}[input]) # My opponent most recent move
            my = self.my_hist[-1] # My most recent move

            self.my_opp_hist.append((my, opp))
            self.opp_hist.append(opp)

            outcome = {0: 0, 1: 1, 2: -1}[(my - opp) % 3] # 0 for tie, 1 for win, -1 for loss
            self.outcome_hist.append(outcome)

            action, jjj = self.action()

            self.my_hist.append(action) # IMPORTANT FOR ANTI!!!!!!!!

            return action, jjj
        except Exception:
            traceback.print_exc(file=sys.stderr)
            raise

    def action(self):
        pass


class Agent(BaseAgent):
    def __init__(self, alpha=0.01):
        super().__init__()

        self.predictor = Pred(alpha=alpha)

    def action(self):
        self.train()

        pred = self.preds()

        return_action = sample_from_z(pred) # Choose np.random.choice, pred is a probability of [0,1,2]

        return return_action, complex_to_probs(pred)

    def train(self):
        last_beat_opp = z_from_action((self.opp_hist[-1] + 1) % 3) # "last_beat_opp" is the complex number which beats the most recent opponent action
        self.predictor.train(last_beat_opp)

    def preds(self):
        hist_match = find_all_longest(self.my_opp_hist, max_len=20)

        if not hist_match:
             return 0 # In the middle of the triangle, so it becomes 33% for each probability

        feat = z_from_action(self.opp_hist[hist_match[0].idx])

        pred = self.predictor.predict(feat)

        return pred
        
if not input:
    agent = Agent()

aaaa, bbbb = agent(input)
output = 'RPS'[aaaa]
COREYSPROBS = {'R':bbbb[0],'P':bbbb[1],'S':bbbb[2]}

""", '<string>', 'exec')


import random
import secrets
import numpy as np
import datetime
score_so_far = 0 # The score so far (negative = im losing, positive = im winning)
hist = []
opp_preds = {
    #'deepbot':[],
    'mutation':[],
    'diludefense':[],
    'dumbneural4':[],
    'puppy1003':[],
    'neural12':[],
    'puppy16H':[],
    'puppy17':[],
    'roseguadeloupe':[],
    'randomdropswitch':[],
   # 'omelettedeimos':[],
    'rps_meta_fixed':[],
    'rps_meta_fixed2':[],
    'rps_meta_fixed3':[],
    'rps_meta_fixed4':[],
    'rps_meta_fixed5':[],
    'rps_meta_fixed6':[],
    'rps_meta_fixed7':[],
    'rps_meta_fixed8':[],
    'rps_meta_fixed9':[],
    'rps_meta_fix':[],
    'skipblenderfixed':[],
    'ZASMC':[],
    'flight825':[],
   # 'funbots':[],
    'decay6':[],
    'derp':[],
   # 'metaioc':[], #
#     'db10':[],
#    'german':{0:[],1:[],2:[]},
#     'farn':[],
#     'galton':[],
#     'laurius':[],
#     'zazaza':[],
#     'sr4':[],
    'notorious':[], #
    #'john2':[],
    'cereal4':[],
    'zdgsteal':[],
    'ZCC2':[],
    'ZMH3':[],
    'zaibayes':[],
    'zaimarkov':[],
    'zaidfa3':[],
    'lucker':[],
    'dilu1':[],
#     'ignore2':[],
    'io2fighting':[],
    'bayes14':[],
 #   'random':[],
 #   'bandit':[],
 #   'decisiontree':[],
    'greenberg':[],
    #'iocaine':[],
    'rps30':[],
  #  'dsad':[],
    #'copymodel':[],
    #'BothCM_CH10RP6':[],
    'kera':[],
    'noob':[],
    'spinoza13':[],
    'spinoza23':[],
 #   'pi':[],
    #'sunocos':[],
    'rf':[],
    'ant':[]
}

anti_opp_preds = {}
for strat in opp_preds.keys():
    anti_opp_preds[strat] = []
    
my_preds_prob = {}
for strat in opp_preds.keys():
    my_preds_prob[strat] = []
    
# anti_opp_preds_prob = {}
# for strat in opp_preds.keys():
#     anti_opp_preds_prob[strat] = []

my_ggs = {}
for strat in opp_preds.keys():
    my_ggs[strat] = {}
    
anti_ggs = {}
for strat in opp_preds.keys():
    anti_ggs[strat] = {}
    
my_codes = {
    #'deepbot':code_deepbot,
    'mutation':code_mutation,
    'diludefense':code_diludefensive,
    'dumbneural4':code_dumbneural4,
    'puppy1003':code_puppy1003,
    'neural12':code_neural12,
    'puppy16H':code_puppy16H,
    'puppy17':code_puppy17,
    'roseguadeloupe':code_roseguadeloupe,
    'randomdropswitch':code_randomdropswitch,
    #'omelettedeimos':code_omelettedeimos,
    'rps_meta_fixed':code_rps_meta_fixed,
    'rps_meta_fixed2':code_rps_meta_fixed2,
    'rps_meta_fixed3':code_rps_meta_fixed3,
    'rps_meta_fixed4':code_rps_meta_fixed4,
    'rps_meta_fixed5':code_rps_meta_fixed5,
    'rps_meta_fixed6':code_rps_meta_fixed6,
    'rps_meta_fixed7':code_rps_meta_fixed7,
    'rps_meta_fixed8':code_rps_meta_fixed8,
    'rps_meta_fixed9':code_rps_meta_fixed9,
    'rps_meta_fix':code_rps_meta_fix,
    'skipblenderfixed':code_skipblenderfixed,
    'ZASMC':code_ZASMC,
    'flight825':code_flight825,
  #  'funbots':code_funbots,
    'decay6':code_decay6,
    'derp':code_derp,
    #'metaioc':code_metaioc,
#     'db10':[],
#    'german':code_german,
#     'farn':[],
#     'galton':[],
#     'laurius':[],
#     'zazaza':[],
#     'sr4':[],
    'notorious':code_notorious,
    #'john2':code_john2,
    'cereal4':code_cereal4,
    'zdgsteal':code_zdgsteal,
    'ZCC2':code_ZCC2,
    'ZMH3':code_ZMH3,
    'zaibayes':code_zaibayes,
    'zaimarkov':code_zaimarkov,
    'zaidfa3':code_zaidfa3,
    'lucker':code_lucker,
    'dilu1':code_dilu1,
#     'ignore2':[],
    'io2fighting':code_io2fighting,
    'bayes14':code_bayes14,
 #   'random':[],
 #   'bandit':[],
 #   'decisiontree':[],
    #'iocaine':code_iocaine,
    'greenberg':code_greenberg,
    'rps30':code_rps30,
 #   'dsad':code_dsad,
    #'copymodel':code_copymodel,
    #'BothCM_CH10RP6':code_BothCM_CH10RP6,
    'kera':code_kera,
    'noob':code_noob,
    'spinoza13':code_spinoza13,
    'spinoza23':code_spinoza23,
 #   'pi':code_pi,
    #'sunocos':code_sunocos
    'rf':code_rf,
    'ant':code_ant
}

def run(observation, configuration):
    global score_so_far
    global hist
    global opp_preds
    global anti_opp_preds
    global my_ggs
    global anti_ggs
    global my_codes
    global true_output
    global str_output
    
    l = []
    
    def overwrite(ggs, the_overwrite):
      #  if ggs['deepbot']['input'] == '':
      #      ggs['deepbot']['output'] = the_overwrite
      #      ggs['deepbot']['prediction'] = the_overwrite
      #  elif len(ggs['deepbot']['history']) < 16:
      #      ggs['deepbot']['output'] = the_overwrite
      #      ggs['deepbot']['prediction'] = the_overwrite
      #  elif ggs['deepbot']['counter'] < 100:
      #      ggs['deepbot']['output'] = the_overwrite
      #      ggs['deepbot']['prediction'] = the_overwrite
      #  else:
      #      ggs['deepbot']['output'] = the_overwrite
      #      ggs['deepbot']['last'] = the_overwrite

        ggs['mutation']['output'] = the_overwrite
        ggs['diludefense']['output'] = the_overwrite
        
        ggs['dumbneural4']['output'] = the_overwrite
        ggs['dumbneural4']['ourmoves'] = ggs['dumbneural4']['ourmoves'][:-1]+the_overwrite
        
        ggs['puppy1003']['output'] = the_overwrite
        
        ggs['neural12']['output'] = the_overwrite
        ggs['neural12']['rps'].my_last_letter = ggs['neural12']['rps'].letter2move[the_overwrite]
        
        ggs['puppy16H']['output'] = the_overwrite
        ggs['puppy17']['output'] = the_overwrite
        ggs['roseguadeloupe']['output'] = the_overwrite
        ggs['randomdropswitch']['output'] = the_overwrite
       # ggs['omelettedeimos']['output'] = the_overwrite
        ggs['rps_meta_fixed']['output'] = the_overwrite
        ggs['rps_meta_fixed2']['output'] = the_overwrite
        ggs['rps_meta_fixed3']['output'] = the_overwrite
        ggs['rps_meta_fixed4']['output'] = the_overwrite
        ggs['rps_meta_fixed5']['output'] = the_overwrite
        ggs['rps_meta_fixed6']['output'] = the_overwrite
        ggs['rps_meta_fixed7']['output'] = the_overwrite
        ggs['rps_meta_fixed8']['output'] = the_overwrite
        ggs['rps_meta_fixed9']['output'] = the_overwrite
        ggs['rps_meta_fix']['output'] = the_overwrite
        ggs['io2fighting']['output'] = the_overwrite
        ggs['bayes14']['output'] = the_overwrite
        ggs['skipblenderfixed']['output'] = the_overwrite
        ggs['ZASMC']['output'] = the_overwrite
        
        ggs['flight825']['output'] = the_overwrite
        ggs['flight825']['hist'] = ggs['flight825']['hist'][:-1]+the_overwrite
        
   #     ggs['funbots']['output'] = the_overwrite
   #     ggs['funbots']['hist'] = ggs['funbots']['hist'][:-1]+the_overwrite
        
        ggs['decay6']['output'] = the_overwrite
        
        ggs['derp']['output'] = the_overwrite
        ggs['derp']['my_prev'] = the_overwrite
        
        #ggs['metaioc']['output'] = the_overwrite
        
 #       ggs['german']['output'] = the_overwrite
 #       ggs['german']['yo'] = {'R':0,'S':1,'P':2}[the_overwrite]
    
        ggs['notorious']['output'] = the_overwrite
        ggs['notorious']['my_prev'] = the_overwrite
        
        #ggs['john2']['output'] = the_overwrite
        
        ggs['cereal4']['output'] = the_overwrite
        if ggs['cereal4']['rnd']==1.1:
            ggs['cereal4']['mylastmove'] = the_overwrite
            
        ggs['zdgsteal']['output'] = the_overwrite
        
        ggs['ZCC2']['output'] = the_overwrite
        ggs['ZCC2']['my_hands'] = ggs['ZCC2']['my_hands'][:-1]+[ggs['ZCC2']['to_num'](the_overwrite)]
        
        ggs['ZMH3']['output'] = the_overwrite
        ggs['zaibayes']['output'] = the_overwrite
        ggs['zaimarkov']['output'] = the_overwrite
        ggs['zaidfa3']['output'] = the_overwrite
        ggs['lucker']['output'] = the_overwrite
        ggs['dilu1']['output'] = the_overwrite
        
        #ggs['iocaine']['act'] = {'R':0,'P':1,'S':2}[the_overwrite]
        #ggs['iocaine']['output'] = the_overwrite
        #ggs['iocaine']['iocaine'].histories[0][-1] = ggs['iocaine']['act']
        
        ggs['greenberg']['act'] = {'R':0,'P':1,'S':2}[the_overwrite]
        ggs['greenberg']['output'] = the_overwrite

        if len(ggs['greenberg']['my_hist']) > 0:
            ggs['greenberg']['my_hist'][-1] = ('rock','paper','scissors')[ggs['greenberg']['act']]
            
        ggs['rps30']['output'] = the_overwrite
     #   ggs['dsad']['output'] = the_overwrite
        #ggs['copymodel']['output'] = the_overwrite
        #ggs['copymodel']['uval'] = {'R':0,'P':1,'S':2}[the_overwrite]
        #ggs['BothCM_CH10RP6']['output'] = the_overwrite
        #ggs['BothCM_CH10RP6']['lval'] = {'R':0,'P':1,'S':2}[the_overwrite]
        ggs['kera']['output'] = the_overwrite # https://web.archive.org/web/20190810194336/http://www.rpscontest.com/entry/5639510700326912
        ggs['kera']['oldOutput'] = the_overwrite
        ggs['noob']['output'] = the_overwrite
        ggs['spinoza13']['output'] = the_overwrite
        ggs['spinoza23']['output'] = the_overwrite
        #ggs['pi']['output'] = the_overwrite
        #ggs['sunocos']['output'] = the_overwrite
        ggs['rf']['output'] = the_overwrite
        ggs['rf']['actions'][-1] = {'R':0,'P':1,'S':2}[the_overwrite]
        ggs['ant']['output'] = the_overwrite
        ggs['ant']['agent'].my_hist[-1] = {'R':0,'P':1,'S':2}[the_overwrite]
        
        return ggs
    
    def score_corey(list_of_dicts, hist):
        coreys_score = 0
        for coreysidx, opp_move in enumerate(hist):
            if opp_move == 0:
                coreys_score += (list_of_dicts[coreysidx]['P'] - list_of_dicts[coreysidx]['S'])
            elif opp_move == 1:
                coreys_score += (list_of_dicts[coreysidx]['S'] - list_of_dicts[coreysidx]['R'])
            else:
                coreys_score += (list_of_dicts[coreysidx]['R'] - list_of_dicts[coreysidx]['P'])
        return coreys_score
    
    my_scores = {}
    for strat in opp_preds.keys():
        my_scores[strat] = -1000
        my_scores['Coreyanti_'+strat] = -1000
        
    my_scores_prob = {}
    for strat in opp_preds.keys():
        my_scores_prob[strat] = -1000
        
    hand_votes = np.array([0,0,0]) # Corresponding to R, P, S

    inp = ''
    reverse_inp = ''
    try:
        inp = 'RPS'[observation.lastOpponentAction]
        reverse_inp = str_output
        hist.append(observation.lastOpponentAction)
        if ((inp == 'R' and str_output == 'S') or # If opponent played rock and I played Scissors
         (inp == 'P' and str_output == 'R') or # If opponent played paper and I played rock
         (inp == 'S' and str_output == 'P')):
            score_so_far -= 1
        elif ((inp == 'R' and str_output == 'P') or # If opponent played rock and I played Paper
         (inp == 'P' and str_output == 'S') or # If opponent played paper and I played Scissors
         (inp == 'S' and str_output == 'R')):
            score_so_far += 1
    except:
        pass
    
    # Loop through your strategies and get their output and historical score
    global_best_score = -1000
    global_best_strat = 'undecided'
    global_best_score_prob = -1000
    global_best_strat_prob = 'undecided'
    
    if inp!='':
        anti_ggs = overwrite(anti_ggs, inp)
    
    for strat in opp_preds.keys():
       # print(strat)
        my_ggs[strat]['input'] = inp
        anti_ggs[strat]['input'] = reverse_inp
        starttime = datetime.datetime.now()
        exec(my_codes[strat], my_ggs[strat])
        exec(my_codes[strat], anti_ggs[strat])
        endtime = datetime.datetime.now()
        elapsed_time = (endtime - starttime).seconds + (endtime - starttime).microseconds/1000000
        if elapsed_time > 0.1:
            print('Longtime:',strat,elapsed_time)
        opp_pred = (({'R': 0, 'P': 1, 'S': 2}[my_ggs[strat]['output']] + 5) % 3)
        opp_preds[strat].append(opp_pred)
        anti_opp_pred = {'R':0,'P':1,'S':2}[anti_ggs[strat]['output']]
        anti_opp_preds[strat].append(anti_opp_pred)
        cp = my_ggs[strat]['COREYSPROBS']
        minval = min(cp[key] for key in cp.keys())
        if minval < 0:
            for key in cp.keys():
                cp[key] += (abs(minval)+1)
        for key in 'RPS':
            if key not in cp.keys():
                cp[key] = 0
        sumvals = sum(cp[key] for key in cp.keys())
        if sumvals == 0:
            cp = {'R':1/3,'P':1/3,'S':1/3}
        else:
            cp = {'R':cp['R']/sumvals,'P':cp['P']/sumvals,'S':cp['S']/sumvals}
            
        my_preds_prob[strat].append(cp)
        
        best_score = my_scores[strat]
        if len(hist)!=0:
            score = np.sum(np.equal(hist, opp_preds[strat][:-1])[-5:]) - np.sum(np.equal(hist, [(j+2)%3 for j in opp_preds[strat][:-1]])[-5:])
            anti_score = np.sum(np.equal(hist, anti_opp_preds[strat][:-1])[-5:]) - np.sum(np.equal(hist, [(j+2)%3 for j in anti_opp_preds[strat][:-1]])[-5:])
            score_prob = score_corey(list_of_dicts=my_preds_prob[strat][:-1][-5:], hist=hist[-5:])
            my_scores[strat] = score
            my_scores['Coreyanti_'+strat] = anti_score
            my_scores_prob[strat] = score_prob
            if score > best_score:
                best_score = score
            if anti_score > best_score:
                best_score = anti_score
                
            if score_prob >= 0.01:
                l.append((strat,score_prob))
                the_score = score_prob * 1.0
                if 'rps_meta_fixed' in strat:
                    the_score /= 3
                elif 'spinoza' in strat:
                    the_score /= 2
                hand_votes[0] += (the_score * cp['R'])
                hand_votes[1] += (the_score * cp['P'])
                hand_votes[2] += (the_score * cp['S'])
            
        if my_scores[strat] >= global_best_score:
            global_best_strat = strat
            global_best_score = my_scores[strat]
        if my_scores['Coreyanti_'+strat] >= global_best_score:
            global_best_strat = 'Coreyanti_'+strat
            global_best_score = my_scores['Coreyanti_'+strat]
    
    #print(my_scores, '\nScore so far:',score_so_far)
    print('Step:',len(hist),'Score so far:',score_so_far,'global best strat&score:',global_best_strat,global_best_score)
    if global_best_score >= 5:
        print('Using global_best_strat&score:',global_best_strat,global_best_score)
        if 'Coreyanti_' in global_best_strat:
            true_output = ({'R': 0, 'P': 1, 'S': 2}[anti_ggs[global_best_strat[10:]]['output']] + 1) % 3
        else:
            true_output = ({'R': 0, 'P': 1, 'S': 2}[my_ggs[global_best_strat]['output']]) % 3
    else:
        #if len(hist)!=0:
        #    print('Used strats:',l)
        sum_hands = np.sum(hand_votes)
        if sum_hands == 0:
            hand_votes = np.array([1,1,1])
            sum_hands = 3
        hand_votes = hand_votes / sum_hands
        true_output = int(np.random.choice([0,1,2],1,p=hand_votes)[0])
        print('Hand weights:',hand_votes,'choosing output:',true_output)
    
#     if 'Coreyanti_' in global_best_strat:
#         true_output = ({'R': 0, 'P': 1, 'S': 2}[anti_ggs[global_best_strat[10:]]['output']] + 1) % 3
#     else:
#         true_output = ({'R': 0, 'P': 1, 'S': 2}[my_ggs[global_best_strat]['output']]) % 3
    
    
    #if (score_so_far <= -5 and global_best_score < 24 and secrets.choice([0,1])==0) or score_so_far >= 60:
    #    print('uhh')
    #    true_output = secrets.choice([0,1,2])
    if score_so_far >= 60:
        true_output = secrets.choice([0,1,2])
    if score_so_far <= -15 and secrets.choice([0,1])==0:
        true_output = secrets.choice([0,1,2])
    elif score_so_far <= -12 and secrets.choice([0,1,2])==0:
        true_output = secrets.choice([0,1,2])
    elif score_so_far <= -9 and secrets.choice([0,1,2,3])==0:
        true_output = secrets.choice([0,1,2])
    elif score_so_far <= -6 and secrets.choice([0,1,2,3,4])==0:
        true_output = secrets.choice([0,1,2])
    elif score_so_far <= -3 and secrets.choice([0,1,2,3,4,5])==0:
        true_output = secrets.choice([0,1,2])
    elif score_so_far <= 0 and secrets.choice([0,1,2,3,4,5,6])==0:
        true_output = secrets.choice([0,1,2])
   # if len(hist) <= 499 or secrets.choice([0,1,2,3,4])==0:
   #     true_output = secrets.choice([0,1,2])
    str_output = {0:'R',1:'P',2:'S'}[true_output]
    
    my_ggs = overwrite(my_ggs, str_output)
            
    return true_output


In [ ]:
%%writefile rps_meta_fix_meta.py
code = compile(
    """
import random

RNA={'RR':'1','RP':'2','RS':'3','PR':'4','PP':'5','PS':'6','SR':'7','SP':'8','SS':'9'}
mix={'RR':'R','RP':'R','RS':'S','PR':'R','PP':'P','PS':'P','SR':'S','SP':'P','SS':'S'}
rot={'R':'P','P':'S','S':'R'}

if not input:
   DNA=[""]*3
   prin=[random.choice("RPS")]*18
   meta=[random.choice("RPS")]*6
   skor1=[[0]*18,[0]*18,[0]*18,[0]*18,[0]*18,[0]*18]
   skor2=[0]*6
else:
   for j in range(18):
       for i in range(4):
           skor1[i][j]*=0.8
       for i in range(4,6):
           skor1[i][j]*=0.5
       for i in range(0,6,2):
           skor1[i][j]-=(input==rot[rot[prin[j]]])
           skor1[i+1][j]-=(output==rot[rot[prin[j]]])
       for i in range(2,6,2):
           skor1[i][j]+=(input==prin[j])
           skor1[i+1][j]+=(output==prin[j])
       skor1[0][j]+=1.3*(input==prin[j])-0.3*(input==rot[prin[j]])
       skor1[1][j]+=1.3*(output==prin[j])-0.3*(output==rot[prin[j]])
   for i in range(6):
       skor2[i]=0.9*skor2[i]+(input==meta[i])-(input==rot[rot[meta[i]]])
   DNA[0]+=input
   DNA[1]+=output
   DNA[2]+=RNA[input+output]
   for i in range(3):
       j=min(21,len(DNA[2]))
       k=-1
       while j>1 and k<0:
             j-=1
             k=DNA[i].rfind(DNA[i][-j:],0,-1)
       prin[2*i]=DNA[0][j+k]
       prin[2*i+1]=rot[DNA[1][j+k]]
       k=DNA[i].rfind(DNA[i][-j:],0,j+k-1)
       prin[2*i]=mix[prin[2*i]+DNA[0][j+k]]
       prin[2*i+1]=mix[prin[2*i+1]+rot[DNA[1][j+k]]]
   for i in range(6,18):
       prin[i]=rot[prin[i-6]]
   for i in range(0,6,2):
       meta[i]=prin[skor1[i].index(max(skor1[i]))]
       meta[i+1]=rot[prin[skor1[i+1].index(max(skor1[i+1]))]]
output=rot[meta[skor2.index(max(skor2))]]
""", '<string>', 'exec')
gg = {}
hist = []
rps_meta_fix_opp_preds = []
import numpy as np

def run(observation, configuration):
    global gg
    global code
    global hist
    global rps_meta_fix_opp_preds
    inp = ''
    try:
        inp = 'RPS'[observation.lastOpponentAction]
        hist.append(observation.lastOpponentAction)
    except:
        pass
    gg['input'] = inp
    exec(code, gg)
    rps_meta_fix_opp_preds.append(({'R': 0, 'P': 1, 'S': 2}[gg['output']] + 5) % 3)
    best_shift = 0
    best_score = -1
    if len(hist)!=0:
        for shift in [0,1,2]:
            score = np.sum(np.equal(hist, [(r+shift)%3 for r in rps_meta_fix_opp_preds[:-1]]))
            if score > best_score:
                best_shift = shift
                best_score = score
    return ({'R': 0, 'P': 1, 'S': 2}[gg['output']] + best_shift) % 3

In [ ]:
from kaggle_environments import make, evaluate

env = make("rps", configuration={"episodeSteps": 1000},debug=True)
#env.run(["ensemble_v3.py", "rps_meta_fix.py"])
env.run(["ensemble_v3.py", "rps_meta_fix_meta.py"])